In [3]:
#!/usr/bin/env python3
"""
Example configuration file for PFN training.
This is a Hebo+ prior configuration, as found in the PFNs4BO paper.
This file demonstrates how to configure the MainConfig for training using Python.
"""

import math
from math import log, sqrt

import torch
from pfns.model import bar_distribution
from pfns.model.encoders import EncoderConfig, StyleEncoderConfig
from pfns.priors.hyperparameter_sampling import ChoiceDistConfig, UniformFloatDistConfig
from pfns.priors.prior import AdhocPriorConfig
from pfns.train import (
    BatchShapeSamplerConfig,
    MainConfig,
    OptimizerConfig,
    TransformerConfig,
)
from pfns.utils import product_dict

from tqdm import tqdm

config_dicts = product_dict(
    {
        "sampled_hp_prior": [False],
        "emsize": [256],
        "nlayers": [12],
        "epochs": [200],
        "lr": [2e-4],
        "batch_size": [256],
        "batch_size_per_gp_sample": [8],
        "num_workers": [6],  # while more workers would be good they lead to segfaults
        "max_num_features": [18, 30],
        "features_per_group": [1, 2],
        "sample_clustered_x": [True, False],
    }
)


def get_config(config_index: int):
    config_dict = list(config_dicts)[config_index]

    sampled_hp_prior = config_dict["sampled_hp_prior"]
    emsize = config_dict["emsize"]
    epochs = config_dict["epochs"]
    lr = config_dict["lr"]
    nlayers = config_dict["nlayers"]
    batch_size = config_dict["batch_size"]
    num_workers = config_dict["num_workers"]
    max_num_features = config_dict["max_num_features"]

    steps_per_epoch = 1000

    def get_prior_config(sampled_hp_prior=sampled_hp_prior, plotting=False):
        hyperparameters = {
            "mean_width": 2.0,
            "lengthscale_loc_constant_add": 0.,#sqrt(2),
            "lengthscale_loc_feature_mul": 0.5,
            "lengthscale_scale": sqrt(3),
            "noise_var_loc": -4.0,
            "noise_var_scale": 1.0,
            "noisy_predictions": False,
            "sample_clustered_x": config_dict["sample_clustered_x"],
        }
        if sampled_hp_prior:
            hyperparameters.update(
                {
                    "hyperparameter_sampling_add_hps_to_style": "all_sampled",
                    "hyperparameter_sampling_skip_style_prob": 0.1,
                    # "num_hyperparameter_samples_per_batch": -1,
                }
            )

            hyperparameters["mean_width"] = UniformFloatDistConfig(0.01, 4.0)
            hyperparameters["lengthscale_loc_constant_add"] = UniformFloatDistConfig(
                1, 2
            )
            hyperparameters["lengthscale_loc_feature_mul"] = UniformFloatDistConfig(
                0.1, 1.0
            )
            hyperparameters["lengthscale_scale"] = UniformFloatDistConfig(0.1, 2.0)
            hyperparameters["noise_var_loc"] = UniformFloatDistConfig(-7.0, -3.0)
            hyperparameters["noise_var_scale"] = UniformFloatDistConfig(0.1, 3.0)
            hyperparameters["noisy_predictions"] = ChoiceDistConfig([False, True])

        prior_config = AdhocPriorConfig(
            prior_names=["singletaskgp"]
            + (["hyperparameter_sampling"] if sampled_hp_prior else []),
            # while the strings are more ugly, they allow for reusable configs
            # get_batch_methods=[pfns.priors.singletaskgp.get_batch]
            # + (
            #     [pfns.priors.hyperparameter_sampling.get_batch]
            #     if sampled_hp_prior
            #     else []
            # ),
            prior_kwargs={
                "num_features": 1 if plotting else max_num_features,
                "hyperparameters": {**hyperparameters},
                "batch_size_per_gp_sample": config_dict["batch_size_per_gp_sample"],
            },
        )
        return prior_config, hyperparameters

    prior_config, hps = get_prior_config(sampled_hp_prior=sampled_hp_prior)

    gb = prior_config.create_get_batch_method()

    ys = []
    for num_features in tqdm(list(range(1, max_num_features)) * 10):
        ys.append(
            gb(batch_size=32, seq_len=100, num_features=num_features).target_y.flatten()
        )

    ys = torch.cat(ys)
    print(f"{len(ys)=}")

    borders = bar_distribution.get_bucket_borders(5000, ys=ys)

    return MainConfig(
        priors=[prior_config],
        optimizer=OptimizerConfig("adamw", lr=lr, weight_decay=0.0),
        scheduler="cosine_decay",
        model=TransformerConfig(
            criterion=bar_distribution.BarDistributionConfig(
                borders.tolist(), full_support=True
            ),
            emsize=emsize,
            nhead=emsize // 32,
            nhid=emsize * 4,
            nlayers=nlayers,
            encoder=EncoderConfig(
                variable_num_features_normalization=True,
                constant_normalization_mean=0.5,
                constant_normalization_std=1 / math.sqrt(12),
                hidden_size=emsize * 2,
            ),
            y_encoder=EncoderConfig(
                nan_handling=True,
                hidden_size=emsize * 2,
            ),
            attention_between_features=True,
            features_per_group=config_dict["features_per_group"],
            style_encoder=StyleEncoderConfig(normalize_to_hyperparameters=hps)
            if sampled_hp_prior
            else None,
            y_style_encoder=StyleEncoderConfig(normalize_to_hyperparameters=hps)
            if sampled_hp_prior
            else None,
        ),
        batch_shape_sampler=BatchShapeSamplerConfig(
            batch_size=batch_size,
            max_seq_len=60,
            fixed_num_test_instances=10,
            max_num_features=max_num_features,
        ),
        epochs=epochs,
        warmup_epochs=epochs // 10,
        steps_per_epoch=steps_per_epoch,
        num_workers=num_workers,
        train_mixed_precision=True,
        verbose=True,
        n_targets_per_input=32,
    )


# View with: tensorboard --logdir=runs


In [4]:
c = get_config(0)
gb = c.priors[0].create_get_batch_method()

ModuleNotFoundError: No module named 'pfns.priors.singletaskgp'

In [5]:
import matplotlib.pyplot as plt

monotonic_dims = 0
increase_shares = []
maxs_on_border = 0

bs = 32

for num_features in range(1,18):
    b = gb(bs, 1000, num_features=num_features, hyperparameters={
            "mean_width": 2.0,
            "lengthscale_loc_constant_add": 5*sqrt(2),
            "lengthscale_loc_feature_mul": 0.9,
            "lengthscale_scale": sqrt(3),
            "noise_var_loc": -4.0,
            "noise_var_scale": 1.0,
            "noisy_predictions": False,
            "sample_clustered_x": False,
        })
        
    for d in range(b.x.shape[-1]):
        # Check for monotonicity in b.target_y for points next to each other in one dimension of b.x
        sorted_indices = b.x[:, :, d].argsort(dim=1)
        sorted_target_y = b.target_y.squeeze().gather(dim=1, index=sorted_indices)
        increases = (sorted_target_y.diff(dim=1) >= 0).all(1) | (sorted_target_y.diff(dim=1) <= 0).all(1)
        increase_shares += [(sorted_target_y.diff(dim=1) >= 0).sum(1) / b.x.shape[1]]
        monotonic_dims += increases.sum()
        if num_features == 1:
            for i in range(len(b.x)):
                plt.plot(b.x[i,:,0][sorted_indices[i]], sorted_target_y[i])
            plt.show()

    
    max_i = b.target_y.squeeze(2).max(1).indices
    max_x = b.x[torch.arange(len(max_i)), max_i]
    mins = b.x.min(1).values
    maxs = b.x.max(1).values

    is_max_or_min_on_border = (max_x == mins) | (max_x == maxs)
    maxs_on_border += is_max_or_min_on_border.any(1).sum()
    #print(max_x.shape, mins.shape)


print(monotonic_dims.sum(), maxs_on_border / bs / 17)

NameError: name 'gb' is not defined

In [19]:
#don't delete

import numpy as np
import matplotlib.pyplot as plt
on_border_prob = np.array(border_styles)
on_border = np.array(on_border)
on_border_prob[on_border].mean(), on_border_prob[~on_border].mean()
#plt.scatter(on_border_prob, on_border)

#order = torch.argsort(on_border_prob)

for b in [.1*i for i in range(1,10)]:
    mask = (on_border_prob < b) & (on_border_prob >= b - .1)
    print(b, on_border[mask].mean(), mask.sum())

0.1 0.001105583195135434 1809
0.2 0.007926829268292683 1640
0.30000000000000004 0.00327653997378768 1526
0.4 0.022332506203473945 1209
0.5 0.02333931777378815 1114
0.6000000000000001 0.04157549234135667 914
0.7000000000000001 0.03631647211413749 771
0.8 0.0903954802259887 531
0.9 0.1419939577039275 331


In [23]:
torch.tensor(1.)/ 10e40,

(tensor(0.),)

In [7]:
border_styles = [0.16381055116653442, 0.24149799346923828, 0.12614339590072632, 0.38972777128219604, 0.845837414264679, 0.37630385160446167, 0.11200207471847534, 0.2105833888053894, 0.07912790775299072, 0.03244072198867798, 0.05746293067932129, 0.1688237190246582, 0.22788727283477783, 0.6407415270805359, 0.47361844778060913, 0.10653889179229736, 0.836825966835022, 0.9348931908607483, 0.2537940740585327, 0.7761909365653992, 0.40612149238586426, 0.9376307129859924, 0.46556729078292847, 0.1845737099647522, 0.30017775297164917, 0.4779350161552429, 0.7340158820152283, 0.5655716061592102, 0.5625150203704834, 0.3197460174560547, 0.6779114007949829, 0.0572352409362793, 0.581635057926178, 0.4075837731361389, 0.49234873056411743, 0.38559281826019287, 0.03760933876037598, 0.7349210977554321, 0.5114566087722778, 0.12343817949295044, 0.3823511004447937, 0.052324533462524414, 0.18587273359298706, 0.4648723602294922, 0.6535191535949707, 0.041318297386169434, 0.25734156370162964, 0.0679129958152771, 0.49494296312332153, 0.0629204511642456, 0.8617863059043884, 0.16568422317504883, 0.05618816614151001, 0.0003210902214050293, 0.06036794185638428, 0.009161174297332764, 0.3744446635246277, 0.09215331077575684, 0.5905824899673462, 0.38493984937667847, 0.10753190517425537, 0.502727746963501, 0.814067542552948, 0.5676053166389465, 0.673051118850708, 0.09694105386734009, 0.1380467414855957, 0.20345956087112427, 0.15356707572937012, 0.8246403932571411, 0.5042554140090942, 0.3380228281021118, 0.3166341185569763, 0.5424852967262268, 0.6007550358772278, 0.11984962224960327, 0.18681836128234863, 0.09630852937698364, 0.5697934031486511, 0.11620056629180908, 0.52690589427948, 0.17356055974960327, 0.16679507493972778, 0.1272716522216797, 0.22643452882766724, 0.053492963314056396, 0.2664504051208496, 0.2671971321105957, 0.48030298948287964, 0.035093486309051514, 0.23919719457626343, 0.4694904685020447, 0.10732501745223999, 0.27740758657455444, 0.20080959796905518, 0.5544178485870361, 0.4703555703163147, 0.3242875337600708, 0.5811861157417297, 0.16922587156295776, 0.1165735125541687, 0.21780413389205933, 0.293582558631897, 0.5941606760025024, 0.05096536874771118, 0.38201236724853516, 0.1761130690574646, 0.572944700717926, 0.8831941485404968, 0.7131157517433167, 0.5649142861366272, 0.3235289454460144, 0.4011092782020569, 0.0017268061637878418, 0.10023921728134155, 0.10887753963470459, 0.33952128887176514, 0.45478856563568115, 0.7336657643318176, 0.10871553421020508, 0.03490787744522095, 0.8861747980117798, 0.5847815871238708, 0.2970042824745178, 0.24511367082595825, 0.4510957598686218, 0.05629432201385498, 0.438732385635376, 0.1105988621711731, 0.5124863982200623, 0.07962745428085327, 0.4732515811920166, 0.2315496802330017, 0.024103999137878418, 0.0007070302963256836, 0.6365591287612915, 0.24326515197753906, 0.7264488935470581, 0.40101224184036255, 0.084919273853302, 0.49241191148757935, 0.008906662464141846, 0.23136061429977417, 0.3261854648590088, 0.4610177278518677, 0.1559423804283142, 0.193267822265625, 0.05489164590835571, 0.6437252759933472, 0.15923738479614258, 0.5559098124504089, 0.20153766870498657, 0.20878905057907104, 0.5784331560134888, 0.13091790676116943, 0.05152428150177002, 0.07769268751144409, 0.11650532484054565, 0.3792344331741333, 0.0005715489387512207, 0.2479744553565979, 0.281791090965271, 0.06203395128250122, 0.9859344363212585, 0.243341326713562, 0.10022109746932983, 0.20486927032470703, 0.015146911144256592, 0.2213064432144165, 0.12749570608139038, 0.2054632306098938, 0.3965865969657898, 0.04987877607345581, 0.45372170209884644, 0.3372025489807129, 0.34580689668655396, 0.9690937399864197, 0.15951836109161377, 0.18637889623641968, 0.5751436948776245, 0.0827375054359436, 0.014427125453948975, 0.23277682065963745, 0.618951141834259, 0.3413499593734741, 0.25506407022476196, 0.9478741884231567, 0.13350224494934082, 0.09709054231643677, 0.03998619318008423, 0.46328699588775635, 0.28528261184692383, 0.09772521257400513, 0.49785101413726807, 0.578534722328186, 0.2983296513557434, 0.1199454665184021, 0.6867879629135132, 0.4526452422142029, 0.15163546800613403, 0.1662740707397461, 0.19686341285705566, 0.07860416173934937, 0.7203462719917297, 0.3620888590812683, 0.37428784370422363, 0.5665250420570374, 0.5954092741012573, 0.6393340229988098, 0.45144253969192505, 0.30032777786254883, 0.10694772005081177, 0.39831918478012085, 0.518463134765625, 0.39391475915908813, 0.007617533206939697, 0.12675046920776367, 0.09795194864273071, 0.23814207315444946, 0.4647679924964905, 0.0323793888092041, 0.5578628778457642, 0.17821872234344482, 0.112338125705719, 0.07654839754104614, 0.21840250492095947, 0.01307600736618042, 0.0016922354698181152, 0.7005534172058105, 0.39398694038391113, 0.2405051589012146, 0.21034908294677734, 0.7096841931343079, 0.5314376354217529, 0.3141412138938904, 0.7207634449005127, 0.6876961588859558, 0.2158207893371582, 0.11442232131958008, 0.3532000780105591, 0.3768579363822937, 0.23118174076080322, 0.2617281675338745, 0.2722555994987488, 0.20981544256210327, 0.6428701281547546, 0.3360532522201538, 0.148806631565094, 0.1611642837524414, 0.12562799453735352, 0.3465192914009094, 0.21562641859054565, 0.04153645038604736, 0.5715212225914001, 0.4447060823440552, 0.05978125333786011, 0.34895557165145874, 0.3562154769897461, 0.32265138626098633, 0.061141788959503174, 0.5846540927886963, 0.3714209794998169, 0.22560608386993408, 0.2119339108467102, 0.42930668592453003, 0.04369473457336426, 0.8195823431015015, 0.17736124992370605, 0.12687820196151733, 0.09873735904693604, 0.3616759777069092, 0.7247050404548645, 0.00043964385986328125, 0.508421778678894, 0.12750130891799927, 0.03366076946258545, 0.5675796866416931, 0.033741652965545654, 0.107086181640625, 0.16826295852661133, 0.18216979503631592, 0.7530564069747925, 0.5649333000183105, 0.1086277961730957, 0.5037078261375427, 0.34700703620910645, 0.33220553398132324, 0.2787320017814636, 0.5186879634857178, 0.5619102716445923, 0.31028735637664795, 0.24406063556671143, 0.5682904124259949, 0.18488258123397827, 0.10867917537689209, 0.08300328254699707, 0.3972337245941162, 0.8687431216239929, 0.5811030864715576, 0.37596166133880615, 0.25907444953918457, 0.23336607217788696, 0.12790554761886597, 0.8061492443084717, 0.8528739809989929, 0.34480392932891846, 0.09483623504638672, 0.5524814128875732, 0.3743845820426941, 0.36422693729400635, 0.00953763723373413, 0.33753126859664917, 0.8172246813774109, 0.32738375663757324, 0.8913642168045044, 0.03602677583694458, 0.2062261700630188, 0.5700047016143799, 0.49591726064682007, 0.19016748666763306, 0.01348644495010376, 0.30929672718048096, 0.6811864972114563, 0.05812174081802368, 0.748395562171936, 0.5333366394042969, 0.054125845432281494, 0.06305968761444092, 0.310518741607666, 0.6990670561790466, 0.3409251570701599, 0.20351642370224, 0.07683444023132324, 0.6527494192123413, 0.602104127407074, 0.2792714834213257, 0.006803393363952637, 0.3568136692047119, 0.32635343074798584, 0.3812462091445923, 0.3973410725593567, 0.3267989754676819, 0.12118315696716309, 0.03326374292373657, 0.1015891432762146, 0.5479664206504822, 0.2668203115463257, 0.9614073038101196, 0.2609711289405823, 0.690744936466217, 0.08275008201599121, 0.09723860025405884, 0.2620343565940857, 0.19212281703948975, 0.09966713190078735, 0.27024924755096436, 0.9378544688224792, 0.156624436378479, 0.6969866752624512, 0.19212639331817627, 0.6442466378211975, 0.44718748331069946, 0.1393958330154419, 0.5249231457710266, 0.7671242952346802, 0.0009969472885131836, 0.3024181127548218, 0.7825308442115784, 0.007365107536315918, 0.03719717264175415, 0.48002755641937256, 0.1202692985534668, 0.6389692425727844, 0.5624537467956543, 0.33111101388931274, 0.661314070224762, 0.1336386799812317, 0.6460355520248413, 0.330891489982605, 0.2872328758239746, 0.12929916381835938, 0.5877024531364441, 0.42180103063583374, 0.49580103158950806, 0.19391119480133057, 0.3196016550064087, 0.027955949306488037, 0.11421293020248413, 0.6195122003555298, 0.6163544058799744, 0.12343388795852661, 0.11349213123321533, 0.09732687473297119, 0.6793144345283508, 0.33583855628967285, 0.06397432088851929, 0.47893011569976807, 0.2841976284980774, 0.40818125009536743, 0.44849884510040283, 0.6676055192947388, 0.6294088363647461, 0.18395662307739258, 0.4949548840522766, 0.006292164325714111, 0.18868869543075562, 0.38814789056777954, 0.29109203815460205, 0.625909149646759, 0.3398789167404175, 0.0036492347717285156, 0.42096179723739624, 0.49545925855636597, 0.11115223169326782, 0.07775872945785522, 0.23590153455734253, 0.3808673024177551, 0.7522753477096558, 0.22297769784927368, 0.23010432720184326, 0.13381636142730713, 0.1541028618812561, 0.038177549839019775, 0.014444947242736816, 0.12039607763290405, 0.06172889471054077, 0.13324683904647827, 0.07980287075042725, 0.30411821603775024, 0.3776748776435852, 0.08810776472091675, 0.4180186986923218, 0.618478000164032, 0.2572746276855469, 0.09099668264389038, 0.36228883266448975, 0.8528903126716614, 0.5224374532699585, 0.6284229755401611, 0.17208224534988403, 0.43987512588500977, 0.8777012228965759, 0.8508107662200928, 0.37571948766708374, 0.48302197456359863, 0.444802463054657, 0.11531615257263184, 0.030995309352874756, 0.15286779403686523, 0.6889867782592773, 0.21744948625564575, 0.13660526275634766, 0.7438837885856628, 0.5896667242050171, 0.186109721660614, 0.22063040733337402, 0.18770289421081543, 0.02824503183364868, 0.4228594899177551, 0.23947358131408691, 0.538554310798645, 0.43395334482192993, 0.6689168810844421, 0.08890074491500854, 0.28450095653533936, 0.003695249557495117, 0.20439070463180542, 0.29779237508773804, 0.6506646871566772, 0.7322900891304016, 0.4451025128364563, 0.16899245977401733, 0.6920649409294128, 0.24397093057632446, 0.13960039615631104, 0.20810943841934204, 0.8280943036079407, 0.7327244877815247, 0.29693472385406494, 0.23649024963378906, 0.35192930698394775, 0.38988959789276123, 0.41288673877716064, 0.3016224503517151, 0.12202572822570801, 0.3358871340751648, 0.9468472003936768, 0.16512203216552734, 0.15318506956100464, 0.08891361951828003, 0.36651766300201416, 0.24648189544677734, 0.32978081703186035, 0.7902456521987915, 0.5566412210464478, 0.22325021028518677, 0.12241917848587036, 0.8556107878684998, 0.1596938967704773, 0.28323179483413696, 0.46498727798461914, 0.4453859329223633, 0.018402934074401855, 0.6492279767990112, 0.42906492948532104, 0.19297975301742554, 0.1675175428390503, 0.6167635917663574, 0.23588985204696655, 0.43786484003067017, 0.20574951171875, 0.46614426374435425, 0.2103574275970459, 0.4361923933029175, 0.1637478470802307, 0.550299346446991, 0.4861605167388916, 0.09373503923416138, 0.24611598253250122, 0.6159747242927551, 0.5463578701019287, 0.15971088409423828, 0.18025600910186768, 0.5018539428710938, 0.6210067272186279, 0.11130672693252563, 0.05073225498199463, 0.4068300724029541, 0.6592065095901489, 0.4253033399581909, 0.17273575067520142, 0.6830917596817017, 0.926511287689209, 0.36731410026550293, 0.4652894139289856, 0.5333501696586609, 0.6476345658302307, 0.2922760248184204, 0.39007627964019775, 0.05812263488769531, 0.35463637113571167, 0.7125056982040405, 0.45334309339523315, 0.5890762209892273, 0.38421201705932617, 0.3367358446121216, 0.12010091543197632, 0.3660869002342224, 0.05442994832992554, 0.5609815120697021, 0.2933053970336914, 0.002115190029144287, 0.33665651082992554, 0.556710958480835, 0.4822882413864136, 0.08799415826797485, 0.41325539350509644, 0.05446523427963257, 0.25486719608306885, 0.5621784925460815, 0.6257644891738892, 0.05663019418716431, 0.6852800846099854, 0.2774586081504822, 0.020233511924743652, 0.28565794229507446, 0.4143572449684143, 0.41672348976135254, 0.17018985748291016, 0.6755034923553467, 0.3093356490135193, 0.24190866947174072, 0.4030132293701172, 0.22876280546188354, 0.8570116758346558, 0.331157922744751, 0.03315293788909912, 0.5582804083824158, 0.3729237914085388, 0.01539158821105957, 0.1929314136505127, 0.35578423738479614, 0.3608688712120056, 0.01696878671646118, 0.41630858182907104, 0.15255671739578247, 0.3819766640663147, 0.3135278820991516, 0.14220672845840454, 0.5025705695152283, 0.6727227568626404, 0.21100389957427979, 0.7246488928794861, 0.29745423793792725, 0.3684793710708618, 0.3928588032722473, 0.22064191102981567, 0.5182545781135559, 0.8421460390090942, 0.10649752616882324, 0.5117897391319275, 0.43616509437561035, 0.1027371883392334, 0.3693926930427551, 0.1716090440750122, 0.5413579344749451, 0.5077283978462219, 0.25493568181991577, 0.1590714454650879, 0.07912355661392212, 0.07498061656951904, 0.3821631073951721, 0.6082422137260437, 0.5687893033027649, 0.19569337368011475, 0.0994030237197876, 0.26322561502456665, 0.3312811851501465, 0.1651078462600708, 0.5407848954200745, 0.31673407554626465, 0.3839260935783386, 0.5156182646751404, 0.15142720937728882, 0.6443091630935669, 0.231772780418396, 0.19323086738586426, 0.0019840598106384277, 0.4185575842857361, 0.4143680930137634, 0.1584150195121765, 0.2694942355155945, 0.5563890933990479, 0.007085859775543213, 0.3224261999130249, 0.2413361668586731, 0.10543608665466309, 0.5001564621925354, 0.5914560556411743, 0.18277466297149658, 0.0005397200584411621, 0.5055962204933167, 0.5832539200782776, 0.10184663534164429, 0.03781086206436157, 0.34361934661865234, 0.31678569316864014, 0.0468517541885376, 0.026348114013671875, 0.49033498764038086, 0.10076826810836792, 0.3902111053466797, 0.05431199073791504, 0.5284008383750916, 0.3716884255409241, 0.12080490589141846, 0.3773515820503235, 0.6990373730659485, 0.7317056059837341, 0.208376944065094, 0.2372782826423645, 0.905462920665741, 0.16074711084365845, 0.058524906635284424, 0.09514856338500977, 0.00515669584274292, 0.3241976499557495, 0.025973081588745117, 0.072299063205719, 0.5825644731521606, 0.035694658756256104, 0.3384144902229309, 0.6493951082229614, 0.8114885687828064, 0.47144418954849243, 0.28821229934692383, 0.15684163570404053, 0.12031638622283936, 0.13983869552612305, 0.6446420550346375, 0.16898185014724731, 0.3408104181289673, 0.28377479314804077, 0.9828972816467285, 0.7605352997779846, 0.2699171304702759, 0.43093496561050415, 0.474770724773407, 0.030842900276184082, 0.6361237168312073, 0.2767256498336792, 0.3804149627685547, 0.3825044631958008, 0.16090762615203857, 0.2510949969291687, 0.3348521590232849, 0.7497813701629639, 0.13742977380752563, 0.061478495597839355, 0.25998425483703613, 0.3209695816040039, 0.5868546366691589, 0.05835944414138794, 0.3992128372192383, 0.54140704870224, 0.234103262424469, 0.5876807570457458, 0.1532987356185913, 0.09500503540039062, 0.3865410089492798, 0.08126986026763916, 0.27037543058395386, 0.5141448378562927, 0.2687259912490845, 0.25135523080825806, 0.15961170196533203, 0.407662034034729, 0.5418782830238342, 0.11981219053268433, 0.12955915927886963, 0.17572355270385742, 0.8013582229614258, 0.5577786564826965, 0.5335747599601746, 0.4532129764556885, 0.059731364250183105, 0.2907724380493164, 0.374860942363739, 0.8447008728981018, 0.042873263359069824, 0.5164450407028198, 0.4897843599319458, 0.03538024425506592, 0.4362555146217346, 0.6497281789779663, 0.9350144863128662, 0.4695715308189392, 0.7820791006088257, 0.06556427478790283, 0.144090473651886, 0.06853675842285156, 0.7432471513748169, 0.21464425325393677, 0.016920089721679688, 0.3257683515548706, 0.27395856380462646, 0.20488518476486206, 0.28593868017196655, 0.22649353742599487, 0.02201300859451294, 0.34977561235427856, 0.05783843994140625, 0.05743861198425293, 0.21532052755355835, 0.7887174487113953, 0.31797945499420166, 0.2943602204322815, 0.4952177405357361, 0.2490505576133728, 0.2754558324813843, 0.22152596712112427, 0.13586348295211792, 0.679214596748352, 0.4403287172317505, 0.5351839661598206, 0.3312808871269226, 0.08178955316543579, 0.6680355668067932, 0.6281073689460754, 0.019693851470947266, 0.7953809499740601, 0.6916069388389587, 0.11629176139831543, 0.6572016477584839, 0.6688319444656372, 0.27259600162506104, 0.6048845052719116, 0.017783403396606445, 0.28147298097610474, 0.23694157600402832, 0.4556201100349426, 0.6253647804260254, 0.9206182956695557, 0.22951704263687134, 0.09756994247436523, 0.6078963279724121, 0.3457792401313782, 0.012008309364318848, 0.34898841381073, 0.4725223779678345, 0.14416754245758057, 0.17963248491287231, 0.6879709362983704, 0.3817140460014343, 0.3832166790962219, 0.17171692848205566, 0.7499545216560364, 0.14339792728424072, 0.18658030033111572, 0.5735201835632324, 0.3650626540184021, 0.7411714196205139, 0.4641284942626953, 0.2888093590736389, 0.20990794897079468, 0.1080390214920044, 0.3095606565475464, 0.8535628914833069, 0.06151062250137329, 0.029298722743988037, 0.33550798892974854, 0.432125985622406, 0.10321182012557983, 0.07790946960449219, 0.454123854637146, 0.2840835452079773, 0.4142741560935974, 0.35706549882888794, 0.7629685401916504, 0.3407793641090393, 0.20580995082855225, 0.04521310329437256, 0.396459698677063, 0.4493439197540283, 0.10737067461013794, 0.2905094623565674, 0.15041857957839966, 0.23381847143173218, 0.031073927879333496, 0.1417199969291687, 0.28652602434158325, 0.13691115379333496, 0.45665937662124634, 0.6321154832839966, 0.2053336501121521, 0.06676983833312988, 0.15144133567810059, 0.23798060417175293, 0.28371328115463257, 0.5057846307754517, 0.1421028971672058, 0.05161309242248535, 0.698321521282196, 0.0838354229927063, 0.2819502353668213, 0.23275303840637207, 0.41864413022994995, 0.3856699466705322, 0.45582276582717896, 0.023777008056640625, 0.46711593866348267, 0.20068126916885376, 0.6264737844467163, 0.1717631220817566, 0.9407885074615479, 0.6020733714103699, 0.13490694761276245, 0.6617002487182617, 0.5121736526489258, 0.700495719909668, 0.7144150137901306, 0.08998799324035645, 0.4844059944152832, 0.012812674045562744, 0.4413227438926697, 0.12442058324813843, 0.0005739927291870117, 0.13041937351226807, 0.20581823587417603, 0.048104166984558105, 0.32284456491470337, 0.1382288932800293, 0.09365808963775635, 0.1774420142173767, 0.5353729128837585, 0.05651170015335083, 0.21533989906311035, 0.26441019773483276, 0.11649757623672485, 0.8093169927597046, 0.3576051592826843, 0.32226336002349854, 0.7873736619949341, 0.2065379023551941, 0.2646005153656006, 0.5661088824272156, 0.14225900173187256, 0.07652562856674194, 0.3012810945510864, 0.6566038727760315, 0.48658818006515503, 0.2780447006225586, 0.12758618593215942, 0.31504398584365845, 0.07190108299255371, 0.5896049737930298, 0.6445514559745789, 0.5546543002128601, 0.09326714277267456, 0.2554432153701782, 0.11081355810165405, 0.35907524824142456, 0.3897120952606201, 0.711703360080719, 0.4937788248062134, 0.043765246868133545, 0.27832382917404175, 0.8818793296813965, 0.6130228042602539, 0.20154893398284912, 0.009523570537567139, 0.414780855178833, 0.22385239601135254, 0.0590825080871582, 0.0619964599609375, 0.6956322193145752, 0.029624879360198975, 0.5988569855690002, 0.49437206983566284, 0.0326005220413208, 0.09130251407623291, 0.13752299547195435, 0.3651229739189148, 0.7059892416000366, 0.7893807888031006, 0.8043603897094727, 0.3893231749534607, 0.6724768877029419, 0.10610353946685791, 0.15633141994476318, 0.4584531784057617, 0.18562471866607666, 0.2577930688858032, 0.6174575686454773, 0.18498504161834717, 0.45901262760162354, 0.5785747170448303, 0.28672081232070923, 0.5280101895332336, 0.6783703565597534, 0.5404062867164612, 0.4874008893966675, 0.11694455146789551, 0.8205033540725708, 0.051177978515625, 0.2935827374458313, 0.32080453634262085, 0.493247389793396, 0.32614099979400635, 0.48258358240127563, 0.0292360782623291, 0.151569664478302, 0.5065492391586304, 0.20408415794372559, 0.6647379398345947, 0.2953682541847229, 0.42270004749298096, 0.5588909387588501, 0.6472219824790955, 0.24453628063201904, 0.07859170436859131, 0.8576555252075195, 0.37489694356918335, 0.42789530754089355, 0.07351773977279663, 0.6577102541923523, 0.6119503378868103, 0.24617964029312134, 0.6441767811775208, 0.25046515464782715, 0.06271350383758545, 0.7023924589157104, 0.6046692132949829, 0.8298457860946655, 0.44476091861724854, 0.840423583984375, 0.6829368472099304, 0.6077096462249756, 0.9504132270812988, 0.18615728616714478, 0.48122239112854004, 0.16522181034088135, 0.5675811767578125, 0.3535487651824951, 0.3020027279853821, 0.02277928590774536, 0.520490825176239, 0.30037933588027954, 0.4080130457878113, 0.7886730432510376, 0.06777143478393555, 0.09651005268096924, 0.8352100253105164, 0.7109531164169312, 0.06283700466156006, 0.7253614664077759, 0.0017727017402648926, 0.7462899088859558, 0.6862643361091614, 0.32125937938690186, 0.7585752010345459, 0.48846304416656494, 0.6170090436935425, 0.09357810020446777, 0.5448417067527771, 0.1531272530555725, 0.18548494577407837, 0.13670992851257324, 0.14692258834838867, 0.1458449363708496, 0.23518598079681396, 0.1533890962600708, 0.23838752508163452, 0.7062322497367859, 0.5200967788696289, 0.2560766339302063, 0.7387305498123169, 0.21756452322006226, 0.3688504695892334, 0.09569436311721802, 0.2710888981819153, 0.08677130937576294, 0.007428109645843506, 0.05011779069900513, 0.3421613574028015, 0.22819828987121582, 0.3118687868118286, 0.3017919063568115, 0.5532622337341309, 0.5570541024208069, 0.20768731832504272, 0.4482121467590332, 0.01600039005279541, 0.11501073837280273, 0.0013786554336547852, 0.5306578278541565, 0.7773256897926331, 0.5296761393547058, 0.38237082958221436, 0.36541181802749634, 0.47077101469039917, 0.7793595790863037, 0.6407261490821838, 0.5495519638061523, 0.15888702869415283, 0.8348280787467957, 0.10663557052612305, 0.048209309577941895, 0.5437097549438477, 0.38052111864089966, 0.13593626022338867, 0.33421361446380615, 0.3223887085914612, 0.6225074529647827, 0.6914464831352234, 0.6890602707862854, 0.19904232025146484, 0.02762758731842041, 0.2527616620063782, 0.1885266900062561, 0.7258642911911011, 0.17631995677947998, 0.6263709664344788, 0.5137715935707092, 0.41388416290283203, 0.02017754316329956, 0.2645719051361084, 0.5742102265357971, 0.8626760840415955, 0.2544846534729004, 0.18802565336227417, 0.30425530672073364, 0.3914529085159302, 0.5287913084030151, 0.33274734020233154, 0.6593757271766663, 0.19777458906173706, 0.3123928904533386, 0.789588212966919, 0.03585946559906006, 0.8602823615074158, 0.17224252223968506, 0.5894072651863098, 0.40748363733291626, 0.8314450979232788, 0.7999683022499084, 0.11344587802886963, 0.032553136348724365, 0.2787562608718872, 0.7067510485649109, 0.0061054229736328125, 0.4820958971977234, 0.4827483892440796, 0.011256158351898193, 0.16390460729599, 0.6905714869499207, 0.07162237167358398, 0.19106870889663696, 0.7285386919975281, 0.31890708208084106, 0.6293582916259766, 0.8484393954277039, 0.07676756381988525, 0.27885913848876953, 0.8712777495384216, 0.41089487075805664, 0.39824867248535156, 0.005494475364685059, 0.6714205741882324, 0.4446835517883301, 0.3650304675102234, 0.13342612981796265, 0.1595270037651062, 0.7424696087837219, 0.5893031358718872, 0.11254113912582397, 0.5434311032295227, 0.3512381315231323, 0.7329157590866089, 0.13782846927642822, 0.13039737939834595, 0.4280357360839844, 0.5889268517494202, 0.3817453384399414, 0.24881893396377563, 0.018265843391418457, 0.45048362016677856, 0.690761148929596, 0.4025891423225403, 0.12371307611465454, 0.15053308010101318, 0.19669461250305176, 0.31911468505859375, 0.3288798928260803, 0.10569131374359131, 0.20985478162765503, 0.11143815517425537, 0.5447571873664856, 0.26697081327438354, 0.7484610676765442, 0.6292003393173218, 0.1005893349647522, 0.5567710995674133, 0.7038406133651733, 0.5271602869033813, 0.2035210132598877, 0.15890401601791382, 0.08895206451416016, 0.22728091478347778, 0.44953465461730957, 0.37524062395095825, 0.6797018647193909, 0.02547156810760498, 0.2907746434211731, 0.7155574560165405, 0.3749573230743408, 0.5884278416633606, 0.3709898591041565, 0.023659944534301758, 0.06815016269683838, 0.41941845417022705, 0.18369388580322266, 0.18921512365341187, 0.12400633096694946, 0.6170716285705566, 0.13324016332626343, 0.3922569751739502, 0.1305796504020691, 0.28658348321914673, 0.0024567246437072754, 0.47369807958602905, 0.11922711133956909, 0.15913069248199463, 0.3156996965408325, 0.8696470856666565, 0.2597609758377075, 0.23502492904663086, 0.9023688435554504, 0.3799450397491455, 0.6587518453598022, 0.17331725358963013, 0.24623602628707886, 0.11404907703399658, 0.4391336441040039, 0.9535530805587769, 0.5126557946205139, 0.3122894763946533, 0.5088592171669006, 0.41016459465026855, 0.46555429697036743, 0.07599729299545288, 0.10841983556747437, 0.014408648014068604, 0.5544269680976868, 0.36217135190963745, 0.48380428552627563, 0.030396759510040283, 0.5217747092247009, 0.5494642853736877, 0.7253727912902832, 0.0550265908241272, 0.43715906143188477, 0.38999229669570923, 0.558300256729126, 0.030885815620422363, 0.4638780951499939, 0.36080676317214966, 0.02502286434173584, 0.26857125759124756, 0.26399725675582886, 0.009776532649993896, 0.144589364528656, 0.19868439435958862, 0.4914708137512207, 0.3236010670661926, 0.5412454009056091, 0.8797417283058167, 0.4874356985092163, 0.47694236040115356, 0.9158347249031067, 0.7066726684570312, 0.557233452796936, 0.09702479839324951, 0.3400493860244751, 0.37529444694519043, 0.5933868885040283, 0.03617203235626221, 0.01471555233001709, 0.16899263858795166, 0.5230678915977478, 0.006106197834014893, 0.19435590505599976, 0.3531334400177002, 0.017916083335876465, 0.7587651610374451, 0.4877026677131653, 0.12655246257781982, 0.5812488794326782, 0.2861788272857666, 0.4588688015937805, 0.10641151666641235, 0.5013840794563293, 0.08950620889663696, 0.20010030269622803, 0.5672621130943298, 0.09789812564849854, 0.36555516719818115, 0.09861844778060913, 0.18803554773330688, 0.021228909492492676, 0.21058470010757446, 0.018386006355285645, 0.18682235479354858, 0.4002584218978882, 0.8020928502082825, 0.18099653720855713, 0.6172474026679993, 0.427742063999176, 0.05425471067428589, 0.3506776690483093, 0.21545058488845825, 0.15428727865219116, 0.1072651743888855, 0.43114978075027466, 0.11516791582107544, 0.10192030668258667, 0.14282768964767456, 0.03989899158477783, 0.034992098808288574, 0.6130777597427368, 0.02092134952545166, 0.22381848096847534, 0.8969252705574036, 0.2477920651435852, 0.05020284652709961, 0.7543950080871582, 0.6717394590377808, 0.11821210384368896, 0.40676558017730713, 0.014531075954437256, 0.03969293832778931, 0.013368546962738037, 0.022545933723449707, 0.7001596689224243, 0.5512366890907288, 0.49071943759918213, 0.2715379595756531, 0.2630213499069214, 0.6815502643585205, 0.2382194995880127, 0.20199888944625854, 0.15579700469970703, 0.47200483083724976, 0.301194429397583, 0.18032026290893555, 0.1346815824508667, 0.24857836961746216, 0.24684715270996094, 0.297701358795166, 0.06831127405166626, 0.3977532982826233, 0.6388596892356873, 0.6865886449813843, 0.3675970435142517, 0.5738488435745239, 0.1054641604423523, 0.6084437966346741, 0.18989014625549316, 0.8008061647415161, 0.6166174411773682, 0.3143478035926819, 0.8485716581344604, 0.47784239053726196, 0.9719151854515076, 0.764373242855072, 0.41513943672180176, 0.3147149682044983, 0.9145644307136536, 0.7962819933891296, 0.014785408973693848, 0.35399729013442993, 0.5336095094680786, 0.3589354157447815, 0.1365770697593689, 0.33042997121810913, 0.7947219610214233, 0.6795245409011841, 0.4711645245552063, 0.32862555980682373, 0.1926979422569275, 0.45348095893859863, 0.008914649486541748, 0.3660624623298645, 0.49687135219573975, 0.04781651496887207, 0.628710925579071, 0.03888338804244995, 0.06635379791259766, 0.37374770641326904, 0.8236749768257141, 0.08187741041183472, 0.5420716404914856, 0.648557186126709, 0.09960699081420898, 0.3118136525154114, 0.3674352169036865, 0.2068062424659729, 0.17080044746398926, 0.1932949423789978, 0.548895001411438, 0.15152901411056519, 0.1900048851966858, 0.3705739974975586, 0.577373206615448, 0.6193020939826965, 0.4167100191116333, 0.10306483507156372, 0.19741344451904297, 0.3418329954147339, 0.27942460775375366, 0.19952499866485596, 0.3594388961791992, 0.11362475156784058, 0.22756725549697876, 0.29949063062667847, 0.3779813051223755, 0.21773666143417358, 0.1375446319580078, 0.44716936349868774, 0.1389792561531067, 0.5231781005859375, 0.16013920307159424, 0.6653123497962952, 0.4376258850097656, 0.012628376483917236, 0.34588295221328735, 0.2459983229637146, 0.385265588760376, 0.7418644428253174, 0.10662180185317993, 0.17719662189483643, 0.36377304792404175, 0.5140881538391113, 0.1599695086479187, 0.235154926776886, 0.3330838084220886, 0.9202867746353149, 0.3511176109313965, 0.38968414068222046, 0.01787877082824707, 0.039219021797180176, 0.3302648067474365, 0.4855620265007019, 0.7911632657051086, 0.27417701482772827, 0.4943353533744812, 0.8496120572090149, 0.16702312231063843, 0.1402263641357422, 0.013912796974182129, 0.09941595792770386, 0.24330562353134155, 0.10628610849380493, 0.6157547235488892, 0.5553236603736877, 0.6699509024620056, 0.11320716142654419, 0.02064383029937744, 0.11142027378082275, 0.16957688331604004, 0.2092897891998291, 0.7999997138977051, 0.13093101978302002, 0.4777955412864685, 0.19556260108947754, 0.1983763575553894, 0.1630617380142212, 0.035531699657440186, 0.5452784895896912, 0.48563259840011597, 0.6420955061912537, 0.21646124124526978, 0.4506107568740845, 0.03705239295959473, 0.029616117477416992, 0.6870680451393127, 0.2946687340736389, 0.11972343921661377, 0.5036450624465942, 0.09688472747802734, 0.4963826537132263, 0.5216763615608215, 0.4471483826637268, 0.49063408374786377, 0.7261625528335571, 0.12435352802276611, 0.8061212301254272, 0.47173094749450684, 0.6747047901153564, 0.13704615831375122, 0.577376127243042, 0.46173715591430664, 0.009226799011230469, 0.708961546421051, 0.3364619016647339, 0.16844779253005981, 0.17633122205734253, 0.6928467154502869, 0.7914433479309082, 0.1289687156677246, 0.16699457168579102, 0.4326835870742798, 0.18784856796264648, 0.04110640287399292, 0.45853191614151, 0.0790373682975769, 0.21475166082382202, 0.11624115705490112, 0.48430633544921875, 0.3716633915901184, 0.7604001760482788, 0.17480725049972534, 0.40682679414749146, 0.3268738389015198, 0.25568652153015137, 0.00363314151763916, 0.39255887269973755, 0.25975263118743896, 0.7963920831680298, 0.5906494855880737, 0.1798117756843567, 0.16759634017944336, 0.6599488854408264, 0.17450332641601562, 0.24213480949401855, 0.34137481451034546, 0.34089887142181396, 0.24374568462371826, 0.4636613726615906, 0.1509108543395996, 0.9138983488082886, 0.2588678002357483, 0.9656578898429871, 0.7085896730422974, 0.3630499839782715, 0.06180286407470703, 0.798738420009613, 0.300193727016449, 0.1774476170539856, 0.20494788885116577, 0.29905736446380615, 0.7960351705551147, 0.808718204498291, 0.058021724224090576, 0.06811589002609253, 0.1405658721923828, 0.12180733680725098, 0.3965498208999634, 0.6225727796554565, 0.4342103600502014, 0.05902242660522461, 0.8796389698982239, 0.32651758193969727, 0.5812770128250122, 0.6087952256202698, 0.6917865872383118, 0.1445763111114502, 0.8271085619926453, 0.3030986189842224, 0.5905429124832153, 0.33345693349838257, 0.43709760904312134, 0.4996876120567322, 0.08871400356292725, 0.20847606658935547, 0.6819576621055603, 0.3531930446624756, 0.5808806419372559, 0.4609801769256592, 0.19368934631347656, 0.25183534622192383, 0.41720831394195557, 0.08351927995681763, 0.23452162742614746, 0.4923768639564514, 0.02873891592025757, 0.2154821753501892, 0.37453728914260864, 0.17788487672805786, 0.556582510471344, 0.11231940984725952, 0.03867143392562866, 0.07357054948806763, 0.19815897941589355, 0.27643001079559326, 0.27807021141052246, 0.026596903800964355, 0.07152563333511353, 0.4718847870826721, 0.3172047734260559, 0.6928157210350037, 0.5690871477127075, 0.3827299475669861, 0.3263177275657654, 0.2681494355201721, 0.5066072940826416, 0.3831031322479248, 0.19153577089309692, 0.15139323472976685, 0.25372016429901123, 0.465265691280365, 0.8596975207328796, 0.1783667802810669, 0.13692301511764526, 0.08870738744735718, 0.24434906244277954, 0.1915191411972046, 0.01087886095046997, 0.6625969409942627, 0.03361696004867554, 0.25003743171691895, 0.13844066858291626, 0.12110710144042969, 0.9895939826965332, 0.3232518434524536, 0.055772483348846436, 0.2001757025718689, 0.02311795949935913, 0.14854437112808228, 0.6589288115501404, 0.961397111415863, 0.166664719581604, 0.0731360912322998, 0.5237721800804138, 0.7351474165916443, 0.22351545095443726, 0.1842218041419983, 0.08661288022994995, 0.6197059154510498, 0.09717589616775513, 0.3424462676048279, 0.23666447401046753, 0.0019266009330749512, 0.27370327711105347, 0.5771273374557495, 0.1025192141532898, 0.48403894901275635, 0.15438193082809448, 0.4930803179740906, 0.14374524354934692, 0.7058203816413879, 0.5855528712272644, 0.1360154151916504, 0.43024569749832153, 0.43250060081481934, 0.1875590682029724, 0.46927279233932495, 0.22578126192092896, 0.2991827726364136, 0.9019526839256287, 0.5421391129493713, 0.018709301948547363, 0.2921816110610962, 0.46601784229278564, 0.273637056350708, 0.46425896883010864, 0.18744289875030518, 0.3073190450668335, 0.02760714292526245, 0.10657501220703125, 0.3155422806739807, 0.3595812916755676, 0.7340840101242065, 0.505942165851593, 0.14332884550094604, 0.4597802758216858, 0.2403969168663025, 0.30529558658599854, 0.7142919898033142, 0.2073502540588379, 0.820661187171936, 0.2701307535171509, 0.24229568243026733, 0.2778152823448181, 0.119662344455719, 0.8535444140434265, 0.7792513966560364, 0.287850022315979, 0.9936580061912537, 0.37250643968582153, 0.2556546926498413, 0.36388182640075684, 0.5520843267440796, 0.8402087688446045, 0.13083726167678833, 0.1315622329711914, 0.24148797988891602, 0.379940927028656, 0.457115113735199, 0.09193336963653564, 0.11003482341766357, 0.2535135746002197, 0.08798974752426147, 0.12454491853713989, 0.04005688428878784, 0.17860901355743408, 0.06813931465148926, 0.011471092700958252, 0.4620044231414795, 0.051701903343200684, 0.16867423057556152, 0.05087965726852417, 0.7555040121078491, 0.2107686996459961, 0.6146461367607117, 0.018869638442993164, 0.008468568325042725, 0.5757551789283752, 0.9080585837364197, 0.37933242321014404, 0.052813708782196045, 0.9627772569656372, 0.42295295000076294, 0.05777662992477417, 0.2814321517944336, 0.8697517514228821, 0.5074021220207214, 0.26438844203948975, 0.274736225605011, 0.46220654249191284, 0.2609103322029114, 0.53513503074646, 0.8956213593482971, 0.6743652820587158, 0.7489226460456848, 0.324202299118042, 0.2543897032737732, 0.0355229377746582, 0.5622398853302002, 0.42534399032592773, 0.1036641001701355, 0.4616745710372925, 0.6591119170188904, 0.36020737886428833, 0.2593907117843628, 0.3766651749610901, 0.6380648612976074, 0.22420185804367065, 0.39672648906707764, 0.2710414528846741, 0.4281548261642456, 0.23021572828292847, 0.26600247621536255, 0.6923249959945679, 0.07730382680892944, 0.4921143054962158, 0.6699784994125366, 0.3770543932914734, 0.17754995822906494, 0.04885876178741455, 0.6921665072441101, 0.5773279666900635, 0.26267915964126587, 0.5585647821426392, 0.7924274206161499, 0.4051751494407654, 0.1912301778793335, 0.19526058435440063, 0.0385664701461792, 0.6448498964309692, 0.38428574800491333, 0.6362228989601135, 0.0882386565208435, 0.2971850633621216, 0.04511171579360962, 0.04920011758804321, 0.5794392228126526, 0.19222867488861084, 0.42741233110427856, 0.18584352731704712, 0.34345120191574097, 0.3475078344345093, 0.25225645303726196, 0.3534541130065918, 0.9902380704879761, 0.88311368227005, 0.31653302907943726, 0.518183708190918, 0.7344667911529541, 0.6253409385681152, 0.2867974042892456, 0.6441060304641724, 0.6697983741760254, 0.6132450103759766, 0.20983338356018066, 0.04675692319869995, 0.7740386724472046, 0.6689173579216003, 0.18791258335113525, 0.33173108100891113, 0.38423091173171997, 0.20041334629058838, 0.30839771032333374, 0.08182710409164429, 0.2004687786102295, 0.10005521774291992, 0.2453392744064331, 0.08228486776351929, 0.6130460500717163, 0.3404642343521118, 0.2322005033493042, 0.16934359073638916, 0.06956791877746582, 0.7310025095939636, 0.14566802978515625, 0.09149640798568726, 0.03329747915267944, 0.04208916425704956, 0.5640537738800049, 0.3947606682777405, 0.6336634159088135, 0.45666372776031494, 0.29211390018463135, 0.22997230291366577, 0.42153364419937134, 0.6297598481178284, 0.3487884998321533, 0.1808217167854309, 0.5017910599708557, 0.4109429121017456, 0.12833541631698608, 0.4159788489341736, 0.0016360282897949219, 0.09044593572616577, 0.6330139636993408, 0.39340806007385254, 0.4565616846084595, 0.3558269739151001, 0.020222127437591553, 0.11330926418304443, 0.19865351915359497, 0.17975056171417236, 0.5728119611740112, 0.07321560382843018, 0.8905152678489685, 0.11808085441589355, 0.7874062657356262, 0.07176446914672852, 0.0443648099899292, 0.014320850372314453, 0.18724459409713745, 0.43866050243377686, 0.31895530223846436, 0.2872990369796753, 0.587569534778595, 0.053331971168518066, 0.33625054359436035, 0.21840745210647583, 0.45469170808792114, 0.5341286063194275, 0.1902030110359192, 0.004254639148712158, 0.09408384561538696, 0.12421435117721558, 0.24539226293563843, 0.6597522497177124, 0.23703277111053467, 0.13992947340011597, 0.2537570595741272, 0.6857505440711975, 0.12931722402572632, 0.04005199670791626, 0.09093666076660156, 0.039718806743621826, 0.3747156262397766, 0.03036797046661377, 0.5258442759513855, 0.1546313762664795, 0.6744599342346191, 0.5549911260604858, 0.5712142586708069, 0.6615030765533447, 0.0528985857963562, 0.2999584674835205, 0.3450993299484253, 0.07565927505493164, 0.07297283411026001, 0.044356346130371094, 0.8831465840339661, 0.1932048201560974, 0.05100613832473755, 0.6590126156806946, 0.395668089389801, 0.26586973667144775, 0.5835763812065125, 0.49586963653564453, 0.10856896638870239, 0.28165292739868164, 0.44618892669677734, 0.11517250537872314, 0.46746575832366943, 0.2097894549369812, 0.3431386351585388, 0.2949329614639282, 0.25385892391204834, 0.10683399438858032, 0.6245757937431335, 0.24255818128585815, 0.6042017340660095, 0.19216328859329224, 0.022775888442993164, 0.5113710761070251, 0.36327892541885376, 0.6428033709526062, 0.005164802074432373, 0.8481492400169373, 0.03857767581939697, 0.753609836101532, 0.7557845115661621, 0.061698317527770996, 0.16554558277130127, 0.12244284152984619, 0.21550637483596802, 0.1601831316947937, 0.5374140739440918, 0.3236740231513977, 0.6944597363471985, 0.8776770830154419, 0.3615381717681885, 0.07004350423812866, 0.23156940937042236, 0.13649380207061768, 0.76044100522995, 0.12847954034805298, 0.14349490404129028, 0.307547390460968, 0.03372877836227417, 0.48166340589523315, 0.053594231605529785, 0.19454288482666016, 0.2703455686569214, 0.7887381315231323, 0.09647750854492188, 0.6935285329818726, 0.3117150068283081, 0.7085102796554565, 0.5645732879638672, 0.4879736304283142, 0.038514554500579834, 0.0009722709655761719, 0.6566292643547058, 0.16987276077270508, 0.6095383167266846, 0.009832322597503662, 0.04496324062347412, 0.6313234567642212, 0.6185088753700256, 0.36357784271240234, 0.3960450291633606, 0.3420814871788025, 0.29508334398269653, 0.6343786120414734, 0.19541668891906738, 0.37375640869140625, 0.054252684116363525, 0.3174387216567993, 0.38205140829086304, 0.7202672362327576, 0.6883023977279663, 0.4628859758377075, 0.2767055034637451, 0.3114315867424011, 0.23694276809692383, 0.06204479932785034, 0.6188622117042542, 0.5397614240646362, 0.2737188935279846, 0.2273983359336853, 0.17393618822097778, 0.4043647050857544, 0.28399330377578735, 0.11514651775360107, 0.9080119132995605, 0.8159610629081726, 0.5613257884979248, 0.14169245958328247, 0.8508604764938354, 0.4642707109451294, 0.2563325762748718, 0.8574329614639282, 0.36311817169189453, 0.08406615257263184, 0.09509199857711792, 0.8265230059623718, 0.10423791408538818, 0.07808583974838257, 0.9033504128456116, 0.01089853048324585, 0.23120886087417603, 0.329383909702301, 0.13141852617263794, 0.15690380334854126, 0.36230331659317017, 0.008849024772644043, 0.028790056705474854, 0.007204890251159668, 0.10810351371765137, 0.26612675189971924, 0.07058656215667725, 0.696268618106842, 0.546403169631958, 0.5136880874633789, 0.16785210371017456, 0.2893909811973572, 0.1633920669555664, 0.1687307357788086, 0.39680641889572144, 0.2492353320121765, 0.37131989002227783, 0.05447012186050415, 0.3917744755744934, 0.5862212181091309, 0.07336193323135376, 0.4962838888168335, 0.7619773745536804, 0.17768269777297974, 0.13608604669570923, 0.03331780433654785, 0.03823959827423096, 0.4150964021682739, 0.39210784435272217, 0.1944618821144104, 0.335676372051239, 0.0798424482345581, 0.21094542741775513, 0.6450448036193848, 0.610223114490509, 0.0928577184677124, 0.0842127799987793, 0.3550528883934021, 0.7719781994819641, 0.7236397862434387, 0.3450137972831726, 0.9840043187141418, 0.021469831466674805, 0.24975550174713135, 0.010472476482391357, 0.6233329176902771, 0.15383780002593994, 0.10451376438140869, 0.7458292841911316, 0.360764741897583, 0.6004778146743774, 0.3808882236480713, 0.6795049905776978, 0.051586270332336426, 0.025378406047821045, 0.6396876573562622, 0.7627541422843933, 0.3257552981376648, 0.41949355602264404, 0.21269696950912476, 0.03832215070724487, 0.5856065154075623, 0.23087447881698608, 0.05384010076522827, 0.40160828828811646, 0.19077622890472412, 0.5017145872116089, 0.07543396949768066, 0.013933897018432617, 0.338550865650177, 0.33083170652389526, 0.3250075578689575, 0.0008158683776855469, 0.2800830006599426, 0.1485581398010254, 0.5078197717666626, 0.008756041526794434, 0.30814826488494873, 0.03041374683380127, 0.36685532331466675, 0.47563332319259644, 0.2502368688583374, 0.14453339576721191, 0.16098934412002563, 0.3397182822227478, 0.13094955682754517, 0.13330042362213135, 0.017372071743011475, 0.7166422605514526, 0.013726115226745605, 0.4030940532684326, 0.09050428867340088, 0.02019643783569336, 0.13235598802566528, 0.14338034391403198, 0.6823422312736511, 0.6029868125915527, 0.32977861166000366, 0.02114325761795044, 0.5113218426704407, 0.5353528261184692, 0.5552297830581665, 0.22186875343322754, 0.35811811685562134, 0.11425089836120605, 0.09603196382522583, 0.1677330732345581, 0.20450353622436523, 0.5817774534225464, 0.000644385814666748, 0.18630963563919067, 0.0681648850440979, 0.18281614780426025, 0.13383269309997559, 0.2546762228012085, 0.27778947353363037, 0.7560199499130249, 0.6999359130859375, 0.04077887535095215, 0.3377031087875366, 0.2409077286720276, 0.18643981218338013, 0.5603033900260925, 0.009338140487670898, 0.536359965801239, 0.26261645555496216, 0.9107420444488525, 0.25570178031921387, 0.661070704460144, 0.024923980236053467, 0.213700532913208, 0.7467600703239441, 0.6300323009490967, 0.6026948690414429, 0.6247162222862244, 0.25691360235214233, 0.288067102432251, 0.10866856575012207, 0.24743056297302246, 0.004430949687957764, 0.2624536156654358, 0.22507888078689575, 0.8616869449615479, 0.5704179406166077, 0.4741484522819519, 0.08505839109420776, 0.54315185546875, 0.14022374153137207, 0.7527753710746765, 0.577320396900177, 0.09879183769226074, 0.02239125967025757, 0.7012069821357727, 0.31622517108917236, 0.1243773102760315, 0.2806587219238281, 0.6382128000259399, 0.5109254717826843, 0.4128033518791199, 0.238481342792511, 0.4626028537750244, 0.8099384307861328, 0.9231826066970825, 0.34108108282089233, 0.4393625855445862, 0.19932615756988525, 0.3011261224746704, 0.3131035566329956, 0.4019944667816162, 0.32988911867141724, 0.25523459911346436, 0.4928478002548218, 0.5626381635665894, 0.4996762275695801, 0.8550645112991333, 0.4207465648651123, 0.7782979011535645, 0.7124228477478027, 0.19493025541305542, 0.07812637090682983, 0.344900906085968, 0.040223121643066406, 0.25269126892089844, 0.1808086633682251, 0.7918254733085632, 0.26953911781311035, 0.457817018032074, 0.13382726907730103, 0.20008808374404907, 0.7582071423530579, 0.09212124347686768, 0.582633912563324, 0.2964409589767456, 0.3818991184234619, 0.49429523944854736, 0.0887569785118103, 0.11913776397705078, 0.9225149154663086, 0.05476725101470947, 0.4869338274002075, 0.07863938808441162, 0.6254088878631592, 0.5363404154777527, 0.5684577822685242, 0.2274135947227478, 0.6223553419113159, 0.04094773530960083, 0.12392479181289673, 0.5818532705307007, 0.48452359437942505, 0.4473062753677368, 0.05486029386520386, 0.8634198904037476, 0.31540149450302124, 0.19523048400878906, 0.513123095035553, 0.31447672843933105, 0.07035255432128906, 0.2655336856842041, 0.02193540334701538, 0.5123392343521118, 0.6499109268188477, 0.2779582142829895, 0.0027161836624145508, 0.26754432916641235, 0.7023688554763794, 0.26170700788497925, 0.18526840209960938, 0.5178350806236267, 0.38673824071884155, 0.40228962898254395, 0.14159303903579712, 0.06593257188796997, 0.0020531415939331055, 0.29875820875167847, 0.5661636590957642, 0.6889581680297852, 0.47009003162384033, 0.39117181301116943, 0.07548302412033081, 0.4766538143157959, 0.27714353799819946, 0.5713177919387817, 0.598385751247406, 0.23865461349487305, 0.82037752866745, 0.04245728254318237, 0.19078707695007324, 0.0991174578666687, 0.1560010313987732, 0.1958523392677307, 0.702501118183136, 0.07212132215499878, 0.07361888885498047, 0.13362109661102295, 0.5458702445030212, 0.33245372772216797, 0.24905407428741455, 0.5764150023460388, 0.1543535590171814, 0.3000752329826355, 0.20350193977355957, 0.5336818695068359, 0.3978274464607239, 0.7485272288322449, 0.03223913908004761, 0.363128125667572, 0.0749121904373169, 0.8052709102630615, 0.4659094214439392, 0.09125924110412598, 0.10416179895401001, 0.0700647234916687, 0.33280956745147705, 0.3619859218597412, 0.255967915058136, 0.3844812512397766, 0.8537462949752808, 0.2478134036064148, 0.34628361463546753, 0.6224931478500366, 0.8181982040405273, 0.25716692209243774, 0.09312599897384644, 0.25588226318359375, 0.30008578300476074, 0.6294910311698914, 0.7125717401504517, 0.08669984340667725, 0.207328200340271, 0.7689465880393982, 0.6074448227882385, 0.29206860065460205, 0.47775667905807495, 0.5418938398361206, 0.8109647035598755, 0.4426206350326538, 0.49973416328430176, 0.24358826875686646, 0.6533606052398682, 0.18413227796554565, 0.024648606777191162, 0.0057621002197265625, 0.5681761503219604, 0.44640469551086426, 0.15481829643249512, 0.25041335821151733, 0.010734021663665771, 0.06619620323181152, 0.0952407717704773, 0.13740700483322144, 0.002537846565246582, 0.02896958589553833, 0.9467032551765442, 0.6585557460784912, 0.0441204309463501, 0.5964229702949524, 0.29260218143463135, 0.5278152823448181, 0.14794403314590454, 0.07405537366867065, 0.027141988277435303, 0.03325200080871582, 0.16792339086532593, 0.4849720001220703, 0.6466550827026367, 0.24122804403305054, 0.775640070438385, 0.3812553882598877, 0.37393444776535034, 0.8351064324378967, 0.34456688165664673, 0.7099165320396423, 0.714155375957489, 0.4733726978302002, 0.16638535261154175, 0.6068639159202576, 0.7668676376342773, 0.7907803058624268, 0.12641477584838867, 0.030815601348876953, 0.20309782028198242, 0.2211187481880188, 0.074154794216156, 0.27165377140045166, 0.11190086603164673, 0.0029391050338745117, 0.05846935510635376, 0.5514259338378906, 0.2474880814552307, 0.8823404908180237, 0.7537647485733032, 0.9046541452407837, 0.2987927198410034, 0.32040220499038696, 0.6193526983261108, 0.21480298042297363, 0.15227216482162476, 0.1589553952217102, 0.007176339626312256, 0.19032764434814453, 0.5231552124023438, 0.11294227838516235, 0.6114161610603333, 0.38231325149536133, 0.09929680824279785, 0.7920922040939331, 0.3357107639312744, 0.5785777568817139, 0.49632734060287476, 0.24678033590316772, 0.2076851725578308, 0.2883481979370117, 0.5536497831344604, 0.28430962562561035, 0.512567937374115, 0.1424521803855896, 0.1497117280960083, 0.28814131021499634, 0.323073148727417, 0.0571402907371521, 0.8669700026512146, 0.9525338411331177, 0.4658345580101013, 0.1186705231666565, 0.01558607816696167, 0.21138310432434082, 0.5544434785842896, 0.38554394245147705, 0.2518376111984253, 0.21789991855621338, 0.2951887249946594, 0.3215755820274353, 0.6628501415252686, 0.024064600467681885, 0.8853209614753723, 0.03287094831466675, 0.24056589603424072, 0.488137423992157, 0.024602234363555908, 0.046440958976745605, 0.5245329141616821, 0.5776016712188721, 0.4009894132614136, 0.21831417083740234, 0.35535287857055664, 0.5693161487579346, 0.12028604745864868, 0.20631515979766846, 0.9063761234283447, 0.26227885484695435, 0.09776169061660767, 0.11506980657577515, 0.1287786364555359, 0.761272132396698, 0.05582493543624878, 0.22747820615768433, 0.14004802703857422, 0.4802764058113098, 0.07306069135665894, 0.8239003419876099, 0.32578790187835693, 0.4306786060333252, 0.7651957273483276, 0.46802109479904175, 0.6023448705673218, 0.19876223802566528, 0.16101223230361938, 0.7140587568283081, 0.005189955234527588, 0.37200111150741577, 0.29875391721725464, 0.14259767532348633, 0.049367308616638184, 0.020596325397491455, 0.7227105498313904, 0.9986600875854492, 0.11682778596878052, 0.36404621601104736, 0.6242408156394958, 0.07914870977401733, 0.38223469257354736, 0.4512714743614197, 0.40767282247543335, 0.02714061737060547, 0.7909156680107117, 0.20538806915283203, 0.11325645446777344, 0.20129930973052979, 0.2224113941192627, 0.49451756477355957, 0.5254344940185547, 0.19568049907684326, 0.5534564256668091, 0.0960809588432312, 0.03347325325012207, 0.6671048402786255, 0.7101510167121887, 0.8012573719024658, 0.5190631747245789, 0.24969863891601562, 0.4814068078994751, 0.14676731824874878, 0.07499533891677856, 0.06160050630569458, 0.3728891611099243, 0.1986730694770813, 0.16480517387390137, 0.21852761507034302, 0.08216094970703125, 0.5373535752296448, 0.5020534992218018, 0.016841351985931396, 0.1422021985054016, 0.41038739681243896, 0.4000771641731262, 0.1296612024307251, 0.2965066432952881, 0.0954088568687439, 0.430477499961853, 0.3846611976623535, 0.12629753351211548, 0.1632431149482727, 0.40446943044662476, 0.7694490551948547, 0.7060815095901489, 0.9475953578948975, 0.2286924123764038, 0.7804908156394958, 0.6288813948631287, 0.14982444047927856, 0.21114176511764526, 0.26186060905456543, 0.30676841735839844, 0.24401241540908813, 0.03756767511367798, 0.40330713987350464, 0.7175893187522888, 0.45515602827072144, 0.2852932810783386, 0.15197700262069702, 0.48983830213546753, 0.45821839570999146, 0.37094229459762573, 0.1927950382232666, 0.028654634952545166, 0.16484123468399048, 0.26215583086013794, 0.0029851794242858887, 0.06334513425827026, 0.5681149363517761, 0.1650756597518921, 0.5385308265686035, 0.4604474902153015, 0.2582951784133911, 0.0627393126487732, 0.2360019087791443, 0.2765098810195923, 0.09761583805084229, 0.44076693058013916, 0.47469431161880493, 0.16127973794937134, 0.29586631059646606, 0.36150604486465454, 0.39763349294662476, 0.4565628170967102, 0.4377174973487854, 0.6647759079933167, 0.6650365591049194, 0.5402906537055969, 0.5402312874794006, 0.7254310846328735, 0.3673533797264099, 0.10441482067108154, 0.12288558483123779, 0.03844493627548218, 0.552666425704956, 0.23502594232559204, 0.1540089249610901, 0.5044607520103455, 0.003799617290496826, 0.36819612979888916, 0.24809712171554565, 0.22346049547195435, 0.4615488648414612, 0.5442361831665039, 0.7078405618667603, 0.4235961437225342, 0.0887002944946289, 0.38976162672042847, 0.8689817190170288, 0.43397635221481323, 0.7992848753929138, 0.0028901100158691406, 0.5121026635169983, 0.7321841716766357, 0.09511834383010864, 0.44722849130630493, 0.22188299894332886, 0.5204576849937439, 0.7019521594047546, 0.2787129282951355, 0.6222110986709595, 0.5793117880821228, 0.42594993114471436, 0.560049295425415, 0.6217032074928284, 0.17362159490585327, 0.28305643796920776, 0.6067923903465271, 0.0851668119430542, 0.8422002792358398, 0.11205917596817017, 0.4225161671638489, 0.2290869951248169, 0.3843320608139038, 0.03094947338104248, 0.4859028458595276, 0.6235018372535706, 0.4131063222885132, 0.2038155198097229, 0.11194896697998047, 0.05337947607040405, 0.3722352385520935, 0.3263216018676758, 0.2668490409851074, 0.41996973752975464, 0.25557994842529297, 0.1732819676399231, 0.7210686802864075, 0.057172179222106934, 0.5230165123939514, 0.4644281268119812, 0.0011609792709350586, 0.3982287645339966, 0.490749716758728, 0.23074018955230713, 0.2881805896759033, 0.42829185724258423, 0.20777833461761475, 0.3904600739479065, 0.4790676236152649, 0.6724830269813538, 0.6236110329627991, 0.27134519815444946, 0.7028695940971375, 0.02326291799545288, 0.001370549201965332, 0.2513526678085327, 0.0027006864547729492, 0.11055862903594971, 0.03614991903305054, 0.264660120010376, 0.2922927141189575, 0.5898316502571106, 0.42369675636291504, 0.32634103298187256, 0.1629246473312378, 0.2516862154006958, 0.6127235293388367, 0.12999427318572998, 0.8531032204627991, 0.8471171259880066, 0.0994497537612915, 0.3740151524543762, 0.4028978943824768, 0.12401032447814941, 0.15997278690338135, 0.4285738468170166, 0.20567572116851807, 0.09854573011398315, 0.26017624139785767, 0.27981674671173096, 0.0654294490814209, 0.7313706278800964, 0.01690441370010376, 0.6377604007720947, 0.2960699796676636, 0.9062864184379578, 0.6712642908096313, 0.20468610525131226, 0.10787022113800049, 0.35162925720214844, 0.18128645420074463, 0.5772203803062439, 0.5365534424781799, 0.2635253071784973, 0.4527842402458191, 0.017261505126953125, 0.10579335689544678, 0.771058201789856, 0.22215431928634644, 0.7673525214195251, 0.020594298839569092, 0.2690166234970093, 0.3531581163406372, 0.24391645193099976, 0.041911780834198, 0.4682987928390503, 0.638183057308197, 0.3898612856864929, 0.4741772413253784, 0.48354315757751465, 0.01961207389831543, 0.11674696207046509, 0.36260437965393066, 0.23014098405838013, 0.23073750734329224, 0.25050634145736694, 0.7171963453292847, 0.1713348627090454, 0.5413708090782166, 0.30122095346450806, 0.12080639600753784, 0.04303199052810669, 0.270652711391449, 0.3586527705192566, 0.5372992157936096, 0.5267775058746338, 0.7023227214813232, 0.3008776903152466, 0.032007038593292236, 0.23536866903305054, 0.12674325704574585, 0.3470703959465027, 0.17522871494293213, 0.551794707775116, 0.024147868156433105, 0.8008124828338623, 0.6528962254524231, 0.14472109079360962, 0.30212610960006714, 0.4079274535179138, 0.41453129053115845, 0.5203582644462585, 0.1772700548171997, 0.17274582386016846, 0.41779714822769165, 0.5944892168045044, 0.5634604096412659, 0.20195764303207397, 0.3696012496948242, 0.25726187229156494, 0.5582267642021179, 0.8494088649749756, 0.01668637990951538, 0.5182832479476929, 0.4570543169975281, 0.6759578585624695, 0.08931463956832886, 0.3098999857902527, 0.45537620782852173, 0.6715852618217468, 0.25979477167129517, 0.3107307553291321, 0.8794057369232178, 0.4346873164176941, 0.2904636859893799, 0.5604636669158936, 0.1862301230430603, 0.3313515782356262, 0.008322536945343018, 0.12359875440597534, 0.23590219020843506, 0.8162575960159302, 0.7954255938529968, 0.19493097066879272, 0.057788074016571045, 0.2233104109764099, 0.03593790531158447, 0.523961067199707, 0.27070891857147217, 0.18242448568344116, 0.07353907823562622, 0.020678937435150146, 0.6021145582199097, 0.35378944873809814, 0.4723813533782959, 0.2782905697822571, 0.5474599599838257, 0.05464494228363037, 0.6382644772529602, 0.0751723051071167, 0.5762564539909363, 0.18940162658691406, 0.2778926491737366, 0.05392032861709595, 0.3536220192909241, 0.25599199533462524, 0.28562748432159424, 0.8107366561889648, 0.12476122379302979, 0.2413889765739441, 0.6662002801895142, 0.13744550943374634, 0.35120344161987305, 0.4186614155769348, 0.31093645095825195, 0.7406802773475647, 0.4103614091873169, 0.1506534218788147, 0.41489189863204956, 0.4558143615722656, 0.01036769151687622, 0.17703872919082642, 0.05901646614074707, 0.018074989318847656, 0.27618175745010376, 0.2799171805381775, 0.4991270899772644, 0.32558226585388184, 0.08532339334487915, 0.5417302846908569, 0.002873539924621582, 0.19538724422454834, 0.21646875143051147, 0.1370602250099182, 0.04456430673599243, 0.022939622402191162, 0.503229022026062, 0.8057676553726196, 0.5842092633247375, 0.11897724866867065, 0.17318296432495117, 0.31878960132598877, 0.5447614192962646, 0.8301793932914734, 0.4099499583244324, 0.008066177368164062, 0.7880712151527405, 0.3731246590614319, 0.9649072885513306, 0.3642765283584595, 0.12262910604476929, 0.426679790019989, 0.6644271612167358, 0.372717022895813, 0.41480791568756104, 0.4474552273750305, 0.00844883918762207, 0.4597393274307251, 0.002508401870727539, 0.35832512378692627, 0.0531192421913147, 0.3786877393722534, 0.8572873473167419, 0.7156192064285278, 0.04883188009262085, 0.5493101477622986, 0.04226827621459961, 0.6074405908584595, 0.5965610146522522, 0.2657666802406311, 0.2129281759262085, 0.4468998312950134, 0.8620080351829529, 0.47929930686950684, 0.7136749029159546, 0.27555614709854126, 0.05075216293334961, 0.07998597621917725, 0.19152617454528809, 0.5742021203041077, 0.05178952217102051, 0.0205228328704834, 0.6271637678146362, 0.2847086787223816, 0.11302202939987183, 0.565509557723999, 0.6033837795257568, 0.3006821870803833, 0.30828016996383667, 0.3658663034439087, 0.34975117444992065, 0.5429704189300537, 0.4731360077857971, 0.2447720766067505, 0.2894660234451294, 0.1591288447380066, 0.8847566246986389, 0.9145291447639465, 0.41481077671051025, 0.20994490385055542, 0.2196102738380432, 0.4873965382575989, 0.761823832988739, 0.4949648380279541, 0.18470972776412964, 0.11962932348251343, 0.8576502203941345, 0.10777777433395386, 0.5591999888420105, 0.046131432056427, 0.1617887020111084, 0.011890769004821777, 0.492545485496521, 0.5230661034584045, 0.05768364667892456, 0.65542072057724, 0.011667191982269287, 0.4360969662666321, 0.03254282474517822, 0.292652428150177, 0.45124340057373047, 0.762158215045929, 0.05356895923614502, 0.7779478430747986, 0.353329062461853, 0.08558619022369385, 0.1510908007621765, 0.7956047058105469, 0.16331976652145386, 0.006430268287658691, 0.7754207849502563, 0.5168695449829102, 0.8056328892707825, 0.228898823261261, 0.2621866464614868, 0.48448801040649414, 0.500939667224884, 0.7030854225158691, 0.6137178540229797, 0.6428977251052856, 0.03908485174179077, 0.1644657850265503, 0.381233811378479, 0.4191596508026123, 0.01766258478164673, 0.04928922653198242, 0.811174750328064, 0.39967572689056396, 0.04859417676925659, 0.207683265209198, 0.3525809645652771, 0.6198136210441589, 0.2110743522644043, 0.530737042427063, 0.49181807041168213, 0.3941304087638855, 0.4474538564682007, 0.006203770637512207, 0.05471146106719971, 0.8298943638801575, 0.16600364446640015, 0.5111364126205444, 0.180059015750885, 0.4174611568450928, 0.5763744711875916, 0.3704509735107422, 0.4265702962875366, 0.21171051263809204, 0.25785523653030396, 0.0847846269607544, 0.21862047910690308, 0.09702068567276001, 0.3368896245956421, 0.0957258939743042, 0.10579168796539307, 0.0890539288520813, 0.30884575843811035, 0.0619853138923645, 0.8036333322525024, 0.3494325876235962, 0.4610961079597473, 0.10654038190841675, 0.5707128643989563, 0.09850257635116577, 0.6313660740852356, 0.4580632448196411, 0.43452274799346924, 0.033756256103515625, 0.34040480852127075, 0.11407017707824707, 0.29476237297058105, 0.39413684606552124, 0.21980631351470947, 0.5062515735626221, 0.09118962287902832, 0.5206208825111389, 0.38325053453445435, 0.554567813873291, 0.7921947836875916, 0.23308688402175903, 0.21503806114196777, 0.09383988380432129, 0.07348030805587769, 0.9125609993934631, 0.24988079071044922, 0.3617814779281616, 0.11375820636749268, 0.35383903980255127, 0.09510457515716553, 0.2635497450828552, 0.42009347677230835, 0.6262926459312439, 0.031147778034210205, 0.019219696521759033, 0.35955333709716797, 0.3606083393096924, 0.08921754360198975, 0.3412654995918274, 0.6992650628089905, 0.08397442102432251, 0.3617171049118042, 0.24265438318252563, 0.22079682350158691, 0.3091869354248047, 0.10461169481277466, 0.42108064889907837, 0.20363008975982666, 0.3228011131286621, 0.01595515012741089, 0.11050134897232056, 0.6348232626914978, 0.4035012722015381, 0.6726061701774597, 0.3044447898864746, 0.14139080047607422, 0.19846111536026, 0.773932695388794, 0.5646571516990662, 0.23341971635818481, 0.6163620352745056, 0.13149213790893555, 0.24208801984786987, 0.2086675763130188, 0.2523499131202698, 0.16964709758758545, 0.1691686511039734, 0.37944722175598145, 0.7397962808609009, 0.5113720297813416, 0.044017016887664795, 0.09676021337509155, 0.77656090259552, 0.14094603061676025, 0.5564761757850647, 0.8447319865226746, 0.7290897965431213, 0.21052426099777222, 0.4938698410987854, 0.48834455013275146, 0.3525075912475586, 0.11210209131240845, 0.1862824559211731, 0.08894997835159302, 0.07278633117675781, 0.03852134943008423, 0.1778017282485962, 0.26191580295562744, 0.29269546270370483, 0.01931583881378174, 0.22568368911743164, 0.19465363025665283, 0.07820618152618408, 0.2786422371864319, 0.19526511430740356, 0.31589841842651367, 0.425573468208313, 0.34154343605041504, 0.21425718069076538, 0.053603947162628174, 0.6503801345825195, 0.2983880639076233, 0.19228333234786987, 0.39858460426330566, 0.1653178334236145, 0.08646106719970703, 0.17725509405136108, 0.4033355116844177, 0.30410027503967285, 0.9602206945419312, 0.08194780349731445, 0.38437598943710327, 0.1745052933692932, 0.43014514446258545, 0.18386530876159668, 0.12826639413833618, 0.030164480209350586, 0.8451429009437561, 0.5783483982086182, 0.4318350553512573, 0.16776257753372192, 0.8698345422744751, 0.5827850699424744, 0.579961359500885, 0.2658798098564148, 0.5674423575401306, 0.04516822099685669, 0.06358414888381958, 0.0406683087348938, 0.670631468296051, 0.06382858753204346, 0.6840788722038269, 0.589482843875885, 0.354248046875, 0.44147932529449463, 0.4271724224090576, 0.3133571147918701, 0.1741400957107544, 0.18486613035202026, 0.010982751846313477, 0.21599233150482178, 0.3092859983444214, 0.06750988960266113, 0.4715496897697449, 0.29129934310913086, 0.13465601205825806, 0.1534191370010376, 0.03919774293899536, 0.04006385803222656, 0.2276896834373474, 0.5608339309692383, 0.6184353828430176, 0.46295809745788574, 0.296153724193573, 0.16864889860153198, 0.28650885820388794, 0.5596276521682739, 0.4739588499069214, 0.8352384567260742, 0.7631791830062866, 0.008971750736236572, 0.37101465463638306, 0.45944535732269287, 0.021150946617126465, 0.48230892419815063, 0.7553964257240295, 0.09750008583068848, 0.5225490927696228, 0.1754246950149536, 0.4350414276123047, 0.7365646958351135, 0.33684414625167847, 0.2647257447242737, 0.208260178565979, 0.44404733180999756, 0.43614327907562256, 0.5071038603782654, 0.16796457767486572, 0.47704291343688965, 0.13222837448120117, 0.35970062017440796, 0.5442570447921753, 0.3868314027786255, 0.45187193155288696, 0.28519660234451294, 0.11432015895843506, 0.3578735589981079, 0.5849003791809082, 0.11224693059921265, 0.058701157569885254, 0.7692009806632996, 0.6930657625198364, 0.5750844478607178, 0.6655616164207458, 0.2346341609954834, 0.6936293244361877, 0.09007883071899414, 0.6903904676437378, 0.6174211502075195, 0.1230478286743164, 0.9796403646469116, 0.430838406085968, 0.5152223706245422, 0.09482216835021973, 0.4510325789451599, 0.035398006439208984, 0.21043258905410767, 0.15174496173858643, 0.807397723197937, 0.9486615657806396, 0.48895251750946045, 0.15351003408432007, 0.8443572521209717, 0.5439209938049316, 0.22982770204544067, 0.11990851163864136, 0.007937014102935791, 0.4606349468231201, 0.38005805015563965, 0.07726985216140747, 0.5021378397941589, 0.353681743144989, 0.11596226692199707, 0.8603413701057434, 0.3911570906639099, 0.061596810817718506, 0.013919591903686523, 0.1393829584121704, 0.03637361526489258, 0.7373092770576477, 0.03940707445144653, 0.12603795528411865, 0.08444041013717651, 0.24195408821105957, 0.4285525679588318, 0.26144498586654663, 0.20756137371063232, 0.5780953168869019, 0.5072348117828369, 0.14481985569000244, 0.3451915979385376, 0.3363445997238159, 0.2237754464149475, 0.5563679337501526, 0.37262392044067383, 0.87473464012146, 0.7871847748756409, 0.14805614948272705, 0.7030897736549377, 0.21403563022613525, 0.02122783660888672, 0.23694217205047607, 0.40804344415664673, 0.2281479835510254, 0.059783101081848145, 0.10701209306716919, 0.5433886051177979, 0.27834343910217285, 0.13983750343322754, 0.26308876276016235, 0.6962820887565613, 0.5388122797012329, 0.6474459767341614, 0.14278864860534668, 0.1439957618713379, 0.325894296169281, 0.026419460773468018, 0.023395001888275146, 0.38454848527908325, 0.7431985139846802, 0.04984104633331299, 0.5563170909881592, 0.37736546993255615, 0.6202465295791626, 0.4651188254356384, 0.03480660915374756, 0.6713323593139648, 0.38013267517089844, 0.11001342535018921, 0.08501178026199341, 0.6390198469161987, 0.7682086825370789, 0.36356669664382935, 0.46613895893096924, 0.2718238830566406, 0.25817495584487915, 0.23870795965194702, 0.461731493473053, 0.6812980771064758, 0.18305838108062744, 0.6456577777862549, 0.7928662896156311, 0.10977864265441895, 0.40976643562316895, 0.25778913497924805, 0.16555839776992798, 0.4598769545555115, 0.41439640522003174, 0.46113044023513794, 0.3986412286758423, 0.17631280422210693, 0.18860501050949097, 0.19313961267471313, 0.23031574487686157, 0.21624022722244263, 0.6847255825996399, 0.09235972166061401, 0.8511065244674683, 0.2918880581855774, 0.5345868468284607, 0.5698029398918152, 0.26484251022338867, 0.21823513507843018, 0.4188686013221741, 0.02393585443496704, 0.031245112419128418, 0.30758136510849, 0.5538695454597473, 0.42304736375808716, 0.508392333984375, 0.1814732551574707, 0.1347295045852661, 0.4510679841041565, 0.4050258994102478, 0.6677301526069641, 0.7541391253471375, 0.42163658142089844, 0.4131840467453003, 0.7425106167793274, 0.07682991027832031, 0.0731242299079895, 0.27158457040786743, 0.3559507727622986, 0.04463529586791992, 0.23803269863128662, 0.3414720296859741, 0.007762730121612549, 0.20808351039886475, 0.5880749225616455, 0.3006032109260559, 0.2493826150894165, 0.027601778507232666, 0.5158697366714478, 0.1581065058708191, 0.01059865951538086, 0.44414883852005005, 0.055163681507110596, 0.2326609492301941, 0.13304883241653442, 0.22885733842849731, 0.016890883445739746, 0.23676860332489014, 0.4466398358345032, 0.003004312515258789, 0.3727095127105713, 0.38618308305740356, 0.18599534034729004, 0.28487420082092285, 0.23470807075500488, 0.16161245107650757, 0.15019762516021729, 0.027945220470428467, 0.5014710426330566, 0.15087568759918213, 0.9265869855880737, 0.5349989533424377, 0.16711193323135376, 0.4599758982658386, 0.1751716136932373, 0.5249786376953125, 0.1512364149093628, 0.5890569686889648, 0.3602578639984131, 0.44474494457244873, 0.8372452259063721, 0.6919496655464172, 0.13149869441986084, 0.39604008197784424, 0.5488859415054321, 0.03237473964691162, 0.13008564710617065, 0.6887239217758179, 0.11610358953475952, 0.33216989040374756, 0.07901942729949951, 0.8954989910125732, 0.22098177671432495, 0.19199854135513306, 0.01835310459136963, 0.8471012711524963, 0.17238444089889526, 0.12622952461242676, 0.18040335178375244, 0.20506834983825684, 0.5974936485290527, 0.24848592281341553, 0.6793423891067505, 0.4738500714302063, 0.2298310399055481, 0.5348778367042542, 0.20910096168518066, 0.5862210988998413, 0.11890125274658203, 0.029197394847869873, 0.23345947265625, 0.37654948234558105, 0.5524002909660339, 0.6200321316719055, 0.4248061776161194, 0.1416705846786499, 0.44623249769210815, 0.598656952381134, 0.46766120195388794, 0.17118430137634277, 0.3848962187767029, 0.7558807134628296, 0.7284669876098633, 0.7286603450775146, 0.006805896759033203, 0.17926812171936035, 0.473837673664093, 0.5861667394638062, 0.25763243436813354, 0.48281359672546387, 0.28425395488739014, 0.6504719257354736, 0.4829402565956116, 0.45152556896209717, 0.25336021184921265, 0.5581091642379761, 0.708008885383606, 0.6296876072883606, 0.3926084041595459, 0.3420822024345398, 0.02440023422241211, 0.012736618518829346, 0.1255766749382019, 0.05748707056045532, 6.03795051574707e-05, 0.5498443841934204, 0.15591537952423096, 0.6684653162956238, 0.0767824649810791, 0.009534180164337158, 0.21564710140228271, 0.8761433362960815, 0.7268587946891785, 0.1247902512550354, 0.29898595809936523, 0.2779976725578308, 0.17511087656021118, 0.5473044514656067, 0.8150667548179626, 0.9538915753364563, 0.27801620960235596, 0.8327429890632629, 0.1514795422554016, 0.07378607988357544, 0.6708166003227234, 0.42273515462875366, 0.17439919710159302, 0.5211141705513, 0.9071663022041321, 0.18630975484848022, 0.2497098445892334, 0.02141934633255005, 0.181948721408844, 0.3685104250907898, 0.6594823598861694, 0.43666601181030273, 0.37065833806991577, 0.5222543478012085, 0.3199647068977356, 0.1626846194267273, 0.1401500701904297, 0.45854878425598145, 0.15104812383651733, 0.612962543964386, 0.41671615839004517, 0.44545185565948486, 0.4337122440338135, 0.3372530937194824, 0.040451884269714355, 0.4976854920387268, 0.2021675705909729, 0.22803252935409546, 0.269985556602478, 0.489032506942749, 0.37404000759124756, 0.043389737606048584, 0.6091185212135315, 0.7178847193717957, 0.40790194272994995, 0.027325868606567383, 0.4843910336494446, 0.009962618350982666, 0.4988250136375427, 0.1797323226928711, 0.004881322383880615, 0.06181412935256958, 0.605262815952301, 0.24454212188720703, 0.06519365310668945, 0.18509608507156372, 0.13916480541229248, 0.19300168752670288, 0.4837672710418701, 0.01004171371459961, 0.5093972682952881, 0.4841465353965759, 0.2536628842353821, 0.0905231237411499, 0.7989276051521301, 0.22447556257247925, 0.025903761386871338, 0.4118567109107971, 0.4558946490287781, 0.23634874820709229, 0.5797840356826782, 0.372284471988678, 0.5143783688545227, 0.2103722095489502, 0.5303688049316406, 0.7202989459037781, 0.8599132895469666, 0.07369041442871094, 0.15949028730392456, 0.22127759456634521, 0.7079834938049316, 0.35801780223846436, 0.9267650842666626, 0.1587201952934265, 0.0608709454536438, 0.337851345539093, 0.8214618563652039, 0.09929108619689941, 0.18479883670806885, 0.21849703788757324, 0.4815698266029358, 0.2874760031700134, 0.1719571352005005, 0.0083160400390625, 0.4018666744232178, 0.019260525703430176, 0.8464950323104858, 0.64020174741745, 0.5083693861961365, 0.36118048429489136, 0.25071799755096436, 0.4315122365951538, 0.24076229333877563, 0.2819637656211853, 0.16687142848968506, 0.7223204970359802, 0.5307448506355286, 0.1821843385696411, 0.6603458523750305, 0.09430956840515137, 0.075539231300354, 0.35044580698013306, 0.0035492777824401855, 0.29759418964385986, 0.033553481101989746, 0.3990558385848999, 0.5731952786445618, 0.31268471479415894, 0.1492571234703064, 0.167899489402771, 0.33655911684036255, 0.08670151233673096, 0.11995434761047363, 0.03588944673538208, 0.42448800802230835, 0.26351654529571533, 0.25987207889556885, 0.2760269045829773, 0.6928909420967102, 0.5966526865959167, 0.22337961196899414, 0.2555886507034302, 0.18662428855895996, 0.3658318519592285, 0.6339378952980042, 0.5452885627746582, 0.14636075496673584, 0.21764391660690308, 0.14655333757400513, 0.6477442383766174, 0.17646247148513794, 0.6257550120353699, 0.3575223684310913, 0.4740675687789917, 0.22312164306640625, 0.09010344743728638, 0.03395986557006836, 0.6698409914970398, 0.3219837546348572, 0.06496244668960571, 0.3955264091491699, 0.05561792850494385, 0.12738364934921265, 0.32291972637176514, 0.37489789724349976, 0.49253034591674805, 0.34634363651275635, 0.5099078416824341, 0.32573574781417847, 0.29556089639663696, 0.4853948950767517, 0.5026410222053528, 0.5648400187492371, 0.055704712867736816, 0.2999191880226135, 0.46658962965011597, 0.04718738794326782, 0.2829340100288391, 0.008639276027679443, 0.2360488772392273, 0.4943477511405945, 0.6198437213897705, 0.03212672472000122, 0.8265991806983948, 0.5653368234634399, 0.7327704429626465, 0.39233821630477905, 0.47681725025177, 0.41479092836380005, 0.3957504630088806, 0.20357435941696167, 0.5463030338287354, 0.7227593064308167, 0.531186580657959, 0.710296630859375, 0.442621111869812, 0.07727622985839844, 0.2649310231208801, 0.19889593124389648, 0.7320338487625122, 0.41898441314697266, 0.566010594367981, 0.14215326309204102, 0.6990099549293518, 0.0776258111000061, 0.05641460418701172, 0.056326091289520264, 0.6862976551055908, 0.23320013284683228, 0.6337991952896118, 0.51875239610672, 0.10236859321594238, 0.24339812994003296, 0.9182304739952087, 0.0916556715965271, 0.7695217728614807, 0.05944693088531494, 0.21021157503128052, 0.9622620940208435, 0.05277824401855469, 0.41925930976867676, 0.2984520196914673, 0.6217151284217834, 0.5777093172073364, 0.22515296936035156, 0.4607864022254944, 0.3954867124557495, 0.35193300247192383, 0.06935739517211914, 0.3298390507698059, 0.21701264381408691, 0.25625675916671753, 0.15327781438827515, 0.93097984790802, 0.3340921998023987, 0.6426529884338379, 0.2099970579147339, 0.4895322322845459, 0.09787631034851074, 0.30711430311203003, 0.27238523960113525, 0.5879901051521301, 0.2834545969963074, 0.6519044041633606, 0.25292569398880005, 0.2717941999435425, 0.11031901836395264, 0.3443039059638977, 0.46519720554351807, 0.09621483087539673, 0.3749857544898987, 0.12687557935714722, 0.8098708987236023, 0.5927818417549133, 0.534374475479126, 0.9572424292564392, 0.49814337491989136, 0.4305194020271301, 0.7192270159721375, 0.34509968757629395, 0.621332585811615, 0.18640929460525513, 0.24078965187072754, 0.5251697897911072, 0.10209774971008301, 0.3634445071220398, 0.5303943753242493, 0.09486079216003418, 0.543369472026825, 0.8780839443206787, 0.45860421657562256, 0.0801960825920105, 0.23874837160110474, 0.22061783075332642, 0.4687035083770752, 0.7384377717971802, 0.4413255453109741, 0.6774985790252686, 0.6281447410583496, 0.20072829723358154, 0.5016149878501892, 0.6644768118858337, 0.4741876721382141, 0.8091378211975098, 0.8071659803390503, 0.008097827434539795, 0.631497323513031, 0.4867326617240906, 0.21624755859375, 0.2937180995941162, 0.7523776888847351, 0.1620488166809082, 0.6481692790985107, 0.26870137453079224, 0.4064186215400696, 0.8554061651229858, 0.9086374044418335, 0.32665371894836426, 0.43736374378204346, 0.20817142724990845, 0.20551162958145142, 0.018376469612121582, 0.13294625282287598, 0.8390915989875793, 0.7913748025894165, 0.31655603647232056, 0.14497095346450806, 0.40112197399139404, 0.15812748670578003, 0.1109616756439209, 0.24727004766464233, 0.4241828918457031, 0.24176108837127686, 0.2567214369773865, 0.30264919996261597, 0.6889566779136658, 0.7461356520652771, 0.4245458245277405, 0.5999674201011658, 0.47163569927215576, 0.22386479377746582, 0.10777127742767334, 0.07994896173477173, 0.0922965407371521, 0.21808898448944092, 0.31665104627609253, 0.12103086709976196, 0.4430199861526489, 0.34776830673217773, 0.7005773782730103, 0.18534427881240845, 0.19924068450927734, 0.2996177673339844, 0.1650601625442505, 0.13583242893218994, 0.23718732595443726, 0.043887555599212646, 0.389360249042511, 0.7969056963920593, 0.4140530824661255, 0.7353425025939941, 0.05193740129470825, 0.35004889965057373, 0.7948887348175049, 0.5140654444694519, 0.3898278474807739, 0.07650250196456909, 0.5387775301933289, 0.5538138151168823, 0.1732136607170105, 0.103954017162323, 0.18615710735321045, 0.750939667224884, 0.7772071361541748, 0.22297686338424683, 0.11969220638275146, 0.1373162865638733, 0.2726778984069824, 0.813755214214325, 0.08768779039382935, 0.06098055839538574, 0.09066390991210938, 0.6158274412155151, 0.3621222972869873, 0.41177070140838623, 0.08321988582611084, 0.3903433084487915, 0.3568918704986572, 0.415763795375824, 0.5783977508544922, 0.12885236740112305, 0.38049447536468506, 0.23658859729766846, 0.07368624210357666, 0.052982449531555176, 0.15699541568756104, 0.6420234441757202, 0.10949039459228516, 0.47685664892196655, 0.16405409574508667, 0.22411400079727173, 0.054755330085754395, 0.4980119466781616, 0.7200546264648438, 0.045873284339904785, 0.11390602588653564, 0.20721161365509033, 0.1005474328994751, 0.40667736530303955, 0.23824602365493774, 0.09922486543655396, 0.43126773834228516, 0.6355616450309753, 0.449337899684906, 0.9709419012069702, 0.6891194581985474, 0.20537787675857544, 0.16631639003753662, 0.2167375683784485, 0.39069122076034546, 0.2878105044364929, 0.3440815210342407, 0.46589183807373047, 0.04418998956680298, 0.0638495683670044, 0.6110357046127319, 0.1422223448753357, 0.4437776803970337, 0.6988321542739868, 0.7942317724227905, 0.7612178325653076, 0.06638473272323608, 0.5375131368637085, 0.298639714717865, 0.5164388418197632, 0.37637126445770264, 0.648511528968811, 0.19717031717300415, 0.28648465871810913, 0.6425784826278687, 0.49345213174819946, 0.20560109615325928, 0.15841466188430786, 0.03157317638397217, 0.17453420162200928, 0.05285459756851196, 0.14974796772003174, 0.17626208066940308, 0.3977377414703369, 0.6957342028617859, 0.3712725043296814, 0.002657175064086914, 0.5893486142158508, 0.05993926525115967, 0.7581847310066223, 0.37634092569351196, 0.08442294597625732, 0.21359401941299438, 0.26385068893432617, 0.5950650572776794, 0.8081335425376892, 0.36432772874832153, 0.17548906803131104, 0.22341835498809814, 0.009014904499053955, 0.288829505443573, 0.03377974033355713, 0.1458835005760193, 0.17074084281921387, 0.8458095192909241, 0.024354994297027588, 0.6700069308280945, 0.3515968918800354, 0.1835576891899109, 0.3612326979637146, 0.21031290292739868, 0.08450621366500854, 0.48841190338134766, 0.5239866375923157, 0.7163066864013672, 0.1053546667098999, 0.2855108380317688, 0.5936889052391052, 0.21781861782073975, 0.29493099451065063, 0.024220526218414307, 0.19229531288146973, 0.42488014698028564, 0.45923107862472534, 0.11234194040298462, 0.3674851655960083, 0.5082831978797913, 0.3199044466018677, 0.7891455888748169, 0.09726357460021973, 0.253274142742157, 0.22475796937942505, 0.0617598295211792, 0.6355436444282532, 0.5539035201072693, 0.4476540684700012, 0.12630975246429443, 0.29774361848831177, 0.3475193381309509, 0.05109572410583496, 0.27365607023239136, 0.011876523494720459, 0.15905386209487915, 0.2798941731452942, 0.41383105516433716, 0.5830579400062561, 0.49753284454345703, 0.3588895797729492, 0.18604952096939087, 0.3727109432220459, 0.2767795920372009, 0.06061136722564697, 0.43855446577072144, 0.6511826515197754, 0.900115430355072, 0.19044101238250732, 0.25843876600265503, 0.4803559184074402, 0.5019094944000244, 0.3173629641532898, 0.4068871736526489, 0.1647084355354309, 0.04286402463912964, 0.26918715238571167, 0.035914599895477295, 0.04775440692901611, 0.13652819395065308, 0.7753229737281799, 0.2245769500732422, 0.2773303985595703, 0.11795169115066528, 0.007364332675933838, 0.44757407903671265, 0.7571106553077698, 0.0396648645401001, 0.4657883644104004, 0.37688398361206055, 0.28888440132141113, 0.7717065215110779, 0.07467734813690186, 0.808358371257782, 0.21362954378128052, 0.22398900985717773, 0.12880390882492065, 0.10047763586044312, 0.8376563191413879, 0.0025236010551452637, 0.5327352285385132, 0.09842705726623535, 0.2344382405281067, 0.28507477045059204, 0.7430756092071533, 0.006992459297180176, 0.28154098987579346, 0.44846004247665405, 0.6888616681098938, 0.0966760516166687, 0.03642630577087402, 0.08691370487213135, 0.8073384165763855, 0.27715688943862915, 0.08170288801193237, 0.08557260036468506, 0.5973537564277649, 0.8756207823753357, 0.2420729398727417, 0.8165029883384705, 0.3230906128883362, 0.5795632004737854, 0.6239054203033447, 0.036471784114837646, 0.6154602766036987, 0.003908097743988037, 0.27941155433654785, 0.032808780670166016, 0.026160776615142822, 0.012334644794464111, 0.21086221933364868, 0.1134270429611206, 0.37221962213516235, 0.6739193201065063, 0.1338266134262085, 0.5504186749458313, 0.780898928642273, 0.1255509853363037, 0.8790408968925476, 0.14319825172424316, 0.7349464297294617, 0.7253897190093994, 0.2287914752960205, 0.26592016220092773, 0.2171010971069336, 0.18896150588989258, 0.22142750024795532, 0.793001651763916, 0.7771344780921936, 0.4990304708480835, 0.31272339820861816, 0.0006992816925048828, 0.162364661693573, 0.04788273572921753, 0.3165815472602844, 0.31135088205337524, 0.1760280728340149, 0.17970138788223267, 0.0725511908531189, 0.1499772071838379, 0.2585962414741516, 0.07213014364242554, 0.008965790271759033, 0.2976047992706299, 0.24541592597961426, 0.5601502060890198, 0.12203508615493774, 0.5290732383728027, 0.22868454456329346, 0.0421825647354126, 0.04921644926071167, 0.06238210201263428, 0.002119779586791992, 0.4890117645263672, 0.17665350437164307, 0.23368364572525024, 0.7827293276786804, 0.18644028902053833, 0.7076835036277771, 0.3637503981590271, 0.16626191139221191, 0.14119797945022583, 0.05605614185333252, 0.3516080379486084, 0.37996727228164673, 0.019378364086151123, 0.581508457660675, 0.25127410888671875, 0.5399290323257446, 0.028416752815246582, 0.5681513547897339, 0.8298743963241577, 0.5661319494247437, 0.7478215098381042, 0.0870010256767273, 0.03542143106460571, 0.33663851022720337, 0.6072749495506287, 0.6108559966087341, 0.4440537095069885, 0.8722466826438904, 0.44629472494125366, 0.13436466455459595, 0.39525938034057617, 0.1074419617652893, 0.4803431034088135, 0.5908960700035095, 0.24116617441177368, 0.3862465023994446, 0.1235055923461914, 0.4432588815689087, 0.016377747058868408, 0.0021843314170837402, 0.6010971665382385, 0.253976047039032, 0.37238621711730957, 0.662272572517395, 0.08182394504547119, 0.17943161725997925, 0.34543895721435547, 0.41246819496154785, 0.11515355110168457, 0.0554240345954895, 0.4974362850189209, 0.6020419001579285, 0.4158613681793213, 0.3762860894203186, 0.21594339609146118, 0.6020311117172241, 0.8911558985710144, 0.3763158321380615, 0.5128079652786255, 0.8401793837547302, 0.21810775995254517, 0.38376325368881226, 0.2288140058517456, 0.7741578221321106, 0.3299090266227722, 0.5231912136077881, 0.728361189365387, 0.8062520623207092, 0.25345903635025024, 0.3111995458602905, 0.4728201627731323, 0.25166213512420654, 0.30578184127807617, 0.1680009365081787, 0.3758038878440857, 0.030484914779663086, 0.14155489206314087, 0.30673325061798096, 0.4139817953109741, 0.2426251769065857, 0.10220009088516235, 0.1858072280883789, 0.7014700174331665, 0.5478894114494324, 0.7384885549545288, 0.3860764503479004, 0.27582037448883057, 0.6381485462188721, 0.2075473666191101, 0.24703413248062134, 0.6854774355888367, 0.19013714790344238, 0.08209431171417236, 0.14130795001983643, 0.40834981203079224, 0.3619999885559082, 0.6968251466751099, 0.35124725103378296, 0.0266454815864563, 0.11354631185531616, 0.5005823969841003, 0.0880744457244873, 0.05819052457809448, 0.12860870361328125, 0.17371827363967896, 0.19089549779891968, 0.12143045663833618, 0.05336320400238037, 0.2390572428703308, 0.23722320795059204, 0.27652573585510254, 0.13343733549118042, 0.07038062810897827, 0.34370702505111694, 0.2246330976486206, 0.43205028772354126, 0.03689157962799072, 0.50362628698349, 0.2604959011077881, 0.4535441994667053, 0.29249435663223267, 0.6247938871383667, 0.019976377487182617, 0.1648322343826294, 0.509848415851593, 0.1635606288909912, 0.4095398783683777, 0.01955336332321167, 0.6137523055076599, 0.6292976140975952, 0.17749041318893433, 0.6878218054771423, 0.6391528844833374, 0.028711557388305664, 0.2995638847351074, 0.5782412886619568, 0.27708882093429565, 0.15162688493728638, 0.08976423740386963, 0.1146973967552185, 0.060901641845703125, 0.592858076095581, 0.32980263233184814, 0.13086211681365967, 0.4722474217414856, 0.39363157749176025, 0.15994256734848022, 0.10714441537857056, 0.2816929817199707, 0.3644036054611206, 0.011641144752502441, 0.01595020294189453, 0.294230580329895, 0.28615856170654297, 0.36199694871902466, 0.03312641382217407, 0.5438553094863892, 0.037944257259368896, 0.3503471612930298, 0.4655870199203491, 0.17668968439102173, 0.3945273160934448, 0.20665597915649414, 0.40034574270248413, 0.011027276515960693, 0.15076398849487305, 0.6216145157814026, 0.6565403938293457, 0.2060588002204895, 0.6321326494216919, 0.45770835876464844, 0.1739504337310791, 0.009540319442749023, 0.06570899486541748, 0.5332839488983154, 0.1793009638786316, 0.12125271558761597, 0.21048378944396973, 0.03483116626739502, 0.12575358152389526, 0.42597877979278564, 0.06776458024978638, 0.2468353509902954, 0.5282444357872009, 0.6528518199920654, 0.21891963481903076, 0.02412855625152588, 0.511200487613678, 0.42256981134414673, 0.5175745487213135, 0.7397880554199219, 0.18515032529830933, 0.4866904020309448, 0.9364588260650635, 0.0072650909423828125, 0.7756879925727844, 0.3102964758872986, 0.21591490507125854, 0.24369794130325317, 0.18892830610275269, 0.4932628870010376, 0.053392648696899414, 0.44552290439605713, 0.8087101578712463, 0.19786232709884644, 0.2883959412574768, 0.1868630051612854, 0.5482742786407471, 0.16320353746414185, 0.1577182412147522, 0.3226879835128784, 0.9233748912811279, 0.05542325973510742, 0.08133584260940552, 0.5973826050758362, 0.5129410028457642, 0.2022320032119751, 0.5254068374633789, 0.005660057067871094, 0.040110766887664795, 0.14002472162246704, 0.41963112354278564, 0.6232847571372986, 0.580873966217041, 0.607714056968689, 0.11491662263870239, 0.01490086317062378, 0.35837358236312866, 0.13397467136383057, 0.24690783023834229, 0.3174377679824829, 0.13661396503448486, 0.10475742816925049, 0.20579558610916138, 0.4723995327949524, 0.8487265110015869, 0.06694293022155762, 0.1622524857521057, 0.9328507781028748, 0.1664670705795288, 0.32674241065979004, 0.6669134497642517, 0.696814775466919, 0.38739460706710815, 0.24768394231796265, 0.3989894986152649, 0.20696735382080078, 0.4979574680328369, 0.5545540452003479, 0.40457481145858765, 0.10451865196228027, 0.02505052089691162, 0.11608201265335083, 0.1284005045890808, 0.26385122537612915, 0.4817550778388977, 0.16873830556869507, 0.49373167753219604, 0.6149609684944153, 0.01383274793624878, 0.36142832040786743, 0.28822726011276245, 0.6001899838447571, 0.4390838146209717, 0.17555534839630127, 0.140597403049469, 0.02523404359817505, 0.777247428894043, 0.2581702470779419, 0.9024813771247864, 0.6411585807800293, 0.5106903910636902, 0.0658993124961853, 0.6907582879066467, 0.7842371463775635, 0.2615949511528015, 0.01282268762588501, 0.1792391538619995, 0.2805975079536438, 0.18345391750335693, 0.3628765940666199, 0.02142423391342163, 0.03233307600021362, 0.5692451000213623, 0.2084888219833374, 0.048098087310791016, 0.5281402468681335, 0.6553629040718079, 0.6050107479095459, 0.5398120284080505, 0.03682756423950195, 0.013035595417022705, 0.17280614376068115, 0.5395709276199341, 0.04873347282409668, 0.8921509385108948, 0.1397441029548645, 0.8241528868675232, 0.7209229469299316, 0.22824937105178833, 0.10206353664398193, 0.1939067840576172, 0.37367892265319824, 0.08825719356536865, 0.08252513408660889, 0.07340145111083984, 0.29577696323394775, 0.1164138913154602, 0.7819495797157288, 0.19902276992797852, 0.4921191334724426, 0.0159110426902771, 0.04644352197647095, 0.6240444183349609, 0.18044191598892212, 0.06808584928512573, 0.16776525974273682, 0.4658469557762146, 0.24331921339035034, 0.3074524402618408, 0.21614325046539307, 0.4299429655075073, 0.49187028408050537, 0.6416921615600586, 0.3539363145828247, 0.019749104976654053, 0.2445506453514099, 0.20602142810821533, 0.38497722148895264, 0.18262088298797607, 0.29701727628707886, 0.09333407878875732, 0.15569442510604858, 0.03604847192764282, 0.007383286952972412, 0.604541540145874, 0.5166152715682983, 0.3239179253578186, 0.1526956558227539, 0.3974815607070923, 0.37070536613464355, 0.7974098324775696, 0.6776232719421387, 0.2455570101737976, 0.5810847878456116, 0.18448305130004883, 0.5556321740150452, 0.4377148151397705, 0.3274935483932495, 0.3300057053565979, 0.4166598320007324, 0.10283803939819336, 0.051939189434051514, 0.2938363552093506, 0.3356431722640991, 0.49966317415237427, 0.01812124252319336, 0.008601486682891846, 0.7600026726722717, 0.72503662109375, 0.2512129545211792, 0.26361095905303955, 0.11653566360473633, 0.18266993761062622, 0.6737304329872131, 0.22628557682037354, 0.9865127205848694, 0.251450777053833, 0.4921818971633911, 0.6914569139480591, 0.27089762687683105, 0.4316525459289551, 0.0819198489189148, 0.5841293334960938, 0.45883142948150635, 0.29224133491516113, 0.38335567712783813, 0.8143904805183411, 0.3285036087036133, 0.6695154309272766, 0.20244181156158447, 0.3935145139694214, 0.139165461063385, 0.04807859659194946, 0.4122609496116638, 0.3355552554130554, 0.26264065504074097, 0.9139415621757507, 0.13187330961227417, 0.37213069200515747, 0.14487224817276, 0.19207030534744263, 0.3350265622138977, 0.7400436997413635, 0.5881142020225525, 0.3860548138618469, 0.6528943777084351, 0.35232555866241455, 0.40041327476501465, 0.3390704393386841, 0.5023207068443298, 0.281264066696167, 0.2359587550163269, 0.24529176950454712, 0.37482261657714844, 0.01843726634979248, 0.56291264295578, 0.055423200130462646, 0.6157270669937134, 0.553879976272583, 0.17309832572937012, 0.3145713210105896, 0.5325942039489746, 0.6270970702171326, 0.051740050315856934, 0.13247185945510864, 0.6175615191459656, 0.4435616731643677, 0.2654419541358948, 0.46253830194473267, 0.634872555732727, 0.4538130760192871, 0.796856701374054, 0.4346012473106384, 0.14007407426834106, 0.2664199471473694, 0.29015398025512695, 0.06815826892852783, 0.2702234387397766, 0.4201415777206421, 0.22292083501815796, 0.11829203367233276, 0.3709377646446228, 0.6914660930633545, 0.5882707834243774, 0.42022961378097534, 0.4238227605819702, 0.05555182695388794, 0.3274400234222412, 0.28806906938552856, 0.07169383764266968, 0.12305724620819092, 0.9202578663825989, 0.02107715606689453, 0.029858410358428955, 0.40788739919662476, 0.4152538776397705, 0.44518399238586426, 0.776672899723053, 0.0186273455619812, 0.522126317024231, 0.002026975154876709, 0.45580029487609863, 0.06421816349029541, 0.5332412123680115, 0.07739293575286865, 0.9246625900268555, 0.21625351905822754, 0.6835017800331116, 0.04819369316101074, 0.43283766508102417, 0.5039318203926086, 0.8068701028823853, 0.42814338207244873, 0.16463744640350342, 0.12979644536972046, 0.1356114149093628, 0.2456621527671814, 0.1801137924194336, 0.024955928325653076, 0.5264312028884888, 0.09358984231948853, 0.3345640301704407, 0.2934766411781311, 0.4030855894088745, 0.00147324800491333, 0.5780050754547119, 0.2300437092781067, 0.6919966340065002, 0.6464875340461731, 0.0527113676071167, 0.3376956582069397, 0.31724947690963745, 0.6951375007629395, 0.5502473711967468, 0.08144211769104004, 0.6815445423126221, 0.19588243961334229, 0.7886415123939514, 0.518974244594574, 0.3046475648880005, 0.24324792623519897, 0.8731927275657654, 0.4850369691848755, 0.30610865354537964, 0.2778615951538086, 0.3697510361671448, 0.29052090644836426, 0.4528690576553345, 0.13317900896072388, 0.20223236083984375, 0.06625485420227051, 0.4257625341415405, 0.2726994752883911, 0.5126298069953918, 0.9231674075126648, 0.3517892360687256, 0.268871009349823, 0.005492806434631348, 0.1816595196723938, 0.053367555141448975, 0.3253466486930847, 0.5066447257995605, 0.6428518891334534, 0.27517610788345337, 0.006903707981109619, 0.8769175410270691, 0.18170547485351562, 0.34028106927871704, 0.606518566608429, 0.10125339031219482, 0.21502745151519775, 0.4388212561607361, 0.2431454062461853, 0.17763465642929077, 0.6233797669410706, 0.11666607856750488, 0.9178521633148193, 0.19719022512435913, 0.06821942329406738, 0.2819068431854248, 0.3144831657409668, 0.9145074486732483, 0.11519426107406616, 0.4090462923049927, 0.10738521814346313, 0.2020159363746643, 0.11953788995742798, 0.636717677116394, 0.10755997896194458, 0.5224930047988892, 0.7110252976417542, 0.19834458827972412, 0.6570678949356079, 0.36837029457092285, 0.3558863401412964, 0.7051932215690613, 0.5253391265869141, 0.03602862358093262, 0.3746569752693176, 0.08283185958862305, 0.486314058303833, 0.04911911487579346, 0.9727455973625183, 0.44922107458114624, 0.13293105363845825, 0.3850577473640442, 0.40207988023757935, 0.45309334993362427, 0.21595919132232666, 0.17278504371643066, 0.2501407265663147, 0.30309927463531494, 0.04473578929901123, 0.7187204360961914, 0.9303106069564819, 0.6320767402648926, 0.6413401961326599, 0.03694802522659302, 0.6965355277061462, 0.8931398391723633, 0.33492255210876465, 0.4843463897705078, 0.5948237776756287, 0.7380577921867371, 0.25685060024261475, 0.42732691764831543, 0.7594627141952515, 0.6840957403182983, 0.6799648404121399, 0.8868497610092163, 0.9015973210334778, 0.20893394947052002, 0.10647344589233398, 0.8584263920783997, 0.3194982409477234, 0.4960283637046814, 0.13646775484085083, 0.7666262984275818, 0.2380765676498413, 0.24843931198120117, 0.020675599575042725, 0.7837091088294983, 0.30850130319595337, 0.3930017352104187, 0.2769097089767456, 0.8214256167411804, 0.5192790627479553, 0.6495228409767151, 0.08838355541229248, 0.49360865354537964, 0.34327608346939087, 0.6891725063323975, 0.41405045986175537, 0.24678099155426025, 0.17222076654434204, 0.33189690113067627, 0.20584321022033691, 0.5385187268257141, 0.033836543560028076, 0.5375051498413086, 0.3929201364517212, 0.45404767990112305, 0.7950960993766785, 0.44496703147888184, 0.3280733823776245, 0.3036158084869385, 0.9620717763900757, 0.27658456563949585, 0.40171748399734497, 0.45892542600631714, 0.032968759536743164, 0.07659715414047241, 0.02373051643371582, 0.056752026081085205, 0.03860384225845337, 0.11329716444015503, 0.524972677230835, 0.6116805672645569, 0.25375044345855713, 0.12800270318984985, 0.20666402578353882, 0.47830259799957275, 0.014844000339508057, 0.44059425592422485, 0.922926127910614, 0.32862287759780884, 0.5830914974212646, 0.03240323066711426, 0.5396360158920288, 0.02367943525314331, 0.13672757148742676, 0.25722014904022217, 0.5125454664230347, 0.1874607801437378, 0.2702518105506897, 0.17985570430755615, 0.22387421131134033, 0.4781956076622009, 0.4134122133255005, 0.1933024525642395, 0.09651583433151245, 0.2545700669288635, 0.8104286193847656, 0.20152050256729126, 0.07763946056365967, 0.017438828945159912, 0.10515367984771729, 0.5931341648101807, 0.7975292205810547, 0.5176846981048584, 0.06954962015151978, 0.378060519695282, 0.4617001414299011, 0.2068139910697937, 0.020810246467590332, 0.4113221764564514, 0.8310896158218384, 0.15419918298721313, 0.23616480827331543, 0.40606993436813354, 0.11131572723388672, 0.07682192325592041, 0.4120759963989258, 0.34518808126449585, 0.03141438961029053, 0.5116945505142212, 0.17425328493118286, 0.6625683903694153, 0.12246650457382202, 0.8868616819381714, 0.2452409267425537, 0.5708637237548828, 0.6173357963562012, 0.4108881950378418, 0.7572691440582275, 0.538906991481781, 0.337812602519989, 0.44535136222839355, 0.23880118131637573, 0.019033491611480713, 0.71951824426651, 0.11012238264083862, 0.08745735883712769, 0.7637671828269958, 0.15630698204040527, 0.09474015235900879, 0.180633544921875, 0.19997060298919678, 0.39060479402542114, 0.10029387474060059, 0.3235247731208801, 0.09693068265914917, 0.08565020561218262, 0.38043785095214844, 0.045549750328063965, 0.6269279718399048, 0.03144073486328125, 0.6995002031326294, 0.4520377516746521, 0.12228995561599731, 0.019259333610534668, 0.9146525263786316, 0.6970286965370178, 0.06567758321762085, 0.33478713035583496, 0.11187863349914551, 0.29141300916671753, 0.6274503469467163, 0.09440147876739502, 0.23970335721969604, 0.10306167602539062, 0.5321874618530273, 0.5048973560333252, 0.25495827198028564, 0.6220899224281311, 0.6479402184486389, 0.19174933433532715, 0.12845540046691895, 0.735669732093811, 0.1491929292678833, 0.021449506282806396, 0.4250069260597229, 0.10053926706314087, 0.4844251275062561, 0.10971921682357788, 0.5503564476966858, 0.3875620365142822, 0.05654507875442505, 0.057436347007751465, 0.8045387864112854, 0.48831290006637573, 0.041381895542144775, 0.8705611228942871, 0.5098862648010254, 0.22722142934799194, 0.5844165682792664, 0.4753038287162781, 0.3346777558326721, 0.2013515830039978, 0.6135591268539429, 0.24939769506454468, 0.39158642292022705, 0.8486123085021973, 0.10682642459869385, 0.6983195543289185, 0.028818607330322266, 0.32811689376831055, 0.33895325660705566, 0.9887776374816895, 0.09146666526794434, 0.18363183736801147, 0.3524501919746399, 0.2456343173980713, 0.11367350816726685, 0.6002746224403381, 0.000676274299621582, 0.23266279697418213, 0.5313782095909119, 0.35104942321777344, 0.6975381970405579, 0.16735190153121948, 0.25601935386657715, 0.008680105209350586, 0.5028682947158813, 0.20575684309005737, 0.07553523778915405, 0.1291387677192688, 0.2741934061050415, 0.40870606899261475, 0.1540616750717163, 0.46477365493774414, 0.10861384868621826, 0.056159913539886475, 0.05447888374328613, 0.23969155550003052, 0.2546204924583435, 0.1726204752922058, 0.4399455785751343, 0.3382209539413452, 0.6805760860443115, 0.19639992713928223, 0.28498613834381104, 0.46322596073150635, 0.6375208497047424, 0.3010183572769165, 0.28951048851013184, 0.23973298072814941, 0.6694410443305969, 0.8634433746337891, 0.28786927461624146, 0.6299746632575989, 0.6173589825630188, 0.25610512495040894, 0.48000192642211914, 0.22723960876464844, 0.044322967529296875, 0.26051998138427734, 0.19954925775527954, 0.39924293756484985, 0.15463846921920776, 0.8584931492805481, 0.5209148526191711, 0.5563279390335083, 0.19081151485443115, 0.7184503078460693, 0.21102207899093628, 0.20739591121673584, 0.6288710832595825, 0.3590569496154785, 0.34389132261276245, 0.0005899667739868164, 0.04695117473602295, 0.2043600082397461, 0.36659616231918335, 0.19554752111434937, 0.434939444065094, 0.09203875064849854, 0.03563547134399414, 0.22660988569259644, 0.09054964780807495, 0.737872838973999, 0.006879568099975586, 0.2874246835708618, 0.030610382556915283, 0.46605825424194336, 0.5285146236419678, 0.6665037274360657, 0.09910070896148682, 0.21496737003326416, 0.06788516044616699, 0.6593487858772278, 0.07402223348617554, 0.007312774658203125, 0.11031681299209595, 0.4890900254249573, 0.1471044421195984, 0.021220684051513672, 0.5256764888763428, 0.3745194673538208, 0.46229052543640137, 0.4602605104446411, 0.7772579789161682, 0.2633187770843506, 0.6582133173942566, 0.6349856853485107, 0.4894949793815613, 0.003748953342437744, 0.06577193737030029, 0.052232980728149414, 0.502245306968689, 0.35431772470474243, 0.2647022604942322, 0.8024634122848511, 0.11352789402008057, 0.6406393647193909, 0.14768582582473755, 0.09234631061553955, 0.6353859901428223, 0.17529016733169556, 0.01623302698135376, 0.13825011253356934, 0.2717403173446655, 0.39472073316574097, 0.2296086549758911, 0.701137125492096, 0.03972208499908447, 0.25457483530044556, 0.012691855430603027, 0.6478234529495239, 0.42537736892700195, 0.004320919513702393, 0.7815893888473511, 0.41200482845306396, 0.03233844041824341, 0.08113092184066772, 0.6161306500434875, 0.8704028129577637, 0.03844559192657471, 0.08019602298736572, 0.2602686285972595, 0.09502136707305908, 0.20455938577651978, 0.4882934093475342, 0.5374552011489868, 0.12769275903701782, 0.6769917011260986, 0.9105322360992432, 0.16114121675491333, 0.02356266975402832, 0.4585840702056885, 0.14546716213226318, 0.1161048412322998, 0.43612945079803467, 0.2928740382194519, 0.43024593591690063, 0.3837445378303528, 0.00753408670425415, 0.6319988965988159, 0.15560173988342285, 0.0342332124710083, 0.8136898875236511, 0.7656915783882141, 0.16317063570022583, 0.16835564374923706, 0.010240495204925537, 0.08744484186172485, 0.4522072672843933, 0.3055644631385803, 0.581809937953949, 0.05701977014541626, 0.7319738268852234, 0.238864004611969, 0.7861205339431763, 0.09738236665725708, 0.27736562490463257, 0.05222463607788086, 0.033627867698669434, 0.6181837916374207, 0.07526952028274536, 0.8706982731819153, 0.5426169037818909, 0.6014606952667236, 0.09625935554504395, 0.09638375043869019, 0.38558346033096313, 0.03217113018035889, 0.13091987371444702, 0.7305147051811218, 0.033928632736206055, 0.28746742010116577, 0.857101321220398, 0.6187069416046143, 0.03895968198776245, 0.43226778507232666, 0.5246488451957703, 0.6296117305755615, 0.0039577484130859375, 0.42206907272338867, 0.07465928792953491, 0.39912307262420654, 0.5488607287406921, 0.8157768845558167, 0.10852283239364624, 0.9417276382446289, 0.1704043745994568, 0.3564026355743408, 0.22608202695846558, 0.2535362243652344, 0.03124523162841797, 0.6866942644119263, 0.08049875497817993, 0.5119099617004395, 0.15043914318084717, 0.42921358346939087, 0.17447638511657715, 0.2851634621620178, 0.3156108856201172, 0.03730207681655884, 0.24916893243789673, 0.07556819915771484, 0.04300504922866821, 0.060425400733947754, 0.24375003576278687, 0.09761583805084229, 0.056739211082458496, 0.02461904287338257, 0.1342373490333557, 0.9763111472129822, 0.5122825503349304, 0.44903290271759033, 0.898532509803772, 0.7089390754699707, 0.04652327299118042, 0.7749232053756714, 0.11205244064331055, 0.09737825393676758, 0.26163625717163086, 0.13852828741073608, 0.09703719615936279, 0.5643923282623291, 0.28598731756210327, 0.05605381727218628, 0.1559881567955017, 0.3998118042945862, 0.6540526747703552, 0.10276973247528076, 0.24470221996307373, 0.08422833681106567, 0.8231749534606934, 0.06892532110214233, 0.736376941204071, 0.20235109329223633, 0.6970036625862122, 0.15524673461914062, 0.2436962127685547, 0.9808788299560547, 0.4744030833244324, 0.6151772141456604, 0.08645874261856079, 0.04244053363800049, 0.21305131912231445, 0.0975072979927063, 0.10340988636016846, 0.262935996055603, 0.35067999362945557, 0.4947444200515747, 0.5415550470352173, 0.06363451480865479, 0.14246153831481934, 0.7946909666061401, 0.3393248915672302, 0.05090242624282837, 0.010310530662536621, 0.20439249277114868, 0.5145833492279053, 0.12204265594482422, 0.4835212826728821, 0.49114537239074707, 0.21587562561035156, 0.38984358310699463, 0.8400095105171204, 0.16725283861160278, 0.6634829044342041, 0.23943787813186646, 0.07061111927032471, 0.5275742411613464, 0.5700786709785461, 0.22032874822616577, 0.7374411821365356, 0.006784915924072266, 0.07412797212600708, 0.007368803024291992, 0.28346502780914307, 0.1726628541946411, 0.07183283567428589, 0.27621883153915405, 0.6838594079017639, 0.3743683099746704, 0.7748096585273743, 0.08734822273254395, 0.639488697052002, 0.05144447088241577, 0.10283094644546509, 0.390733003616333, 0.5925121903419495, 0.34652918577194214, 0.22912836074829102, 0.1277986764907837, 0.0037616491317749023, 0.3418440818786621, 0.39014869928359985, 0.3483394980430603, 0.6639226675033569, 0.48955273628234863, 0.7571417689323425, 0.4741654396057129, 0.4232857823371887, 0.358620285987854, 0.4867956042289734, 0.2266712188720703, 0.4007294178009033, 0.8407384157180786, 0.47279053926467896, 0.4047154188156128, 0.13830137252807617, 0.48998868465423584, 0.19127434492111206, 0.20491379499435425, 0.13477641344070435, 0.010518431663513184, 0.38115644454956055, 0.24903619289398193, 0.28964340686798096, 0.41079598665237427, 0.17140567302703857, 0.2051854133605957, 0.32574528455734253, 0.03389376401901245, 0.6810402870178223, 0.48574334383010864, 0.5128228664398193, 0.16006582975387573, 0.22141236066818237, 0.5103347301483154, 0.9459923505783081, 0.17659693956375122, 0.44988346099853516, 0.3834936022758484, 0.13086450099945068, 0.5852781534194946, 0.7844806909561157, 0.629534900188446, 0.4210089445114136, 0.23731380701065063, 0.13030093908309937, 0.203086256980896, 0.6277090907096863, 0.022008836269378662, 0.5134907364845276, 0.5198687314987183, 0.5276901125907898, 0.32744455337524414, 0.11494332551956177, 0.12445980310440063, 0.36587631702423096, 0.05426037311553955, 0.6418012380599976, 0.2756496071815491, 0.3576256036758423, 0.08919310569763184, 0.23639994859695435, 0.1358584761619568, 0.19090873003005981, 0.423103392124176, 0.084919273853302, 0.0732353925704956, 0.3580176830291748, 0.7868161797523499, 0.26467061042785645, 0.28250616788864136, 0.23260372877120972, 0.023538172245025635, 0.045056819915771484, 0.3952106833457947, 0.49611788988113403, 0.4446009397506714, 0.1722785234451294, 0.058555424213409424, 0.3832051753997803, 0.5047551989555359, 0.07137691974639893, 0.1445409059524536, 0.04393500089645386, 0.7772176265716553, 0.27453750371932983, 0.1947394609451294, 0.189672589302063, 0.06846153736114502, 0.5219745635986328, 0.7664573192596436, 0.23262101411819458, 0.03392744064331055, 0.6288886666297913, 0.08615195751190186, 0.4042541980743408, 0.2659037709236145, 0.9867873787879944, 0.0529555082321167, 0.08990848064422607, 0.11213088035583496, 0.033644914627075195, 0.5347031950950623, 0.17454969882965088, 0.22639179229736328, 0.2954642176628113, 0.6770144701004028, 0.2530568838119507, 0.3076133131980896, 0.32408076524734497, 0.9081672430038452, 0.8304691314697266, 0.0615159273147583, 0.20571941137313843, 0.6420965194702148, 0.7034955024719238, 0.19925332069396973, 0.22649431228637695, 0.11212551593780518, 0.27565163373947144, 0.6624124646186829, 0.11239969730377197, 0.27859920263290405, 0.8586771488189697, 0.6311522722244263, 0.8111013174057007, 0.25496333837509155, 0.04756426811218262, 0.5281226634979248, 0.3235856294631958, 0.2119881510734558, 0.21962666511535645, 0.4369812607765198, 0.25617659091949463, 0.09786832332611084, 0.10582089424133301, 0.6466137766838074, 0.34228408336639404, 0.033208608627319336, 0.01246267557144165, 0.4669494032859802, 0.5686683058738708, 0.5416863560676575, 0.473604679107666, 0.10344994068145752, 0.7335973381996155, 0.6732344627380371, 0.3960745930671692, 0.016980409622192383, 0.3357264995574951, 0.17173731327056885, 0.5346103310585022, 0.1929715871810913, 0.40138083696365356, 0.1036638617515564, 0.47379904985427856, 0.24162477254867554, 0.7661168575286865, 0.16616445779800415, 0.1632123589515686, 0.3674190640449524, 0.7477151155471802, 0.16251516342163086, 0.7589853405952454, 0.03847479820251465, 0.037165284156799316, 0.4340859055519104, 0.033948540687561035, 0.15420562028884888, 0.6359900832176208, 0.6380605101585388, 0.19385981559753418, 0.7893283367156982, 0.06742990016937256, 0.426142156124115, 0.09986275434494019, 0.12398046255111694, 0.011915802955627441, 0.708349883556366, 0.9057815074920654, 0.1619763970375061, 0.26283615827560425, 0.44368118047714233, 0.1511966586112976, 0.48104989528656006, 0.008061468601226807, 0.1233101487159729, 0.31004440784454346, 0.2655240297317505, 0.24803084135055542, 0.5885280966758728, 0.2214874029159546, 0.10220527648925781, 0.8538228273391724, 0.6937188506126404, 0.5703872442245483, 0.0423741340637207, 0.5269749164581299, 0.2777918577194214, 0.18739980459213257, 0.6582112908363342, 0.2048041820526123, 0.03999948501586914, 0.3217869997024536, 0.5660272836685181, 0.6580908894538879, 0.07636195421218872, 0.028949081897735596, 0.25869596004486084, 0.10308152437210083, 0.6687368154525757, 0.6270069479942322, 0.784889280796051, 0.22529196739196777, 0.24969720840454102, 0.6291337013244629, 0.534144937992096, 0.16723382472991943, 0.19666045904159546, 0.3669017553329468, 0.029890596866607666, 0.2051321268081665, 0.6785968542098999, 0.005185604095458984, 0.7862958908081055, 0.23843437433242798, 0.5824100971221924, 0.07498985528945923, 0.22853654623031616, 0.5795310139656067, 0.3257026672363281, 0.3231024742126465, 0.18589311838150024, 0.06499719619750977, 0.5912923812866211, 0.5654581785202026, 0.7293639183044434, 0.4550618529319763, 0.14849668741226196, 0.28841453790664673, 0.053698718547821045, 0.20475101470947266, 0.3550715446472168, 0.49851399660110474, 0.9397283792495728, 0.27470046281814575, 0.0867013931274414, 0.19462275505065918, 0.2918025851249695, 0.18669170141220093, 0.1420089602470398, 0.13267087936401367, 0.47829896211624146, 0.5810641646385193, 0.19578373432159424, 0.0461614727973938, 0.0715264081954956, 0.1993613839149475, 0.12351423501968384, 0.17541378736495972, 0.623384416103363, 0.11932653188705444, 0.3243058919906616, 0.49952006340026855, 0.5075737833976746, 0.05050867795944214, 0.3211477994918823, 0.4435039758682251, 0.756790816783905, 0.4967191815376282, 0.1864384412765503, 0.06282740831375122, 0.12214231491088867, 0.8193162083625793, 0.04873228073120117, 0.1617792248725891, 0.4085269570350647, 0.0573616623878479, 0.0005359053611755371, 0.04849809408187866, 0.11885261535644531, 0.13662010431289673, 0.027317047119140625, 0.1488526463508606, 0.15674781799316406, 0.011900067329406738, 0.43465596437454224, 0.1320229172706604, 0.16421741247177124, 0.06297272443771362, 0.725379467010498, 0.030253291130065918, 0.692985475063324, 0.27402251958847046, 0.514889657497406, 0.4431993365287781, 0.37638944387435913, 0.2853289246559143, 0.22798007726669312, 0.03976243734359741, 0.207280695438385, 0.25217533111572266, 0.07435548305511475, 0.017699837684631348, 0.8286061882972717, 0.20128977298736572, 0.4238095283508301, 0.31771641969680786, 0.26060712337493896, 0.0970110297203064, 0.27812474966049194, 0.16755038499832153, 0.4051963686943054, 0.6924469470977783, 0.09138566255569458, 0.6469357013702393, 0.6801430583000183, 0.2867546081542969, 0.6153575778007507, 0.2203885316848755, 0.00022166967391967773, 0.04969567060470581, 0.4950128197669983, 0.1587667465209961, 0.3273049592971802, 0.3255499005317688, 0.302811861038208, 0.2669823169708252, 0.004305422306060791, 0.2236616015434265, 0.24159330129623413, 0.7853789925575256, 0.14618855714797974, 0.42848891019821167, 0.24297749996185303, 0.2713248133659363, 0.24947750568389893, 0.3587840795516968, 0.10057878494262695, 0.23300057649612427, 0.2690030336380005, 0.08016359806060791, 0.7936707139015198, 0.4055594205856323, 0.31320804357528687, 0.17700397968292236, 0.3567779064178467, 0.2823514938354492, 0.6234805583953857, 0.11431640386581421, 0.14144831895828247, 0.03969752788543701, 0.09751474857330322, 0.8434054851531982, 0.3679060935974121, 0.07427996397018433, 0.5526570677757263, 0.29540586471557617, 0.37842750549316406, 0.1103164553642273, 0.3847687244415283, 0.22497791051864624, 0.19596564769744873, 0.13996630907058716, 0.04689300060272217, 0.6185814738273621, 0.30662524700164795, 0.6580750942230225, 0.19709783792495728, 0.6556210517883301, 0.04162102937698364, 0.8336553573608398, 0.03617888689041138, 0.4287998080253601, 0.16652286052703857, 0.37477898597717285, 0.7519850134849548, 0.28800660371780396, 0.266751766204834, 0.3000604510307312, 0.11758661270141602, 0.5744357705116272, 0.3729264736175537, 0.33019107580184937, 0.4306475520133972, 0.17237085103988647, 0.42521941661834717, 0.18521720170974731, 0.22508567571640015, 0.42297154664993286, 0.4323588013648987, 0.3063002824783325, 0.1726665496826172, 0.09433120489120483, 0.4085771441459656, 0.42985081672668457, 0.029861629009246826, 0.07154470682144165, 0.031540751457214355, 0.23795849084854126, 0.7053011655807495, 0.02869093418121338, 0.24691301584243774, 0.028711140155792236, 0.4908530116081238, 0.4607238173484802, 0.29205000400543213, 0.12672406435012817, 0.12789231538772583, 0.28595954179763794, 0.1299840211868286, 0.7928822636604309, 0.2783796191215515, 0.29218900203704834, 0.5522799491882324, 0.13747382164001465, 0.07542401552200317, 0.27003639936447144, 0.40319979190826416, 0.506791353225708, 0.08915919065475464, 0.3907918930053711, 0.16876518726348877, 0.39070647954940796, 0.4091670513153076, 0.7125303149223328, 0.21043622493743896, 0.40375399589538574, 0.06129348278045654, 0.1239963173866272, 0.02009183168411255, 0.8152191638946533, 0.7409846782684326, 0.3419857621192932, 0.2734612226486206, 0.18737542629241943, 0.27706170082092285, 0.16744810342788696, 0.10788553953170776, 0.06904429197311401, 0.26209133863449097, 0.45834535360336304, 0.1705065369606018, 0.37811529636383057, 0.22269463539123535, 0.09728217124938965, 0.2973804473876953, 0.2676745653152466, 0.12779593467712402, 0.3849310874938965, 0.20812523365020752, 0.3360379934310913, 0.6575919985771179, 0.5833707451820374, 0.11570554971694946, 0.25674331188201904, 0.80021071434021, 0.40602487325668335, 0.2860393524169922, 0.6591538190841675, 0.25912195444107056, 0.06630301475524902, 0.2583996653556824, 0.06049841642379761, 0.45490843057632446, 0.8554303050041199, 0.474418580532074, 0.06797510385513306, 0.7134331464767456, 0.7149187326431274, 0.22125887870788574, 0.5054149031639099, 0.7677213549613953, 0.030247986316680908, 0.4351174235343933, 0.4532496929168701, 0.6566836833953857, 0.11701136827468872, 0.00017648935317993164, 0.6795310378074646, 0.5235510468482971, 0.11341613531112671, 0.37327271699905396, 0.10621696710586548, 0.4700815677642822, 0.34986162185668945, 0.1342703104019165, 0.20490175485610962, 0.40463680028915405, 0.051748812198638916, 0.06795823574066162, 0.655752956867218, 0.013613581657409668, 0.2785574197769165, 0.3331441879272461, 0.2499527931213379, 0.46226346492767334, 0.03520607948303223, 0.48685920238494873, 0.20235228538513184, 0.1413416862487793, 0.605267345905304, 0.27749890089035034, 0.12177467346191406, 0.5263199806213379, 0.5056405067443848, 0.12113112211227417, 0.025814473628997803, 0.9844207763671875, 0.6916800141334534, 0.295995831489563, 0.18535852432250977, 0.4876834750175476, 0.011828720569610596, 0.7797790765762329, 0.1913277506828308, 0.21973472833633423, 0.37450897693634033, 0.2529194951057434, 0.10639411211013794, 0.2647409439086914, 0.19573986530303955, 0.3314467668533325, 0.28817182779312134, 0.01629638671875, 0.5696802735328674, 0.18849575519561768, 0.36862367391586304, 0.7651388049125671, 0.33512985706329346, 0.42193180322647095, 0.05255359411239624, 0.011533081531524658, 0.15543270111083984, 0.3930822014808655, 0.03495556116104126, 0.12379670143127441, 0.037135303020477295, 0.39608365297317505, 0.4375495910644531, 0.040630877017974854, 0.2506110668182373, 0.03254044055938721, 0.5161088705062866, 0.3763654828071594, 0.1759287714958191, 0.1632307767868042, 0.2103745937347412, 0.09249365329742432, 0.1838739514350891, 0.8099634647369385, 0.22158795595169067, 0.7835487127304077, 0.08565628528594971, 0.5245886445045471, 0.013975381851196289, 0.18613433837890625, 0.629873514175415, 0.9057726860046387, 0.6392317414283752, 0.1037454605102539, 0.1216580867767334, 0.028555691242218018, 0.5182174444198608, 0.3162747025489807, 0.19281005859375, 0.005393922328948975, 0.2717072367668152, 0.35159701108932495, 0.0778665542602539, 0.22029227018356323, 0.32104092836380005, 0.6983829736709595, 0.5169909596443176, 0.004119396209716797, 0.28245794773101807, 0.12862318754196167, 0.001977384090423584, 0.28814852237701416, 0.6198976635932922, 0.5680131912231445, 0.4943079948425293, 0.025206446647644043, 0.28790903091430664, 0.15262985229492188, 0.09402889013290405, 0.25089210271835327, 0.3834984302520752, 0.5196694135665894, 0.03429323434829712, 0.8657804727554321, 0.6041672229766846, 0.10675078630447388, 0.2749060392379761, 0.48556631803512573, 0.36591124534606934, 0.09346354007720947, 0.07729780673980713, 0.13843953609466553, 0.08177757263183594, 0.37170904874801636, 0.011551380157470703, 0.4553302526473999, 0.03672736883163452, 0.2927556037902832, 0.6783217191696167, 0.11193341016769409, 0.21694904565811157, 0.2629355788230896, 0.027025699615478516, 0.2002730369567871, 0.5035089254379272, 0.6267601847648621, 0.6531094908714294, 0.09532302618026733, 0.155134379863739, 0.44622617959976196, 0.31353670358657837, 0.19317662715911865, 0.10482585430145264, 0.24619358777999878, 0.43549835681915283, 0.5336251854896545, 0.48540669679641724, 0.21033477783203125, 0.6459658145904541, 0.4027891755104065, 0.030744075775146484, 0.3285066485404968, 0.1030890941619873, 0.08426755666732788, 0.2835312485694885, 0.8386407494544983, 0.19260847568511963, 0.2066848874092102, 0.6947714686393738, 0.07626891136169434, 0.060791969299316406, 0.7225160598754883, 0.2812289595603943, 0.5016530156135559, 0.53020179271698, 0.1673993468284607, 0.3462837338447571, 0.874136745929718, 0.0990830659866333, 0.11310893297195435, 0.27090907096862793, 0.4190483093261719, 0.004071474075317383, 0.5836685299873352, 0.4430907368659973, 0.56095951795578, 0.4358087182044983, 0.15014714002609253, 0.000345766544342041, 0.47371262311935425, 0.3722417950630188, 0.4365006685256958, 0.5998761057853699, 0.03189074993133545, 0.24436593055725098, 0.09317135810852051, 0.5208309292793274, 0.38032710552215576, 0.12742102146148682, 0.7294785976409912, 0.24050652980804443, 0.1608080267906189, 0.006825387477874756, 0.07321065664291382, 0.6059191226959229, 0.09823912382125854, 0.14869654178619385, 0.5054399371147156, 0.6708993315696716, 0.07927805185317993, 0.5086889863014221, 0.32836514711380005, 0.2058579921722412, 0.1959170699119568, 0.611617386341095, 0.08395558595657349, 0.5137832760810852, 0.12520569562911987, 0.2789631485939026, 0.801054060459137, 0.3121775984764099, 0.8136216402053833, 0.25508248805999756, 0.04053056240081787, 0.13857901096343994, 0.43714600801467896, 0.19085007905960083, 0.7169196605682373, 0.07005000114440918, 0.6306611895561218, 0.29109519720077515, 0.8081613183021545, 0.016090035438537598, 0.13377195596694946, 0.18522000312805176, 0.045900702476501465, 0.4961715340614319, 0.03273117542266846, 0.5964704751968384, 0.548240602016449, 0.46206730604171753, 0.3155507445335388, 0.3046850562095642, 0.18912625312805176, 0.7279037237167358, 0.2340279221534729, 0.5769472718238831, 0.4857890009880066, 0.1694326400756836, 0.2863732576370239, 0.47049158811569214, 0.08278030157089233, 0.10872697830200195, 0.3767521381378174, 0.10663211345672607, 0.30292797088623047, 0.47479695081710815, 0.28108614683151245, 0.4929842948913574, 0.8442143797874451, 0.6281846761703491, 0.08976048231124878, 0.6879827380180359, 0.3593319058418274, 0.17653197050094604, 0.469001829624176, 0.5794009566307068, 0.7013288736343384, 0.203449547290802, 0.2380690574645996, 0.2802547812461853, 0.01625978946685791, 0.16008204221725464, 0.3967629075050354, 0.05991941690444946, 0.5769346952438354, 0.03763765096664429, 0.4955868721008301, 0.5075039863586426, 0.04575455188751221, 0.010654807090759277, 0.10591095685958862, 0.24577397108078003, 0.040273964405059814, 0.15645933151245117, 0.008806169033050537, 0.5788275003433228, 0.2845562696456909, 0.16553431749343872, 0.5993428230285645, 0.15337491035461426, 0.09261202812194824, 0.555259644985199, 0.09396147727966309, 0.02626931667327881, 0.10574513673782349, 0.17108154296875, 0.0429682731628418, 0.15318769216537476, 0.08364897966384888, 0.33532601594924927, 0.20665425062179565, 0.07526975870132446, 0.7651167511940002, 0.5161044597625732, 0.34127628803253174, 0.6243883371353149, 0.09237438440322876, 0.4867374897003174, 0.7772956490516663, 0.1949923038482666, 0.040273964405059814, 0.27158379554748535, 0.3632587194442749, 0.11979788541793823, 0.6675500273704529, 0.8076537251472473, 0.5934404730796814, 0.40951651334762573, 0.5071414709091187, 0.5085805058479309, 0.07923918962478638, 0.7318811416625977, 0.17406105995178223, 0.6877455115318298, 0.6133003830909729, 0.49359339475631714, 0.04174858331680298, 0.09918200969696045, 0.6149060726165771, 0.2350466251373291, 0.08665865659713745, 0.10364997386932373, 0.49894416332244873, 0.27917736768722534, 0.41455382108688354, 0.7966728210449219, 0.807034969329834, 0.5260967016220093, 0.25704050064086914, 0.6509689688682556, 0.6715471744537354, 0.6220211982727051, 0.33743900060653687, 0.20830166339874268, 0.00354766845703125, 0.18236327171325684, 0.047370731830596924, 0.11764556169509888, 0.134951651096344, 0.2077317237854004, 0.8993284106254578, 0.23891156911849976, 0.20526844263076782, 0.9008324146270752, 0.643345832824707, 0.3995622992515564, 0.17195773124694824, 0.7076750993728638, 0.18423140048980713, 0.06999063491821289, 0.0703546404838562, 0.11325478553771973, 0.6339349150657654, 0.46379077434539795, 0.22879528999328613, 0.5066272616386414, 0.4189009666442871, 0.3699880838394165, 0.2675495743751526, 0.1827089786529541, 0.45546412467956543, 0.398256778717041, 0.5289178490638733, 0.4902116656303406, 0.002422511577606201, 0.5381109118461609, 0.3078550100326538, 0.5988078713417053, 0.29501765966415405, 0.7020154595375061, 0.2479904294013977, 0.029143333435058594, 0.48750853538513184, 0.45608246326446533, 0.5731855034828186, 0.4252881407737732, 0.07429623603820801, 0.030094802379608154, 0.3480799198150635, 0.4185844659805298, 0.9875288605690002, 0.48068904876708984, 0.12244147062301636, 0.11566656827926636, 0.16017580032348633, 0.4012736678123474, 0.15508568286895752, 0.46965885162353516, 0.3965086340904236, 0.46507567167282104, 0.46236759424209595, 0.24434292316436768, 0.48211759328842163, 0.2705919146537781, 0.022360026836395264, 0.33294975757598877, 0.1981915831565857, 0.2906603217124939, 0.16773474216461182, 0.19064879417419434, 0.4233701229095459, 0.2373950481414795, 0.674119234085083, 0.323635458946228, 0.5784434080123901, 0.22928190231323242, 0.4999127984046936, 0.5250969529151917, 0.5709853172302246, 0.11277425289154053, 0.6052321195602417, 0.8000239729881287, 0.019069671630859375, 0.009124517440795898, 0.6671463251113892, 0.2982972264289856, 0.4297921061515808, 0.48487019538879395, 0.20072239637374878, 0.11068332195281982, 0.3235679864883423, 0.28494828939437866, 0.5495082139968872, 0.007393598556518555, 0.1225627064704895, 0.5841562151908875, 0.08944684267044067, 0.5625259876251221, 0.7479711174964905, 0.7765240669250488, 0.2400825023651123, 0.23557788133621216, 0.45044493675231934, 0.6314356923103333, 0.36663395166397095, 0.0014432668685913086, 0.9487918615341187, 0.36254554986953735, 0.3395910859107971, 0.1273370385169983, 0.2632918357849121, 0.6140110492706299, 0.20338845252990723, 0.6142899990081787, 0.24387019872665405, 0.13861477375030518, 0.4478831887245178, 0.005089461803436279, 0.053604960441589355, 0.30875229835510254, 0.7753579616546631, 0.10910052061080933, 0.2098793387413025, 0.5150649547576904, 0.06030458211898804, 0.0200197696685791, 0.25737059116363525, 0.39266151189804077, 0.3769104480743408, 0.672386109828949, 0.06801563501358032, 0.050839781761169434, 0.3603399395942688, 0.12449508905410767, 0.014626681804656982, 0.710433304309845, 0.2822057008743286, 0.22917604446411133, 0.7661101222038269, 0.9241558313369751, 0.0022096633911132812, 0.06104409694671631, 0.2982019782066345, 0.9798193573951721, 0.04584282636642456, 0.02646738290786743, 0.20497554540634155, 0.5496548414230347, 0.14420819282531738, 0.07531511783599854, 0.4624214768409729, 0.7999770641326904, 0.5532031655311584, 0.7349855899810791, 0.2737446427345276, 0.060221314430236816, 0.6227206587791443, 0.5014328956604004, 0.3657172918319702, 0.3283093571662903, 0.4002060890197754, 0.5358824133872986, 0.47142720222473145, 0.09408140182495117, 0.3002355694770813, 0.1620396375656128, 0.12151753902435303, 0.5230948328971863, 0.06268203258514404, 0.358787477016449, 0.853975772857666, 0.7831385135650635, 0.08605742454528809, 0.6752186417579651, 0.8174262046813965, 0.012263774871826172, 0.29492509365081787, 0.1995188593864441, 0.5426435470581055, 0.07411247491836548, 0.1435496211051941, 0.6281384229660034, 0.5554532408714294, 0.2159113883972168, 0.4731312394142151, 0.26952069997787476, 0.3606722950935364, 0.33100467920303345, 0.6333733201026917, 0.7559706568717957, 0.7310944199562073, 0.04493755102157593, 0.5685619115829468, 0.1581093668937683, 0.23704755306243896, 0.5157313942909241, 0.21669596433639526, 0.43922752141952515, 0.6637806296348572, 0.2714915871620178, 0.0940743088722229, 0.22602182626724243, 0.7271146178245544, 0.7287468910217285, 0.6475335359573364, 0.05076467990875244, 0.3023543357849121, 0.6964979767799377, 0.3476976752281189, 0.04921191930770874, 0.2834394574165344, 0.2063714861869812, 0.3263780474662781, 0.8747990131378174, 0.12339520454406738, 0.45670080184936523, 0.5096578001976013, 0.1301756501197815, 0.7262669801712036, 0.17323261499404907, 0.6024535894393921, 0.482186496257782, 0.22307050228118896, 0.03145664930343628, 0.07270574569702148, 0.01352757215499878, 0.31032848358154297, 0.7345136404037476, 0.3408757448196411, 0.13452494144439697, 0.8322340846061707, 0.6139223575592041, 0.25682520866394043, 0.6259174346923828, 0.7921925783157349, 0.7299354076385498, 0.4460546374320984, 0.6561207175254822, 0.18011319637298584, 0.7056617736816406, 0.02665168046951294, 0.712006151676178, 0.2706305980682373, 0.21497851610183716, 0.498828649520874, 0.06047755479812622, 0.4137035608291626, 0.5988679528236389, 0.8529990911483765, 0.17158907651901245, 0.7334389686584473, 0.13375061750411987, 0.316600501537323, 0.5643799901008606, 0.2527083158493042, 0.08678823709487915, 0.10014009475708008, 0.4489593505859375, 0.9298760890960693, 0.38235408067703247, 0.10882031917572021, 0.5932636857032776, 0.19470620155334473, 0.4273112416267395, 0.5408018231391907, 0.3049173951148987, 0.21603834629058838, 0.158649742603302, 0.2545355558395386, 0.8881954550743103, 0.6817103624343872, 0.0884888768196106, 0.19464325904846191, 0.07700175046920776, 0.2612420320510864, 0.0013731122016906738, 0.35963255167007446, 0.09287828207015991, 0.011683464050292969, 0.15546870231628418, 0.7329037189483643, 0.5000244975090027, 0.06654363870620728, 0.6073477864265442, 0.05881398916244507, 0.4835878014564514, 0.341727077960968, 0.2185078263282776, 0.21745115518569946, 0.293562650680542, 0.40685999393463135, 0.4472668766975403, 0.3429776430130005, 0.24588805437088013, 0.6096287965774536, 0.34218961000442505, 0.37447983026504517, 0.3354390859603882, 0.3983280062675476, 0.5439390540122986, 0.15626496076583862, 0.34048110246658325, 0.12660449743270874, 0.722770094871521, 0.32420361042022705, 0.15098464488983154, 0.2973756790161133, 0.47668808698654175, 0.13075262308120728, 0.4980369806289673, 0.3530231714248657, 0.29020488262176514, 0.3535223603248596, 0.2040737271308899, 0.6806874871253967, 0.16405797004699707, 0.08127975463867188, 0.22130519151687622, 0.20521056652069092, 0.46420133113861084, 0.279413104057312, 0.2581068277359009, 0.18573427200317383, 0.6533098816871643, 0.06150931119918823, 0.30001717805862427, 0.4238293170928955, 0.7199961543083191, 0.1661440134048462, 0.49640148878097534, 0.00423276424407959, 0.3339191675186157, 0.43350881338119507, 0.797866940498352, 0.5429943203926086, 0.18506282567977905, 0.20823538303375244, 0.22777718305587769, 0.1335318684577942, 0.7745791673660278, 0.09688657522201538, 0.11408054828643799, 0.5459064841270447, 0.6593518257141113, 0.33247923851013184, 0.024837374687194824, 0.26999837160110474, 0.5404632687568665, 0.5580571293830872, 0.6096675992012024, 0.7126033306121826, 0.3378216624259949, 0.40597212314605713, 0.9048783183097839, 0.38224172592163086, 0.6528801918029785, 0.4558643102645874, 0.6744803786277771, 0.40448975563049316, 0.0020956993103027344, 0.057345688343048096, 0.6575667262077332, 0.15229523181915283, 0.09029644727706909, 0.009678542613983154, 0.25935471057891846, 0.4739212393760681, 0.23521673679351807, 0.18600785732269287, 0.8388687372207642, 0.21878784894943237, 0.2887386679649353, 0.4411165118217468, 0.5946701169013977, 0.2993161082267761, 0.6625912189483643, 0.43132179975509644, 0.441917359828949, 0.08421128988265991, 0.06003636121749878, 0.8429652452468872, 0.07907694578170776, 0.5225718021392822, 0.8959828615188599, 0.29455095529556274, 0.8318410515785217, 0.34169965982437134, 0.43636035919189453, 0.12410169839859009, 0.2745823264122009, 0.44601011276245117, 0.8961799144744873, 0.29537343978881836, 0.5151318907737732, 0.03667628765106201, 0.34154951572418213, 0.04440683126449585, 0.13576942682266235, 0.3536979556083679, 0.43414217233657837, 0.2293703556060791, 0.9678479433059692, 0.41515493392944336, 0.09250766038894653, 0.181723952293396, 0.49557459354400635, 0.680111289024353, 0.37075209617614746, 0.2529398798942566, 0.25575554370880127, 0.6939331293106079, 0.4150158762931824, 0.26538336277008057, 0.8166524171829224, 0.4355369806289673, 0.05671358108520508, 0.05975896120071411, 0.19121068716049194, 0.25829362869262695, 0.3407488465309143, 0.7348363995552063, 0.7664898633956909, 0.4401945471763611, 0.6959851384162903, 0.41953688859939575, 0.35078418254852295, 0.15995794534683228, 0.20450133085250854, 0.855686366558075, 0.4046010971069336, 0.08863389492034912, 0.46189743280410767, 0.03924107551574707, 0.07220488786697388, 0.19325238466262817, 0.22040438652038574, 0.1819993257522583, 0.2376563549041748, 0.5342486500740051, 0.4686034321784973, 0.6324417591094971, 0.2031370997428894, 0.146817147731781, 0.9351653456687927, 0.6725901961326599, 0.5432873368263245, 0.7577497363090515, 0.6622986197471619, 0.14523553848266602, 0.23100131750106812, 0.4226532578468323, 0.20394080877304077, 0.44739532470703125, 0.017567336559295654, 0.5759331583976746, 0.47733837366104126, 0.6587360501289368, 0.027064144611358643, 0.6274417638778687, 0.20899367332458496, 0.07056766748428345, 0.40407490730285645, 0.1524680256843567, 0.3079604506492615, 0.5218818187713623, 0.30699217319488525, 0.25798845291137695, 0.11991071701049805, 0.44561827182769775, 0.7043562531471252, 0.16695499420166016, 0.030781805515289307, 0.2153119444847107, 0.5000156164169312, 0.0593569278717041, 0.08966326713562012, 0.7760561108589172, 0.17252379655838013, 0.023537933826446533, 0.11471569538116455, 0.1544361114501953, 0.40281766653060913, 0.2551169991493225, 0.1254439353942871, 0.2305482029914856, 0.291351318359375, 0.02083200216293335, 0.06067579984664917, 0.22196930646896362, 0.23201888799667358, 0.11065882444381714, 0.2924904227256775, 0.044836580753326416, 0.0428006649017334, 0.4329580068588257, 0.9236005544662476, 0.6208754181861877, 0.3653794527053833, 0.9441730380058289, 0.6261195540428162, 0.5075944662094116, 0.5612905025482178, 0.42538124322891235, 0.20098739862442017, 0.0772475004196167, 0.4354175329208374, 0.055270254611968994, 0.09881013631820679, 0.8835799098014832, 0.6393222212791443, 0.39635974168777466, 0.34312087297439575, 0.17831271886825562, 0.17225778102874756, 0.50742107629776, 0.027103006839752197, 0.0565679669380188, 0.7143110632896423, 0.7973834276199341, 0.04052531719207764, 0.1721816062927246, 0.052015483379364014, 0.4626883864402771, 0.10926085710525513, 0.4780043959617615, 0.1738041639328003, 0.2246856689453125, 0.700458288192749, 0.9434679746627808, 0.267597496509552, 0.1160362958908081, 0.18740832805633545, 0.25055187940597534, 0.45801496505737305, 0.026939570903778076, 0.6129099726676941, 0.6298443078994751, 0.01424175500869751, 0.33690035343170166, 0.033739328384399414, 0.5407729744911194, 0.4215962886810303, 0.4011749029159546, 0.3613350987434387, 0.4225868582725525, 0.7522432804107666, 0.23350781202316284, 0.10570240020751953, 0.8289042711257935, 0.11397844552993774, 0.07389932870864868, 0.353127121925354, 0.8136468529701233, 0.035071372985839844, 0.006412088871002197, 0.048695504665374756, 0.1277340054512024, 0.3807957172393799, 0.16866350173950195, 0.5543339252471924, 0.7485201358795166, 0.5519359707832336, 0.46553003787994385, 0.24026626348495483, 0.025147855281829834, 0.7652859091758728, 0.0829574465751648, 0.2687162160873413, 0.10850852727890015, 0.25984448194503784, 0.9353047013282776, 0.5764464139938354, 0.2877064347267151, 0.3035603165626526, 0.44536763429641724, 0.6197121143341064, 0.22737020254135132, 0.03250563144683838, 0.39331507682800293, 0.6970370411872864, 0.0029187798500061035, 0.13675320148468018, 0.42335188388824463, 0.8209884762763977, 0.3680015802383423, 0.23616713285446167, 0.4202139377593994, 0.6884370446205139, 0.5130432844161987, 0.084636390209198, 0.5356369614601135, 0.013265073299407959, 0.7299386262893677, 0.3644745945930481, 0.7562670111656189, 0.03992640972137451, 0.4020480513572693, 0.27427804470062256, 0.4200671911239624, 0.40383994579315186, 0.4139302372932434, 0.8769804239273071, 0.19632965326309204, 0.13987016677856445, 0.1127045750617981, 0.6715863347053528, 0.11318176984786987, 0.8432795405387878, 0.36891597509384155, 0.0965871810913086, 0.17840611934661865, 0.6670020818710327, 0.21524029970169067, 0.47129684686660767, 0.7075079083442688, 0.23374229669570923, 0.5554410219192505, 0.3471412658691406, 0.46328872442245483, 0.24487817287445068, 0.14895427227020264, 0.11155003309249878, 0.11461198329925537, 0.5895712971687317, 0.4301150441169739, 0.46716171503067017, 0.2868689298629761, 0.2491428256034851, 0.8117551207542419, 0.8941882252693176, 0.23941028118133545, 0.9434542655944824, 0.346743106842041, 0.46369343996047974, 0.1346995234489441, 0.8435433506965637, 0.3144199848175049, 0.06337064504623413, 0.6886688470840454, 0.07606363296508789, 0.2368175983428955, 0.22929805517196655, 0.7031611204147339, 0.8228035569190979, 0.16951996088027954, 0.1164514422416687, 0.14087235927581787, 0.5341012477874756, 0.2845892906188965, 0.6591649651527405, 0.023028790950775146, 0.05956733226776123, 0.008753180503845215, 0.18232202529907227, 0.6633684635162354, 0.10528707504272461, 0.6393422484397888, 0.49574553966522217, 0.1464112401008606, 0.556575357913971, 0.015881061553955078, 0.26599371433258057, 0.4571535587310791, 0.24513286352157593, 0.22288638353347778, 0.44694095849990845, 0.037099599838256836, 0.015287637710571289, 0.1536552906036377, 0.7550485134124756, 0.22398990392684937, 0.08458113670349121, 0.4419574737548828, 0.1302412748336792, 0.4262714982032776, 0.27959996461868286, 0.4581623077392578, 0.37805432081222534, 0.798294723033905, 0.14565038681030273, 0.8304587006568909, 0.034793078899383545, 0.28424519300460815, 0.06048381328582764, 0.12850642204284668, 0.14443820714950562, 0.09772205352783203, 0.26103389263153076, 0.18207091093063354, 0.5772796273231506, 0.6147114038467407, 0.6037632822990417, 0.11894601583480835, 0.44130468368530273, 0.485498309135437, 0.38454777002334595, 0.3569876551628113, 0.12961113452911377, 0.3080611228942871, 0.061462581157684326, 0.12064659595489502, 0.4731312394142151, 0.22920995950698853, 0.12593895196914673, 0.16168951988220215, 0.07482767105102539, 0.09230595827102661, 0.27721405029296875, 0.2896823287010193, 0.10120481252670288, 0.014774680137634277, 0.39852195978164673, 0.37255001068115234, 0.15945935249328613, 0.0649062991142273, 0.39541560411453247, 0.042154133319854736, 0.4201030731201172, 0.32592880725860596, 0.00590890645980835, 0.2730681300163269, 0.008704423904418945, 0.7858434319496155, 0.4284899830818176, 0.8112316727638245, 0.6792433857917786, 0.07646828889846802, 0.6726838946342468, 0.17669057846069336, 0.00683063268661499, 0.21088451147079468, 0.13881319761276245, 0.2609608769416809, 0.13697963953018188, 0.27803003787994385, 0.6555537581443787, 0.01766026020050049, 0.11674773693084717, 0.3197293281555176, 0.24918854236602783, 0.45096856355667114, 0.2559856176376343, 0.30519020557403564, 0.7351788878440857, 0.5391899943351746, 0.7547821402549744, 0.9443817138671875, 0.6440865397453308, 0.7193707823753357, 0.6355103850364685, 0.12723088264465332, 0.17823737859725952, 0.4265894293785095, 0.1408599615097046, 0.4719449281692505, 0.6550310850143433, 0.30343538522720337, 0.07175153493881226, 0.6757365465164185, 0.032911837100982666, 0.3214470148086548, 0.28503257036209106, 0.6768784523010254, 0.09195065498352051, 0.4555153250694275, 0.3003467917442322, 0.23682492971420288, 0.13976657390594482, 0.6203670501708984, 0.7548109292984009, 0.10798847675323486, 0.1891324520111084, 0.34985625743865967, 0.44544166326522827, 0.05701392889022827, 0.6992323994636536, 0.5180899500846863, 0.36645424365997314, 0.029401838779449463, 0.41272085905075073, 0.18513691425323486, 0.6376991868019104, 0.3291006088256836, 0.5357050895690918, 0.2083485722541809, 0.27798306941986084, 0.08038729429244995, 0.5736456513404846, 0.17717885971069336, 0.45735520124435425, 0.2706940770149231, 0.22621619701385498, 0.5467511415481567, 0.6449710130691528, 0.6202636361122131, 0.293599009513855, 0.07451409101486206, 0.1113402247428894, 0.23029404878616333, 0.002274036407470703, 0.3945552110671997, 0.5269360542297363, 0.16098344326019287, 0.2803521752357483, 0.04997318983078003, 0.277471661567688, 0.2793487310409546, 0.24569851160049438, 0.8900160789489746, 0.5379400849342346, 0.2773279547691345, 0.08271598815917969, 0.6981303691864014, 0.11508029699325562, 0.4960394501686096, 0.055068790912628174, 0.02937072515487671, 0.6983087062835693, 0.41758406162261963, 0.6155681610107422, 0.7215433120727539, 0.27430015802383423, 0.7153080105781555, 0.4518098831176758, 0.060864150524139404, 0.08884400129318237, 0.5118139982223511, 0.26770031452178955, 0.32442396879196167, 0.10309463739395142, 0.21953588724136353, 0.3037901520729065, 0.5670300722122192, 0.48249179124832153, 0.11225885152816772, 0.5109631419181824, 0.20254850387573242, 0.04750138521194458, 0.11855888366699219, 0.5337293744087219, 0.19411319494247437, 0.80995774269104, 0.26391494274139404, 0.21398013830184937, 0.23344409465789795, 0.0014661550521850586, 0.20848804712295532, 0.4603981375694275, 0.1152004599571228, 0.28029197454452515, 0.9247421622276306, 0.04295438528060913, 0.27744799852371216, 0.5359774231910706, 0.7553380131721497, 0.5583397150039673, 0.2667330503463745, 0.05616354942321777, 0.7541296482086182, 0.11154788732528687, 0.24396049976348877, 0.013984084129333496, 0.4817109704017639, 0.7272177934646606, 0.05460715293884277, 0.04048842191696167, 0.09815222024917603, 0.06657266616821289, 0.21932339668273926, 0.09128040075302124, 0.22143924236297607, 0.5000879168510437, 0.6787301898002625, 0.15251541137695312, 0.1793140172958374, 0.8249253630638123, 0.224354088306427, 0.6520256996154785, 0.06565678119659424, 0.05408167839050293, 0.3416820168495178, 0.06057131290435791, 0.13741010427474976, 0.23427724838256836, 0.5173999071121216, 0.4467185139656067, 0.5932231545448303, 0.3368275761604309, 0.7380635738372803, 0.19600814580917358, 0.7341638207435608, 0.30894792079925537, 0.33130741119384766, 0.6555159091949463, 0.22409409284591675, 0.09959292411804199, 0.5071963667869568, 0.13093000650405884, 0.1843663454055786, 0.21819478273391724, 0.6606723070144653, 0.37215346097946167, 0.618170440196991, 0.7314348816871643, 0.38691896200180054, 0.11717164516448975, 0.06518149375915527, 0.3659847378730774, 0.08560645580291748, 0.020594418048858643, 0.1784648895263672, 0.31318527460098267, 0.1527496576309204, 0.9166620373725891, 0.00773245096206665, 0.4602903723716736, 0.3255618214607239, 0.06178992986679077, 0.10637754201889038, 0.6690776944160461, 0.4155269265174866, 0.6602404713630676, 0.09032005071640015, 0.2371584177017212, 0.17424547672271729, 0.06152629852294922, 0.6273806691169739, 0.7393089532852173, 0.36953651905059814, 0.19967669248580933, 0.08599227666854858, 0.005904555320739746, 0.5408865809440613, 0.21258366107940674, 0.7613071799278259, 0.2560051679611206, 0.11774390935897827, 0.3075036406517029, 0.3626629114151001, 0.6486400365829468, 0.7699277400970459, 0.16709589958190918, 0.3527340292930603, 0.00019371509552001953, 0.4432656764984131, 0.062197327613830566, 0.39026182889938354, 0.10137683153152466, 0.06817853450775146, 0.16604357957839966, 0.19909393787384033, 0.09727752208709717, 0.6528032422065735, 0.8529098033905029, 0.3831104040145874, 0.520029604434967, 0.61130291223526, 0.011818647384643555, 0.1214834451675415, 0.4741687774658203, 0.017913103103637695, 0.22882771492004395, 0.05828738212585449, 0.5108380317687988, 0.039988577365875244, 0.377219557762146, 0.7851234674453735, 0.10014122724533081, 0.36678647994995117, 0.5529736280441284, 0.030139923095703125, 0.40483254194259644, 0.5209900140762329, 0.04863762855529785, 0.174921452999115, 0.716849148273468, 0.26942479610443115, 0.5226032733917236, 0.09649401903152466, 0.13865524530410767, 0.03434985876083374, 0.649509608745575, 0.08611816167831421, 0.5713666081428528, 0.5762595534324646, 0.5142778158187866, 0.4400520324707031, 0.16904836893081665, 0.18040776252746582, 0.2104833722114563, 0.4724971652030945, 0.21146780252456665, 0.7680298089981079, 0.2818799614906311, 0.8208790421485901, 0.266421377658844, 0.2608426809310913, 0.3107687830924988, 0.5067662596702576, 0.11938983201980591, 0.916819155216217, 0.009236395359039307, 0.4011076092720032, 0.32862162590026855, 0.08841818571090698, 0.3297848105430603, 0.707344114780426, 0.9656456112861633, 0.10966634750366211, 0.24676227569580078, 0.5999464392662048, 0.07744830846786499, 0.1822737455368042, 0.40667539834976196, 0.45211315155029297, 0.19454681873321533, 0.3046104311943054, 0.39424294233322144, 0.5287058353424072, 0.26882433891296387, 0.38120079040527344, 0.6451328992843628, 0.3555430769920349, 0.17183023691177368, 0.009350001811981201, 0.44592803716659546, 0.08353263139724731, 0.6227178573608398, 0.6676760315895081, 0.39504164457321167, 0.0027254223823547363, 0.22150349617004395, 0.3748167157173157, 0.7518768310546875, 0.23559772968292236, 0.01691603660583496, 0.014590799808502197, 0.5264285802841187, 0.008939146995544434, 0.040342628955841064, 0.6676309108734131, 0.590920090675354, 0.026159167289733887, 0.1263369917869568, 0.10302162170410156, 0.28449708223342896, 0.4479384422302246, 0.02336031198501587, 0.11399519443511963, 0.4320806860923767, 0.03696972131729126, 0.22866421937942505, 0.6149128079414368, 0.5786099433898926, 0.6540908813476562, 0.5641013383865356, 0.17393958568572998, 0.8175355195999146, 0.14869612455368042, 0.5236397385597229, 0.060566484928131104, 0.11069899797439575, 0.2606748342514038, 0.21655434370040894, 0.27021580934524536, 0.4997365474700928, 0.42076772451400757, 0.9161955714225769, 0.5037040710449219, 0.05144864320755005, 0.19930708408355713, 0.17956805229187012, 0.08409738540649414, 0.00303041934967041, 0.13897031545639038, 0.3612198233604431, 0.6961501836776733, 0.5612125396728516, 0.21386784315109253, 0.022432684898376465, 0.07615673542022705, 0.21068179607391357, 0.6619673371315002, 0.6643840074539185, 0.022739768028259277, 0.046753764152526855, 0.346041202545166, 0.6481614112854004, 0.6600987911224365, 0.5171672105789185, 0.25393468141555786, 0.077903151512146, 0.03391444683074951, 0.28514808416366577, 0.0585026741027832, 0.36854875087738037, 0.27499639987945557, 0.5520086288452148, 0.7183642983436584, 0.6866320371627808, 0.7418116331100464, 0.2736161947250366, 0.025742650032043457, 0.7834685444831848, 0.5639862418174744, 0.2513652443885803, 0.4303852915763855, 0.2002023458480835, 0.46455663442611694, 0.013484537601470947, 0.25002050399780273, 0.19082540273666382, 0.32729727029800415, 0.18527686595916748, 0.2046012282371521, 0.043614208698272705, 0.5502289533615112, 0.3044940233230591, 0.22660189867019653, 0.27625274658203125, 0.32915616035461426, 0.1451817750930786, 0.14971119165420532, 0.47507351636886597, 0.5312045812606812, 0.5104866027832031, 0.28352081775665283, 0.23745989799499512, 0.10323601961135864, 0.5519886016845703, 0.7426313161849976, 0.7111661434173584, 0.8001857995986938, 0.48047536611557007, 0.14660614728927612, 0.5637126564979553, 0.01752680540084839, 0.1514376401901245, 0.5910099148750305, 0.5199968814849854, 0.25781452655792236, 0.1468977928161621, 0.1657680869102478, 0.5227564573287964, 0.1430400013923645, 0.12343591451644897, 0.2631549835205078, 0.6277520656585693, 0.48242437839508057, 0.02944815158843994, 0.7178046107292175, 0.3991404175758362, 0.14645344018936157, 0.08770203590393066, 0.05392146110534668, 0.48973315954208374, 0.6425315737724304, 0.1844530701637268, 0.3488815426826477, 0.888125479221344, 0.33692872524261475, 0.3096926212310791, 0.3804277181625366, 0.20873457193374634, 0.3453245759010315, 0.10410618782043457, 0.08443593978881836, 0.43022972345352173, 0.3970645070075989, 0.5046598315238953, 0.040370821952819824, 0.5641207098960876, 0.024671196937561035, 0.39828068017959595, 0.48347729444503784, 0.11871165037155151, 0.2830349802970886, 0.3812652826309204, 0.5309303402900696, 0.40030425786972046, 0.09698641300201416, 0.2492361068725586, 0.4143487215042114, 0.3818432688713074, 0.10621672868728638, 0.22036653757095337, 0.2674700617790222, 0.011307716369628906, 0.1012502908706665, 0.17411035299301147, 0.4843575358390808, 0.6104511022567749, 0.397594690322876, 0.09782469272613525, 0.3997827172279358, 0.011240959167480469, 0.15396052598953247, 0.7948082089424133, 0.12981629371643066, 0.14811962842941284, 0.24168270826339722, 0.5692712068557739, 0.36917513608932495, 0.1601596474647522, 0.20313626527786255, 0.528148353099823, 0.1501672863960266, 0.16105890274047852, 0.6675370931625366, 0.18307781219482422, 0.11158877611160278, 0.6928540468215942, 0.18104606866836548, 0.3825128674507141, 0.31406962871551514, 0.5641029477119446, 0.019264280796051025, 0.5682317614555359, 0.28888803720474243, 0.25442272424697876, 0.3842684030532837, 0.3416528105735779, 0.11388999223709106, 0.35485219955444336, 0.18068379163742065, 0.34986698627471924, 0.9029943346977234, 0.43407535552978516, 0.09707695245742798, 0.37639307975769043, 0.43396782875061035, 0.709357500076294, 0.7267990708351135, 0.7599713802337646, 0.07484745979309082, 0.19418781995773315, 0.07684135437011719, 0.4585498571395874, 0.8402188420295715, 0.33780866861343384, 0.5811384916305542, 0.145424485206604, 0.1351257562637329, 0.1414777636528015, 0.5785828232765198, 0.25810515880584717, 0.7499133348464966, 0.17334115505218506, 0.04760289192199707, 0.5253920555114746, 0.08166462182998657, 0.051527976989746094, 0.7209178805351257, 0.05281609296798706, 0.6893305778503418, 0.1077582836151123, 0.046530306339263916, 0.10021227598190308, 0.6194244623184204, 0.48208755254745483, 0.4991258382797241, 0.012492537498474121, 0.8220227956771851, 0.8192044496536255, 0.26659882068634033, 0.4443790912628174, 0.1809481978416443, 0.2471279501914978, 0.1600974202156067, 0.10121369361877441, 0.3175767660140991, 0.6225335597991943, 0.019028127193450928, 0.2222495675086975, 0.5759332180023193, 0.5656983256340027, 0.1940794587135315, 0.18635064363479614, 0.24367690086364746, 0.02237570285797119, 0.8639070987701416, 0.059455692768096924, 0.4005814790725708, 0.09531998634338379, 0.0027028322219848633, 0.49634993076324463, 0.261829137802124, 0.4171823263168335, 0.6175188422203064, 0.16041773557662964, 0.6564354300498962, 0.00717318058013916, 0.32899177074432373, 0.510107159614563, 0.2478017807006836, 0.2800048589706421, 0.44094377756118774, 0.4445841908454895, 0.051956355571746826, 0.1772385835647583, 0.6915939450263977, 0.32845354080200195, 0.6320414543151855, 0.01919424533843994, 0.3357624411582947, 0.0995219349861145, 0.6676794290542603, 0.3243684768676758, 0.3478105068206787, 0.2775712013244629, 0.023166120052337646, 0.6426859498023987, 0.12888497114181519, 0.07441115379333496, 0.030060291290283203, 0.07891666889190674, 0.5174540281295776, 0.3546844720840454, 0.04226499795913696, 0.6262508630752563, 0.11805480718612671, 0.07251441478729248, 0.31845033168792725, 0.2646191120147705, 0.3006420135498047, 0.024273991584777832, 0.16597110033035278, 0.3471373915672302, 0.08518314361572266, 0.16027969121932983, 0.09301102161407471, 0.484757661819458, 0.6745309233665466, 0.2678349018096924, 0.5375725030899048, 0.218006432056427, 0.7233692407608032, 0.908390462398529, 0.39371180534362793, 0.3589041233062744, 0.8606922626495361, 0.3573141098022461, 0.5436088442802429, 0.10281449556350708, 0.3296493887901306, 0.5099719762802124, 0.5662662982940674, 0.02267080545425415, 0.2832140326499939, 0.03263944387435913, 0.313157320022583, 0.6455124020576477, 0.10554295778274536, 0.040498971939086914, 0.2912270426750183, 0.271915078163147, 0.19078022241592407, 0.4159010052680969, 0.21671736240386963, 0.6307407021522522, 0.9388667345046997, 0.3125419616699219, 0.051646292209625244, 0.4844154119491577, 0.3078235387802124, 0.5949873924255371, 0.17459768056869507, 0.36231768131256104, 0.021557927131652832, 0.3151598572731018, 0.5696583986282349, 0.05961698293685913, 0.27310293912887573, 0.038676440715789795, 0.4946853518486023, 0.3588750958442688, 0.17459291219711304, 0.2617231011390686, 0.1601133942604065, 0.7429474592208862, 0.22123587131500244, 0.03953087329864502, 0.38776570558547974, 0.295681357383728, 0.24361276626586914, 0.45685631036758423, 0.4770333766937256, 0.32112717628479004, 0.16301250457763672, 0.8462668657302856, 0.5800697207450867, 0.587229311466217, 0.8672915697097778, 0.38068002462387085, 0.5015202760696411, 0.027528822422027588, 0.09412914514541626, 0.27158302068710327, 0.19418257474899292, 0.04860198497772217, 0.9685653448104858, 0.9385695457458496, 0.033767879009246826, 0.40599846839904785, 0.42956632375717163, 0.10736948251724243, 0.5995029211044312, 0.17376667261123657, 0.1684250831604004, 0.1817222237586975, 0.07075905799865723, 0.05119359493255615, 0.43107032775878906, 0.16556763648986816, 0.015132009983062744, 0.2125799059867859, 0.05277317762374878, 0.32675641775131226, 0.7146226167678833, 0.4426327347755432, 0.5870943069458008, 0.14375030994415283, 0.6672743558883667, 0.12333095073699951, 0.09536439180374146, 0.17734289169311523, 0.6341814398765564, 0.4518483281135559, 0.09989511966705322, 0.31087619066238403, 0.8494519591331482, 0.01329892873764038, 0.1152833104133606, 0.26426881551742554, 0.16514617204666138, 0.22561049461364746, 0.32038551568984985, 0.5032699704170227, 0.05045801401138306, 0.37717723846435547, 0.15704858303070068, 0.8880988359451294, 0.6677641868591309, 0.12651824951171875, 0.6096523404121399, 0.15615487098693848, 0.4136403203010559, 0.382831871509552, 0.017678141593933105, 0.10886973142623901, 0.24560213088989258, 0.05668032169342041, 0.22046303749084473, 0.31359875202178955, 0.1095116138458252, 0.23825669288635254, 0.25091785192489624, 0.927617609500885, 0.08209222555160522, 0.2279585599899292, 0.25101256370544434, 0.8404723405838013, 0.021101057529449463, 0.7004234790802002, 0.42805153131484985, 0.7428317666053772, 0.20785021781921387, 0.37213122844696045, 0.10200905799865723, 0.37810999155044556, 0.0414312481880188, 0.36331862211227417, 0.3406025767326355, 0.33061444759368896, 0.55427086353302, 0.013246297836303711, 0.35081249475479126, 0.9025378227233887, 0.3362373113632202, 0.3826867341995239, 0.2771432399749756, 0.06749337911605835, 0.0002510547637939453, 0.21204906702041626, 0.3022940754890442, 0.27402448654174805, 0.2446308732032776, 0.6838582158088684, 0.6155703067779541, 0.09222632646560669, 0.6206785440444946, 0.037275612354278564, 0.7814391255378723, 0.20259100198745728, 0.121673583984375, 0.04881197214126587, 0.042054831981658936, 0.36198192834854126, 0.5523326396942139, 0.7205908894538879, 0.1015353798866272, 0.03920072317123413, 0.007380664348602295, 0.7060993909835815, 0.5067755579948425, 0.04396742582321167, 0.3408789038658142, 0.3079338073730469, 0.12542980909347534, 0.22050070762634277, 0.19196951389312744, 0.27814263105392456, 0.5934844017028809, 0.21799343824386597, 0.5471243262290955, 0.03073108196258545, 0.2224695086479187, 0.9624106287956238, 0.666927695274353, 0.24381136894226074, 0.3438689112663269, 0.9968903660774231, 0.022138476371765137, 0.031158030033111572, 0.6984657049179077, 0.5122307538986206, 0.22690504789352417, 0.076321542263031, 0.6516762971878052, 0.4420083165168762, 0.2967289686203003, 0.26163148880004883, 0.17681831121444702, 0.11878180503845215, 0.828066349029541, 0.38678526878356934, 0.5056650042533875, 0.26117372512817383, 0.006418049335479736, 0.47950005531311035, 0.23520493507385254, 0.5637993216514587, 0.009811878204345703, 0.10653698444366455, 0.8552592396736145, 0.5291359424591064, 0.4785993695259094, 0.13881832361221313, 0.09848016500473022, 0.034758925437927246, 0.2595195770263672, 0.8089163899421692, 0.30007404088974, 0.159349262714386, 0.21566718816757202, 0.7148692607879639, 0.7636582851409912, 0.2960658073425293, 0.35125136375427246, 0.5293118953704834, 0.8386703133583069, 0.37084490060806274, 0.07092833518981934, 0.6041048765182495, 0.5495851635932922, 0.03131103515625, 0.31418943405151367, 0.7989077568054199, 0.7991427779197693, 0.5965601801872253, 0.10505175590515137, 0.8620222806930542, 0.4577825665473938, 0.4635418653488159, 0.42025649547576904, 0.09972482919692993, 0.15042412281036377, 0.6110634803771973, 0.0394669771194458, 0.2902469038963318, 0.04159557819366455, 0.7344803810119629, 0.5064525008201599, 0.525987982749939, 0.09391546249389648, 0.08924424648284912, 0.3573358654975891, 0.017915546894073486, 0.22440606355667114, 0.6756619215011597, 0.4466248154640198, 0.32105553150177, 0.12694543600082397, 0.6030839681625366, 0.5527179837226868, 0.38791465759277344, 0.14371329545974731, 0.2543335556983948, 0.324313223361969, 0.17613095045089722, 0.23499763011932373, 0.33043432235717773, 0.5637049078941345, 0.7753251791000366, 0.31118911504745483, 0.4059550166130066, 0.5575730204582214, 0.08265191316604614, 0.6932996511459351, 0.2260134220123291, 0.14922475814819336, 0.38889163732528687, 0.3758939504623413, 0.020794689655303955, 0.5395714640617371, 0.2025548815727234, 0.1589471697807312, 0.17890489101409912, 0.5871977806091309, 0.7567799687385559, 0.7848391532897949, 0.006271660327911377, 0.9450739622116089, 0.19231194257736206, 0.2392197847366333, 0.23444432020187378, 0.03193479776382446, 0.43108630180358887, 0.6721157431602478, 0.6927654147148132, 0.34428560733795166, 0.7370008230209351, 0.22647273540496826, 0.1089661717414856, 0.25037163496017456, 0.12045443058013916, 0.13696521520614624, 0.31635546684265137, 0.09060382843017578, 0.3545724153518677, 0.43799328804016113, 0.1292058825492859, 0.7239950895309448, 0.20637226104736328, 0.2730851173400879, 0.08808737993240356, 0.5414974689483643, 0.27812159061431885, 0.6250966787338257, 0.32169878482818604, 0.12193804979324341, 0.36269569396972656, 0.1344643235206604, 0.627505362033844, 0.5031490325927734, 0.4236002564430237, 0.2853237986564636, 0.14149510860443115, 0.7462141513824463, 0.31068211793899536, 0.3200174570083618, 0.20207840204238892, 0.24149519205093384, 0.7382242679595947, 0.28122544288635254, 0.06504780054092407, 0.0636528730392456, 0.4507526755332947, 0.44612234830856323, 0.2207556962966919, 0.0680466890335083, 0.4620389938354492, 0.8690316677093506, 0.5880040526390076, 0.6307838559150696, 0.812798798084259, 0.6275226473808289, 0.24795114994049072, 0.055860280990600586, 0.17629307508468628, 0.16446518898010254, 0.3389369249343872, 0.657195508480072, 0.7092417478561401, 0.05798196792602539, 0.29237186908721924, 0.6929123401641846, 0.17015892267227173, 0.413649320602417, 0.0541415810585022, 0.3123928904533386, 0.47128117084503174, 0.780124306678772, 0.5539208054542542, 0.6355544328689575, 0.3946669101715088, 0.6924405694007874, 0.12485861778259277, 0.5600256323814392, 0.30558347702026367, 0.357219934463501, 0.5220887064933777, 0.09763342142105103, 0.3572099208831787, 0.024442672729492188, 0.004319965839385986, 0.22107172012329102, 0.48931604623794556, 0.17818200588226318, 0.026653945446014404, 0.2768614888191223, 0.43462836742401123, 0.1254042387008667, 0.1587473750114441, 0.041149914264678955, 0.2546015977859497, 0.6531192660331726, 0.2559847831726074, 0.012769103050231934, 0.01004093885421753, 0.26856350898742676, 0.2654115557670593, 0.2695189118385315, 0.2181289792060852, 0.379744291305542, 0.2183535099029541, 0.581433117389679, 0.10743856430053711, 0.9077345132827759, 0.16045188903808594, 0.5495790243148804, 0.7936310768127441, 0.3792775273323059, 0.31943827867507935, 0.13031750917434692, 0.2547336220741272, 0.4630820155143738, 0.1275818943977356, 0.4001381993293762, 0.5732308030128479, 0.1739027500152588, 0.2212386131286621, 0.6668605208396912, 0.39561396837234497, 0.9500197172164917, 0.25173234939575195, 0.14736884832382202, 0.4786583185195923, 0.513680636882782, 0.6502298712730408, 0.29105478525161743, 0.43568259477615356, 0.08218890428543091, 0.20668822526931763, 0.5629661083221436, 0.01134723424911499, 0.4405297040939331, 0.8472110629081726, 0.36531299352645874, 0.2710835933685303, 0.0009249448776245117, 0.34258097410202026, 0.5038694143295288, 0.860190212726593, 0.7577041983604431, 0.17851734161376953, 0.8957440853118896, 0.3465058207511902, 0.17977315187454224, 0.11701440811157227, 0.028444230556488037, 0.05371159315109253, 0.31711357831954956, 0.1570236086845398, 0.3472754955291748, 0.09182381629943848, 0.42063283920288086, 0.3142378330230713, 0.16466909646987915, 0.06002777814865112, 0.7133134603500366, 0.6954781413078308, 0.14270436763763428, 0.5615492463111877, 0.446475088596344, 0.1892886757850647, 0.6013185977935791, 0.2020847201347351, 0.21709942817687988, 0.3036620020866394, 0.07734489440917969, 0.5993781089782715, 0.06008094549179077, 0.06333255767822266, 0.22988194227218628, 0.07831460237503052, 0.7507689595222473, 0.2482275366783142, 0.1539832353591919, 0.38813090324401855, 0.538348376750946, 0.5128194689750671, 0.3687162399291992, 0.19667792320251465, 0.2944266200065613, 0.11998188495635986, 0.1040952205657959, 0.121762216091156, 0.011281490325927734, 0.5632996559143066, 0.18097203969955444, 0.19610756635665894, 0.7194279432296753, 0.5217838883399963, 0.792304515838623, 0.2811393737792969, 0.3394940495491028, 0.23979783058166504, 0.020127594470977783, 0.5743506550788879, 0.3963102102279663, 0.22013026475906372, 0.2977164387702942, 0.40190547704696655, 0.45889806747436523, 0.5434439778327942, 0.06380641460418701, 0.8750255107879639, 0.5438334941864014, 0.2749805450439453, 0.02767360210418701, 0.1858387589454651, 0.329026997089386, 0.1505919098854065, 0.19304680824279785, 0.47583287954330444, 0.4143006205558777, 0.0767398476600647, 0.267566442489624, 0.283164918422699, 0.057488322257995605, 0.8082854151725769, 0.04956066608428955, 0.30280178785324097, 0.11670255661010742, 0.42691415548324585, 0.418914258480072, 0.2805172801017761, 0.8110893368721008, 0.13264304399490356, 0.4743121266365051, 0.09475940465927124, 0.2489835023880005, 0.46672022342681885, 0.11083751916885376, 0.1246483325958252, 0.0718345046043396, 0.5744244456291199, 0.34055596590042114, 0.361899733543396, 0.052293598651885986, 0.4931318759918213, 0.4345683455467224, 0.16666465997695923, 0.5713855624198914, 0.11552071571350098, 0.23919916152954102, 0.026841342449188232, 0.30219167470932007, 0.10146534442901611, 0.3137941360473633, 0.036045610904693604, 0.5110050439834595, 0.2892168164253235, 0.05114543437957764, 0.3926079273223877, 0.16003239154815674, 0.0040378570556640625, 0.6163381338119507, 0.125352144241333, 0.0653185248374939, 0.028903961181640625, 0.44432181119918823, 0.21331703662872314, 0.5037587881088257, 0.17152363061904907, 0.9704753756523132, 0.21141010522842407, 0.3844830393791199, 0.3203669786453247, 0.8942767381668091, 0.05096715688705444, 0.6624569296836853, 0.22172480821609497, 0.7165933847427368, 0.05851787328720093, 0.037319183349609375, 0.1882837414741516, 0.23806363344192505, 0.5387455224990845, 0.218308687210083, 0.6257885694503784, 0.5199693441390991, 0.0182952880859375, 0.18933147192001343, 0.017720460891723633, 0.05366474390029907, 0.12666696310043335, 0.650072455406189, 0.18975871801376343, 0.7259771227836609, 0.37385112047195435, 0.3898366093635559, 0.46275919675827026, 0.10481125116348267, 0.6875647306442261, 0.5040090084075928, 0.19096875190734863, 0.0064171552658081055, 0.6612942218780518, 0.1930505633354187, 0.1800471544265747, 0.3284098505973816, 0.8811194896697998, 0.10041588544845581, 0.6912360191345215, 0.08695906400680542, 0.15663862228393555, 0.8701098561286926, 0.5395939350128174, 0.024535059928894043, 0.6111119985580444, 0.2854149341583252, 0.636580765247345, 0.1929187774658203, 0.5174077153205872, 0.4872756004333496, 0.4848232865333557, 0.295951783657074, 0.19121813774108887, 0.7860519886016846, 0.08565819263458252, 0.5426350831985474, 0.6391308307647705, 0.5509633421897888, 0.0475199818611145, 0.14443033933639526, 0.6809334754943848, 0.3245496153831482, 0.04382932186126709, 0.8184647560119629, 0.5804588198661804, 0.6103195548057556, 0.10280877351760864, 0.23180580139160156, 0.6357195377349854, 0.5895394086837769, 0.524398922920227, 0.5195617079734802, 0.5099758505821228, 0.8539563417434692, 0.10064232349395752, 0.7928968667984009, 0.08744674921035767, 0.31688106060028076, 0.3125791549682617, 0.5929687023162842, 0.5826447010040283, 0.6673832535743713, 0.05315852165222168, 0.38317668437957764, 0.3286917805671692, 0.21514832973480225, 0.5718242526054382, 0.08901751041412354, 0.42942124605178833, 0.4357909560203552, 0.27276939153671265, 0.24692237377166748, 0.09229511022567749, 0.5111266374588013, 0.42771250009536743, 0.048013925552368164, 0.46266716718673706, 0.032173991203308105, 0.14565086364746094, 0.26095008850097656, 0.18864011764526367, 0.1163945198059082, 0.4598703384399414, 0.387800931930542, 0.027986466884613037, 0.3392242193222046, 0.01941579580307007, 0.0482826828956604, 0.182297945022583, 0.06547677516937256, 0.23239177465438843, 0.4363221526145935, 0.7900422811508179, 0.11551201343536377, 0.09581798315048218, 0.3551144003868103, 0.07443779706954956, 0.5447986125946045, 0.8158504962921143, 0.44558805227279663, 0.4850606918334961, 0.6907020211219788, 0.30518293380737305, 0.7560019493103027, 0.6218165159225464, 0.3463120460510254, 0.9634607434272766, 0.361269474029541, 0.39159518480300903, 0.037536680698394775, 0.046563684940338135, 0.3457469344139099, 0.4843701124191284, 0.6503648161888123, 0.7467344403266907, 0.10614883899688721, 0.5624992251396179, 0.5988109707832336, 0.16954344511032104, 0.3114381432533264, 0.062376976013183594, 0.5153980851173401, 0.19903641939163208, 0.11558973789215088, 0.6855800747871399, 0.10178440809249878, 0.2033172845840454, 0.6448274850845337, 0.15068554878234863, 0.7384257316589355, 0.3501015901565552, 0.9376906156539917, 0.3560726046562195, 0.8843758702278137, 0.3953673243522644, 0.8757843971252441, 0.4984270930290222, 0.47324705123901367, 0.8417021632194519, 0.5509132742881775, 0.6443009972572327, 0.6800321340560913, 0.053815484046936035, 0.041265010833740234, 0.7933430671691895, 0.36544346809387207, 0.034347474575042725, 0.746447741985321, 0.5481712818145752, 0.450813889503479, 0.5365288257598877, 0.037325501441955566, 0.6241016387939453, 0.4143865704536438, 0.4743953347206116, 0.46192651987075806, 0.47382503747940063, 0.11541366577148438, 0.9001601338386536, 0.6903812885284424, 0.5595818161964417, 0.14621669054031372, 0.16057628393173218, 0.10016399621963501, 0.2539352774620056, 0.2663525938987732, 0.5798762440681458, 0.08828181028366089, 0.013869941234588623, 0.06790035963058472, 0.2367786169052124, 0.2839892506599426, 0.26972097158432007, 0.016778290271759033, 0.04204803705215454, 0.3870542049407959, 0.44290977716445923, 0.13969790935516357, 0.03120243549346924, 0.09844833612442017, 0.3751816153526306, 0.5785634517669678, 0.05422395467758179, 0.4040188789367676, 0.42697983980178833, 0.18904322385787964, 0.35412460565567017, 0.14416319131851196, 0.0929727554321289, 0.09292900562286377, 0.1898021697998047, 0.11560261249542236, 0.0413360595703125, 0.09584850072860718, 0.75974440574646, 0.023077964782714844, 0.1363281011581421, 0.03908348083496094, 0.25386732816696167, 0.3672356605529785, 0.01793915033340454, 0.5663045644760132, 0.19661307334899902, 0.5484428405761719, 0.47902393341064453, 0.4694397449493408, 0.5783214569091797, 0.171009361743927, 0.0445711612701416, 0.21603673696517944, 0.24653548002243042, 0.18448203802108765, 0.8103954195976257, 0.2516062259674072, 0.0028171539306640625, 0.44904834032058716, 0.3041396737098694, 0.26637160778045654, 0.5088178515434265, 0.5453086495399475, 0.0423588752746582, 0.672813355922699, 0.6680946350097656, 0.36357688903808594, 0.49212342500686646, 0.4917178750038147, 0.4439328908920288, 0.6727595925331116, 0.1707267165184021, 0.27963078022003174, 0.13617461919784546, 0.5118916630744934, 0.04487091302871704, 0.05390584468841553, 0.3926626443862915, 0.1404362916946411, 0.47842615842819214, 0.40963178873062134, 0.1419278383255005, 0.69240802526474, 0.7493751049041748, 0.7682107090950012, 0.4200354814529419, 0.18735623359680176, 0.8405263423919678, 0.25160157680511475, 0.0977749228477478, 0.29509180784225464, 0.5976454615592957, 0.165601909160614, 0.2255578637123108, 0.3174271583557129, 0.02533632516860962, 0.05015730857849121, 0.3931167721748352, 0.09147202968597412, 0.22162985801696777, 0.3914493918418884, 0.19418543577194214, 0.11030846834182739, 0.5404369831085205, 0.3525848984718323, 0.14649498462677002, 0.380934476852417, 0.22000449895858765, 0.10482519865036011, 0.5551047325134277, 0.6519795656204224, 0.07182198762893677, 0.8046411275863647, 0.07692790031433105, 0.019674241542816162, 0.43769127130508423, 0.14367014169692993, 0.46953415870666504, 0.33671289682388306, 0.6603484153747559, 0.564042866230011, 0.6927578449249268, 0.17367130517959595, 0.14901554584503174, 0.6222423911094666, 0.40384769439697266, 0.4567791819572449, 0.23463726043701172, 0.3414820432662964, 0.002032637596130371, 0.5181748270988464, 0.6019094586372375, 0.02039337158203125, 0.6271494626998901, 0.10304957628250122, 0.8043684959411621, 0.5099406838417053, 0.36814945936203003, 0.0878785252571106, 0.5281661748886108, 0.09674137830734253, 0.3725382685661316, 0.12317562103271484, 0.21489518880844116, 0.38352030515670776, 0.32716310024261475, 0.10560077428817749, 0.05727124214172363, 0.6875538229942322, 0.4950104355812073, 0.032881081104278564, 0.5289701223373413, 0.11219465732574463, 0.44922298192977905, 0.00022679567337036133, 0.8204610347747803, 0.4767046570777893, 0.18294310569763184, 0.37625497579574585, 0.8558797240257263, 0.12550967931747437, 0.39895522594451904, 0.2201576828956604, 0.307964563369751, 0.25312429666519165, 0.32828474044799805, 0.015616178512573242, 0.43728697299957275, 0.8677690029144287, 0.15328574180603027, 0.28409260511398315, 0.4304739236831665, 0.04009634256362915, 0.15424472093582153, 0.12666743993759155, 0.5352054238319397, 0.35430341958999634, 0.021597325801849365, 0.20909804105758667, 0.3214218020439148, 0.40741032361984253, 0.8043124675750732, 0.18141859769821167, 0.10807323455810547, 0.2591501474380493, 0.15845376253128052, 0.00421142578125, 0.24239319562911987, 0.4946223497390747, 0.4900556802749634, 0.12588787078857422, 0.40172678232192993, 0.05835694074630737, 0.24203377962112427, 0.093869149684906, 0.6982458829879761, 0.47339946031570435, 0.24596023559570312, 0.12998497486114502, 0.23999953269958496, 0.682140588760376, 0.3783602714538574, 0.06996238231658936, 0.36953771114349365, 0.4519246816635132, 0.6299076676368713, 0.56969153881073, 0.06902086734771729, 0.49674081802368164, 0.8566563129425049, 0.053069233894348145, 0.5601822137832642, 0.23320120573043823, 0.3122043013572693, 0.0020710229873657227, 0.3560108542442322, 0.016072869300842285, 0.271149218082428, 0.7221117615699768, 0.2988431453704834, 0.48527127504348755, 0.7254210114479065, 0.5292367935180664, 0.09555339813232422, 0.7119138836860657, 0.4936809539794922, 0.48657774925231934, 0.09292334318161011, 0.7529346346855164, 0.012700378894805908, 0.6051537394523621, 0.15225809812545776, 0.09228742122650146, 0.2405034899711609, 0.04890322685241699, 0.08418506383895874, 0.09872180223464966, 0.05310869216918945, 0.15301060676574707, 0.23001617193222046, 0.731022298336029, 0.07017302513122559, 0.30659085512161255, 0.512939989566803, 0.2574564814567566, 0.8043397068977356, 0.08954530954360962, 0.009579122066497803, 0.4032900333404541, 0.1441470980644226, 0.049307048320770264, 0.09479683637619019, 0.2372804880142212, 0.4076639413833618, 0.36010730266571045, 0.6320303678512573, 0.31125032901763916, 0.493094265460968, 0.009772658348083496, 0.05615931749343872, 0.4312780499458313, 0.19074267148971558, 0.6488219499588013, 0.19182217121124268, 0.022167861461639404, 0.07289409637451172, 0.28382986783981323, 0.5480750799179077, 0.16352540254592896, 0.5323532819747925, 0.13144177198410034, 0.8032833337783813, 0.4435575008392334, 0.5943458080291748, 0.4046434760093689, 0.1338481903076172, 0.642723560333252, 0.2705281376838684, 0.06062358617782593, 0.20356637239456177, 0.1770915985107422, 0.3543354272842407, 0.08109939098358154, 0.15594381093978882, 0.49917328357696533, 0.0593562126159668, 0.5914098024368286, 0.4201391935348511, 0.39413923025131226, 0.6592099070549011, 0.7943164706230164, 0.09290647506713867, 0.41368621587753296, 0.18393361568450928, 0.6587798595428467, 0.02369147539138794, 0.23056566715240479, 0.5168402791023254, 0.11000829935073853, 0.48046207427978516, 0.3849050998687744, 0.09823429584503174, 0.02076655626296997, 0.4526553750038147, 0.20131248235702515, 0.07023537158966064, 0.7111674547195435, 0.18360722064971924, 0.9461203813552856, 0.2960425019264221, 0.510141134262085, 0.4694720506668091, 0.5163124799728394, 0.09396994113922119, 0.8022823333740234, 0.4842144250869751, 0.04721033573150635, 0.05752629041671753, 0.2315114140510559, 0.7848222851753235, 0.3323521614074707, 0.23496520519256592, 0.07936620712280273, 0.5757265686988831, 0.5477291345596313, 0.12974131107330322, 0.17837518453598022, 0.5454367995262146, 0.26108449697494507, 0.4404197335243225, 0.7060489058494568, 0.6178244948387146, 0.27264404296875, 0.7225129008293152, 0.8190940022468567, 0.8579466938972473, 0.6120285391807556, 0.35826849937438965, 0.5548704862594604, 0.19113242626190186, 0.03247785568237305, 0.7575440406799316, 0.07867389917373657, 0.004173696041107178, 0.06455248594284058, 0.3849838376045227, 0.12348341941833496, 0.0185621976852417, 0.3063321113586426, 0.29662024974823, 0.7371057868003845, 0.8207443952560425, 0.7282374501228333, 0.38483792543411255, 0.045593321323394775, 0.8665594458580017, 0.2626381516456604, 0.4875808358192444, 0.39435577392578125, 0.31097060441970825, 0.2227874994277954, 0.1693708896636963, 0.16353923082351685, 0.09957617521286011, 0.08510661125183105, 0.35792112350463867, 0.46626073122024536, 0.14794588088989258, 0.0034692883491516113, 0.2749602794647217, 0.5360996723175049, 0.10139787197113037, 0.0886009931564331, 0.4721902012825012, 0.23494374752044678, 0.10645943880081177, 0.26207882165908813, 0.5304669141769409, 0.1746801733970642, 0.2611672282218933, 0.15042728185653687, 0.3083533048629761, 0.30636560916900635, 0.8403701186180115, 0.2809525728225708, 0.13983744382858276, 0.11501872539520264, 0.0017973780632019043, 0.9005544781684875, 0.05843228101730347, 0.2129005789756775, 0.05125570297241211, 0.3936387300491333, 0.6554855108261108, 0.38642096519470215, 0.40233784914016724, 0.6418578624725342, 0.3093051314353943, 0.9243597388267517, 0.23460030555725098, 0.44412732124328613, 0.726547360420227, 0.538227379322052, 0.2190861701965332, 0.11672180891036987, 0.6741816401481628, 0.14088928699493408, 0.4343488812446594, 0.26480621099472046, 0.11230248212814331, 0.2410672903060913, 0.0665251612663269, 0.7719939947128296, 0.25092506408691406, 0.37319910526275635, 0.17356979846954346, 0.7956262230873108, 0.9601469039916992, 0.02128136157989502, 0.8335020542144775, 0.7933542728424072, 0.22445178031921387, 0.13308310508728027, 0.10441911220550537, 0.06161201000213623, 0.7509115934371948, 0.7718603014945984, 0.05219775438308716, 0.29226118326187134, 0.3207699656486511, 0.4681936502456665, 0.012863636016845703, 0.6365417838096619, 0.19541049003601074, 0.10059380531311035, 0.2762368321418762, 0.09904509782791138, 0.39457058906555176, 0.1254062056541443, 0.15775012969970703, 0.3814428448677063, 0.45517218112945557, 0.6045499444007874, 0.6819254159927368, 0.4330875873565674, 0.3433874249458313, 0.13639569282531738, 0.2840309143066406, 0.3515225648880005, 0.04158252477645874, 0.007843554019927979, 0.6807887554168701, 0.5467256307601929, 0.13218361139297485, 0.42027705907821655, 0.25479769706726074, 0.4082931876182556, 0.47196733951568604, 0.17101508378982544, 0.6342776417732239, 0.2317601442337036, 0.44189226627349854, 0.9076133370399475, 0.9180400967597961, 0.011690199375152588, 0.39833056926727295, 0.0005609989166259766, 0.3390648365020752, 0.41300058364868164, 0.6414724588394165, 0.3316735029220581, 0.3001331686973572, 0.3366706967353821, 0.30229902267456055, 0.27496856451034546, 0.2640858292579651, 0.9532109498977661, 0.259790301322937, 0.3607438802719116, 0.05116325616836548, 0.30054938793182373, 0.43289464712142944, 0.23286062479019165, 0.2573661208152771, 0.44646382331848145, 0.7451121807098389, 0.16364502906799316, 0.6224127411842346, 0.4578005075454712, 0.39965754747390747, 0.05683267116546631, 0.6208718419075012, 0.05821138620376587, 0.32798564434051514, 0.48707443475723267, 0.2869986891746521, 0.4187208414077759, 0.011655688285827637, 0.3740299940109253, 0.42373228073120117, 0.18860971927642822, 0.5516403913497925, 0.24804717302322388, 0.3170507550239563, 0.5637247562408447, 0.07159173488616943, 0.1607024073600769, 0.0037250518798828125, 0.8298227190971375, 0.5430229306221008, 0.1600707769393921, 0.35865312814712524, 0.06308436393737793, 0.9023435711860657, 0.47986865043640137, 0.5319557189941406, 0.6413440704345703, 0.2008630633354187, 0.2826836109161377, 0.16388922929763794, 0.2316298484802246, 0.5481144189834595, 0.07716751098632812, 0.11867696046829224, 0.2706097960472107, 0.310737669467926, 0.009884178638458252, 0.4917011857032776, 0.29308295249938965, 0.04246509075164795, 0.12617230415344238, 0.17290574312210083, 0.14830803871154785, 0.03293919563293457, 0.5182862877845764, 0.5490946173667908, 0.4674137234687805, 0.5198853611946106, 0.5060290098190308, 0.17505693435668945, 0.08978497982025146, 0.22690856456756592, 0.5389636754989624, 0.8071083426475525, 0.194768488407135, 0.6723084449768066, 0.0002129077911376953, 0.1215050220489502, 0.053232431411743164, 0.15402072668075562, 0.4596446752548218, 0.2552199363708496, 0.7169804573059082, 0.35970181226730347, 0.3901832103729248, 0.7618016600608826, 0.8549537062644958, 0.1728488802909851, 0.4911412000656128, 0.4123234748840332, 0.4949933886528015, 0.6974943280220032, 0.5655527710914612, 0.0005020499229431152, 0.006240725517272949, 0.4633960723876953, 0.25849246978759766, 0.028696537017822266, 0.07133376598358154, 0.1613561511039734, 0.30140119791030884, 0.4137517213821411, 0.8514267802238464, 0.27327316999435425, 0.7271689772605896, 0.4352349638938904, 0.4996136426925659, 0.6525247693061829, 0.7739745378494263, 0.6652213931083679, 0.16096007823944092, 0.2009933590888977, 0.3682093620300293, 0.11546105146408081, 0.3553527593612671, 0.12301653623580933, 0.2675185203552246, 0.09698253870010376, 0.2811526656150818, 0.27345144748687744, 0.30360299348831177, 0.5482617616653442, 0.003988504409790039, 0.7270441055297852, 0.35493820905685425, 0.11304628849029541, 0.34380435943603516, 0.3563846945762634, 0.25260937213897705, 0.45019102096557617, 0.11753690242767334, 0.709393322467804, 0.43085741996765137, 0.28191035985946655, 0.2694207429885864, 0.5749580264091492, 0.058837950229644775, 0.669383704662323, 0.4301964044570923, 0.7285234332084656, 0.5324608087539673, 0.0621873140335083, 0.7073259353637695, 0.3652573227882385, 0.6894345283508301, 0.03142625093460083, 0.11131179332733154, 0.15699326992034912, 0.10931694507598877, 0.07294553518295288, 0.3351060748100281, 0.26171815395355225, 0.027018487453460693, 0.6065420508384705, 0.5124761462211609, 0.4763762950897217, 0.9516720771789551, 0.2668556571006775, 0.6728116273880005, 0.40608370304107666, 0.3419232964515686, 0.2597762942314148, 0.13059759140014648, 0.516632080078125, 0.19131982326507568, 0.6864405870437622, 0.0029783248901367188, 0.036805152893066406, 0.1376795768737793, 0.4786338806152344, 0.230035662651062, 0.8285560607910156, 0.2751199007034302, 0.5110703110694885, 0.5876664519309998, 0.05175429582595825, 0.06415063142776489, 0.3093796372413635, 0.06494957208633423, 0.308072030544281, 0.1927279233932495, 0.088218092918396, 0.3123265504837036, 0.09498029947280884, 0.18261820077896118, 0.702505886554718, 0.055322885513305664, 0.15776854753494263, 0.3068472743034363, 0.23079347610473633, 0.2492293119430542, 0.19687366485595703, 0.005079030990600586, 0.1004975438117981, 0.520459771156311, 0.12443256378173828, 0.02338939905166626, 0.5544981360435486, 0.5772821307182312, 0.30721884965896606, 0.6282448768615723, 0.24785971641540527, 0.7325736284255981, 0.1396629810333252, 0.542658805847168, 0.11537623405456543, 0.008584737777709961, 0.3331605792045593, 0.6405595541000366, 0.3396303057670593, 0.2332438826560974, 0.13534694910049438, 0.24067062139511108, 0.17847496271133423, 0.14076685905456543, 0.4903019070625305, 0.06933420896530151, 0.3110618591308594, 0.9624053835868835, 0.8690690994262695, 0.14059197902679443, 0.27855128049850464, 0.07407611608505249, 0.22677254676818848, 0.805607795715332, 0.2960318326950073, 0.4781866669654846, 0.28231775760650635, 0.6204461455345154, 0.10143208503723145, 0.26766180992126465, 0.09344291687011719, 0.38264405727386475, 0.048412442207336426, 0.4491886496543884, 0.013138234615325928, 0.5901350378990173, 0.4773637056350708, 0.2378624677658081, 0.022986173629760742, 0.08552402257919312, 0.03624528646469116, 0.07072454690933228, 0.25087636709213257, 0.29094433784484863, 0.12670910358428955, 0.024165630340576172, 0.5856119990348816, 0.18030977249145508, 0.5539964437484741, 0.22293168306350708, 0.5364974141120911, 0.32518255710601807, 0.11779952049255371, 0.2542835474014282, 0.13994050025939941, 0.8827767968177795, 0.22682350873947144, 0.17254972457885742, 0.2132338285446167, 0.942118227481842, 0.06140178442001343, 0.543148934841156, 0.12601125240325928, 0.4517136216163635, 0.4238100051879883, 0.40344351530075073, 0.5656533241271973, 0.11305832862854004, 0.013855814933776855, 0.14681220054626465, 0.8333080410957336, 0.39182740449905396, 0.18613237142562866, 0.11686110496520996, 0.3113676905632019, 0.30137503147125244, 0.4416496157646179, 0.3668098449707031, 0.042053401470184326, 0.1583852767944336, 0.34947288036346436, 0.12874466180801392, 0.276832640171051, 0.5446177124977112, 0.4319975972175598, 0.5529828071594238, 0.23722898960113525, 0.296015202999115, 0.1773783564567566, 0.46936464309692383, 0.6636635065078735, 0.3815215826034546, 0.7301322221755981, 0.11642730236053467, 0.00605010986328125, 0.19231611490249634, 0.11818283796310425, 0.5454174876213074, 0.06986582279205322, 0.2865380048751831, 0.6087097525596619, 0.32713228464126587, 0.3320915102958679, 0.7575031518936157, 0.1895187497138977, 0.15243399143218994, 0.3717639446258545, 0.07552731037139893, 0.5693654417991638, 0.11470949649810791, 0.1137322187423706, 0.4161549210548401, 0.7744149565696716, 0.2812801003456116, 0.22213715314865112, 0.48304373025894165, 0.4897691607475281, 0.2329602837562561, 0.0185738205909729, 0.09990888833999634, 0.47014403343200684, 0.5272682309150696, 0.31227171421051025, 0.5062294006347656, 0.7782036662101746, 0.8980440497398376, 0.829905092716217, 0.5446375608444214, 0.4933363199234009, 0.8368590474128723, 0.07222437858581543, 0.059511542320251465, 0.49194562435150146, 0.31282269954681396, 0.6135741472244263, 0.5330967307090759, 0.712058424949646, 0.3267163038253784, 0.5269837379455566, 0.07038372755050659, 0.3706004023551941, 0.11958938837051392, 0.2913632392883301, 0.45209020376205444, 0.43273794651031494, 0.47202861309051514, 0.04583454132080078, 0.0760200023651123, 0.5704320669174194, 0.3813471794128418, 0.5782755017280579, 0.6098647713661194, 0.8853620290756226, 0.8872771859169006, 0.2199074625968933, 0.375382661819458, 0.08537167310714722, 0.14626538753509521, 0.36314743757247925, 0.008992373943328857, 0.04280167818069458, 0.2280527949333191, 0.24090367555618286, 0.08299881219863892, 0.5745225548744202, 0.038341522216796875, 0.4843384027481079, 0.5148182511329651, 0.3257620334625244, 0.08956515789031982, 0.0186765193939209, 0.1709781289100647, 0.0541645884513855, 0.5148611664772034, 0.5250667929649353, 0.05573582649230957, 0.02109283208847046, 0.9050500392913818, 0.13661664724349976, 0.12662428617477417, 0.48072654008865356, 0.5347782969474792, 0.017534375190734863, 0.15369665622711182, 0.15782326459884644, 0.24164354801177979, 0.7462700009346008, 0.5845717787742615, 0.5250887870788574, 0.39476704597473145, 0.33328360319137573, 0.07457119226455688, 0.35356682538986206, 0.6563432812690735, 0.5333941578865051, 0.14260923862457275, 0.5553343892097473, 0.288250207901001, 0.3138922452926636, 0.5964477062225342, 0.48910677433013916, 0.494462788105011, 0.6388239860534668, 0.20282655954360962, 0.10330307483673096, 0.2852758765220642, 0.7480011582374573, 0.2766209840774536, 0.2729024291038513, 0.1742795705795288, 0.29205435514450073, 0.4831889271736145, 0.6019245982170105, 0.08808410167694092, 0.056100308895111084, 0.6532140374183655, 0.7778228521347046, 0.2747573256492615, 0.12826693058013916, 0.2450631856918335, 0.9787544012069702, 0.9399396777153015, 0.37810665369033813, 0.6194471120834351, 0.26871150732040405, 0.3298867344856262, 0.4669005870819092, 0.30752402544021606, 0.7710262537002563, 0.24180173873901367, 0.3279593586921692, 0.11609810590744019, 0.07920056581497192, 0.5434480905532837, 0.28823381662368774, 0.3897455334663391, 0.38583409786224365, 0.438720703125, 0.2158031463623047, 0.193314790725708, 0.5678521990776062, 0.42875343561172485, 0.014997005462646484, 0.12361305952072144, 0.8378003239631653, 0.8098509907722473, 0.48706793785095215, 0.6539345383644104, 0.6864568591117859, 0.018307268619537354, 0.44205349683761597, 0.335202693939209, 0.4480370283126831, 0.4076692461967468, 0.2685471773147583, 0.2931286692619324, 0.16879451274871826, 0.5854990482330322, 0.07105576992034912, 0.6939573884010315, 0.2835969924926758, 0.306659996509552, 0.24586498737335205, 0.18495899438858032, 0.03308230638504028, 0.19889551401138306, 0.2530475854873657, 0.5948606729507446, 0.0841100811958313, 0.24629008769989014, 0.24304074048995972, 0.05125027894973755, 0.455036461353302, 0.399410605430603, 0.28657859563827515, 0.0365755558013916, 0.9552666544914246, 0.4047531485557556, 0.2217864990234375, 0.1819201111793518, 0.5105131268501282, 0.2521681785583496, 0.20940285921096802, 0.5812300443649292, 0.3111758232116699, 0.6033162474632263, 0.21766000986099243, 0.10079210996627808, 0.4715554714202881, 0.7363464832305908, 0.9000536799430847, 0.10865896940231323, 0.6352744102478027, 0.48450952768325806, 0.16326624155044556, 0.5719397068023682, 0.33896857500076294, 0.6297580599784851, 0.03157162666320801, 0.060497820377349854, 0.6249155402183533, 0.4264891743659973, 0.17771130800247192, 0.2049211859703064, 0.5026881098747253, 0.4041086435317993, 0.3155990242958069, 0.46042442321777344, 0.5289960503578186, 0.48466378450393677, 0.003802657127380371, 0.734025239944458, 0.41216444969177246, 0.42873573303222656, 0.4070087671279907, 0.4633464217185974, 0.2748985290527344, 0.06624245643615723, 0.24917328357696533, 0.8203545808792114, 0.10532963275909424, 0.22372835874557495, 0.11061596870422363, 0.01335364580154419, 0.7841157913208008, 0.3135331869125366, 0.7706400752067566, 0.4137023687362671, 0.0613328218460083, 0.15477609634399414, 0.22255903482437134, 0.10908198356628418, 0.051963746547698975, 0.120838463306427, 0.2049846053123474, 0.02329486608505249, 0.4886857271194458, 0.14322257041931152, 0.15544885396957397, 0.857718288898468, 0.008047699928283691, 0.6019867658615112, 0.4261961579322815, 0.49472296237945557, 0.053785860538482666, 0.6009522676467896, 0.40241092443466187, 0.11582416296005249, 0.9288390278816223, 0.03610175848007202, 0.08880007266998291, 0.0322384238243103, 0.15249699354171753, 0.038570523262023926, 0.13807958364486694, 0.5492897033691406, 0.12595218420028687, 0.2655481696128845, 0.16074800491333008, 0.5255566239356995, 0.01969689130783081, 0.48292386531829834, 0.07415086030960083, 0.37360435724258423, 0.19484078884124756, 0.31348729133605957, 0.40115225315093994, 0.41934460401535034, 0.5828422904014587, 0.17292016744613647, 0.4708860516548157, 0.4873170256614685, 0.28032130002975464, 0.1221703290939331, 0.17079895734786987, 0.8485521078109741, 0.645027220249176, 0.26028478145599365, 0.011142253875732422, 0.1204419732093811, 0.13321971893310547, 0.17909294366836548, 0.05680537223815918, 0.8730046153068542, 0.011570155620574951, 0.012252271175384521, 0.00439375638961792, 0.09577155113220215, 0.11971920728683472, 0.15365546941757202, 0.4081266522407532, 0.671856701374054, 0.2272242307662964, 0.9729939699172974, 0.13416993618011475, 0.9514034390449524, 0.20476889610290527, 0.8467063903808594, 0.8624998331069946, 0.008154988288879395, 0.7814493775367737, 0.3211160898208618, 0.22912907600402832, 0.09857481718063354, 0.5428507328033447, 0.5089612007141113, 0.1036025881767273, 0.09931766986846924, 0.49308353662490845, 0.5443956255912781, 0.16267722845077515, 0.31108832359313965, 0.5050550103187561, 0.6914465427398682, 0.3911697268486023, 0.5052945017814636, 0.22572636604309082, 0.20618784427642822, 0.9319753050804138, 0.2791871428489685, 0.6623499393463135, 0.32288193702697754, 0.5101804733276367, 0.40411555767059326, 0.4186464548110962, 0.06795036792755127, 0.8331884145736694, 0.25347602367401123, 0.3060588240623474, 0.1425284743309021, 0.5921058654785156, 0.6191633343696594, 0.05330091714859009, 0.5915919542312622, 0.42926883697509766, 0.5463163256645203, 0.09882831573486328, 0.8703935742378235, 0.17212718725204468, 0.560085117816925, 0.5180298089981079, 0.006552398204803467, 0.4160342812538147, 0.6944208741188049, 0.30170899629592896, 0.26216787099838257, 0.44540733098983765, 0.4404236674308777, 0.09817224740982056, 0.11939054727554321, 0.8475492000579834, 0.3855680823326111, 0.45059865713119507, 0.3749309182167053, 0.013657748699188232, 0.1659730076789856, 0.018206000328063965, 0.40180593729019165, 0.15106183290481567, 0.7264603972434998, 0.5996129512786865, 0.3723375201225281, 0.7242639064788818, 0.5070078372955322, 0.3738170862197876, 0.7922598719596863, 0.1656133532524109, 0.34695059061050415, 0.5924236178398132, 0.47395217418670654, 0.2012825608253479, 0.11042940616607666, 0.5607585906982422, 0.40082645416259766, 0.48034727573394775, 0.276475191116333, 0.7633900046348572, 0.4488665461540222, 0.4362657070159912, 0.11246699094772339, 0.44683223962783813, 0.08010351657867432, 0.04540860652923584, 0.006120026111602783, 0.4535534381866455, 0.2509012818336487, 0.1184348464012146, 0.17224818468093872, 0.6757148504257202, 0.07424992322921753, 0.4167723059654236, 0.3024432063102722, 0.389337420463562, 0.25154703855514526, 0.3735167384147644, 0.13457077741622925, 0.2699170708656311, 0.4153379797935486, 0.08519601821899414, 0.17856276035308838, 0.7360941171646118, 0.22564303874969482, 0.3115382194519043, 0.3265075087547302, 0.06009763479232788, 0.17080271244049072, 0.401403546333313, 0.21120446920394897, 0.1783604621887207, 0.8828690648078918, 0.46190619468688965, 0.2254355549812317, 0.24993813037872314, 0.2304089069366455, 0.16160327196121216, 0.06898421049118042, 0.30769097805023193, 0.220131516456604, 0.40446484088897705, 0.050266385078430176, 0.3535335063934326, 0.09001034498214722, 0.5916364192962646, 0.5158599019050598, 0.1307356357574463, 0.385423481464386, 0.16237878799438477, 0.3222970962524414, 0.6192646622657776, 0.13938796520233154, 0.37195611000061035, 0.08605211973190308, 0.40409570932388306, 0.299741268157959, 0.0963667631149292, 0.19213122129440308, 0.26063889265060425, 0.5146787762641907, 0.7392054796218872, 0.1671297550201416, 0.20770364999771118, 0.11435425281524658, 0.09096294641494751, 0.7191733717918396, 0.16936874389648438, 0.42740607261657715, 0.6486625671386719, 0.8270893096923828, 0.5809906721115112, 0.6246716976165771, 0.45861583948135376, 0.9470436573028564, 0.3548569083213806, 0.3207874298095703, 0.021084129810333252, 0.36185014247894287, 0.27468013763427734, 0.07639551162719727, 0.0690770149230957, 0.2680494785308838, 0.5059038400650024, 0.21954560279846191, 0.6336624026298523, 0.9179672598838806, 0.8246058821678162, 0.32743269205093384, 0.5472343564033508, 0.5911714434623718, 0.5146954655647278, 0.3890445828437805, 0.2332998514175415, 0.6118850708007812, 0.11111843585968018, 0.13630902767181396, 0.009236752986907959, 0.2643275260925293, 0.3468462824821472, 0.4526636004447937, 0.1056208610534668, 0.147372305393219, 0.7278600335121155, 0.4484856128692627, 0.7907110452651978, 0.20657265186309814, 0.010561823844909668, 0.3082108497619629, 0.11419552564620972, 0.26251786947250366, 0.18806535005569458, 0.4690532684326172, 0.4291032552719116, 0.39053815603256226, 0.5675340890884399, 0.8599393963813782, 0.5239916443824768, 0.7964367866516113, 0.04202145338058472, 0.09130692481994629, 0.6818851828575134, 0.42382293939590454, 0.25620442628860474, 0.4216678738594055, 0.11711013317108154, 0.2801819443702698, 0.6302380561828613, 0.15617585182189941, 0.35209834575653076, 0.420934796333313, 0.734590470790863, 0.3199441432952881, 0.1103353500366211, 0.3919692635536194, 0.6132851243019104, 0.5657588243484497, 0.13140636682510376, 0.461060106754303, 0.4201212525367737, 0.07063907384872437, 0.5030518770217896, 0.48487645387649536, 0.4298018217086792, 0.7722910046577454, 0.4027199149131775, 0.019457638263702393, 0.43825066089630127, 0.10318082571029663, 0.03534477949142456, 0.2516210675239563, 0.7529945969581604, 0.3646126389503479, 0.2582995295524597, 0.46481043100357056, 0.2324429154396057, 0.41475337743759155, 0.32673007249832153, 0.15387386083602905, 0.2195461392402649, 0.11187762022018433, 0.5553264021873474, 0.564135730266571, 0.69325852394104, 0.5027885437011719, 0.07194620370864868, 0.3187366724014282, 0.6031095385551453, 0.3889523148536682, 0.034027814865112305, 0.747592031955719, 0.8164303302764893, 0.3618229031562805, 0.6247137784957886, 0.26561468839645386, 0.6123812198638916, 0.21632057428359985, 0.22323447465896606, 0.5722408294677734, 0.7320860624313354, 0.3650732636451721, 0.04411125183105469, 0.6649846434593201, 0.16830629110336304, 0.646294116973877, 0.2996336817741394, 0.0345805287361145, 0.6613255739212036, 0.4634365439414978, 0.477905809879303, 0.411548912525177, 0.6479406356811523, 0.3397098183631897, 0.30769819021224976, 0.3798587918281555, 0.5222854018211365, 0.7802596688270569, 0.3888269066810608, 0.7058409452438354, 0.3265352249145508, 0.4385838508605957, 0.09966909885406494, 0.20071780681610107, 0.21942627429962158, 0.5690640807151794, 0.05357927083969116, 0.09387212991714478, 0.0544779896736145, 0.43816089630126953, 0.027792811393737793, 0.3224247097969055, 0.14270931482315063, 0.7945313453674316, 0.21440672874450684, 0.06015986204147339, 0.16288971900939941, 0.035520970821380615, 0.06315189599990845, 0.09604382514953613, 0.48363369703292847, 0.30322569608688354, 0.17412960529327393, 0.43114763498306274, 0.48882609605789185, 0.7823113799095154, 0.17230582237243652, 0.17037111520767212, 0.04227721691131592, 0.1215810775756836, 0.35509753227233887, 0.8534300923347473, 0.09913551807403564, 0.2678144574165344, 0.5047406554222107, 0.45782148838043213, 0.4190211296081543, 0.03692835569381714, 0.016016066074371338, 0.6931860446929932, 0.4285564422607422, 0.7104511857032776, 0.21473371982574463, 0.10938626527786255, 0.10819399356842041, 0.25875014066696167, 0.07624918222427368, 0.5425564646720886, 0.5663730502128601, 0.10109853744506836, 0.05958837270736694, 0.5864791870117188, 0.02962130308151245, 0.5196207165718079, 0.14349794387817383, 0.7806845903396606, 0.02334725856781006, 0.021528184413909912, 0.3415066599845886, 0.3269501328468323, 0.2743874192237854, 0.6850181818008423, 0.62904953956604, 0.1944974660873413, 0.32921671867370605, 0.035627126693725586, 0.6391914486885071, 0.23902904987335205, 0.5004402995109558, 0.016301631927490234, 0.07714211940765381, 0.17464661598205566, 0.6698405146598816, 0.6034069061279297, 0.21321099996566772, 0.48118364810943604, 0.5309609174728394, 0.49241185188293457, 0.4200279116630554, 0.5563059449195862, 0.09196233749389648, 0.24555957317352295, 0.26359397172927856, 0.7589033246040344, 0.2757725715637207, 0.5780669450759888, 0.7310447692871094, 0.6352951526641846, 0.6566124558448792, 0.5908995270729065, 0.019656658172607422, 0.881921112537384, 0.12907785177230835, 0.6119362115859985, 0.4233083128929138, 0.045393943786621094, 0.5121017694473267, 0.33478033542633057, 0.22995489835739136, 0.24592357873916626, 0.03750121593475342, 0.3575925827026367, 0.019443988800048828, 0.3931769132614136, 0.6095902323722839, 0.4074116349220276, 0.39138519763946533, 0.15755534172058105, 0.6470179557800293, 0.3479606509208679, 0.09110736846923828, 0.7451764941215515, 0.08057022094726562, 0.1741267442703247, 0.26236939430236816, 0.20514249801635742, 0.16954678297042847, 0.05799269676208496, 0.16123318672180176, 0.691517174243927, 0.11795586347579956, 0.29662781953811646, 0.029714524745941162, 0.5799090266227722, 0.14810311794281006, 0.7594377994537354, 6.413459777832031e-05, 0.5979313254356384, 0.3593432903289795, 0.8243672251701355, 0.9136006832122803, 0.34505748748779297, 0.13724780082702637, 0.17835289239883423, 0.19652551412582397, 0.49142247438430786, 0.2089468240737915, 0.4947367310523987, 0.31148260831832886, 0.6414957046508789, 0.017726600170135498, 0.4093499183654785, 0.35868120193481445, 0.06558042764663696, 0.5306282043457031, 0.17213857173919678, 0.564202070236206, 0.3099623918533325, 0.18605488538742065, 0.6927108764648438, 0.6911493539810181, 0.8270699381828308, 0.13128256797790527, 0.12973153591156006, 0.22880786657333374, 0.21313601732254028, 0.755359411239624, 0.472032368183136, 0.14829576015472412, 0.6007819771766663, 0.7470313906669617, 0.3690316677093506, 0.18936467170715332, 0.1631268858909607, 0.0657796859741211, 0.08562976121902466, 0.4712676405906677, 0.12251335382461548, 0.06614315509796143, 0.5913133025169373, 0.166143000125885, 0.8181393146514893, 0.18893152475357056, 0.40955233573913574, 0.5353978872299194, 0.010307490825653076, 0.7324537038803101, 0.5898097157478333, 0.6094668507575989, 0.3398357629776001, 0.7537925243377686, 0.29473644495010376, 0.046448588371276855, 0.811532199382782, 0.07201439142227173, 0.06922566890716553, 0.36080873012542725, 0.35035037994384766, 0.5687757730484009, 0.4893685579299927, 0.34668391942977905, 0.24991893768310547, 0.021877050399780273, 0.9259808659553528, 0.22770065069198608, 0.5623519420623779, 0.8601018786430359, 0.00372236967086792, 0.6656964421272278, 0.0769350528717041, 0.8478151559829712, 0.0697249174118042, 0.18013513088226318, 0.025135040283203125, 0.6251428723335266, 0.07886070013046265, 0.3207983374595642, 0.09457749128341675, 0.6536886692047119, 0.7151609063148499, 0.5160101056098938, 0.21941518783569336, 0.7676295042037964, 0.411277174949646, 0.7419795989990234, 0.03174591064453125, 0.01051872968673706, 0.4615321755409241, 0.21488404273986816, 0.8502814173698425, 0.12194520235061646, 0.12963002920150757, 0.025432050228118896, 0.23209631443023682, 0.1297587752342224, 0.07565087080001831, 0.718000054359436, 0.021979570388793945, 0.4256899356842041, 0.9552270770072937, 0.5022486448287964, 0.16113555431365967, 0.07495176792144775, 0.4958275556564331, 0.688057005405426, 0.26990824937820435, 0.1860383152961731, 0.14800876379013062, 0.03166913986206055, 0.22138023376464844, 0.04588836431503296, 0.2355860471725464, 0.09660416841506958, 0.41956400871276855, 0.15564632415771484, 0.039067625999450684, 0.3233441114425659, 0.20903348922729492, 0.7038394212722778, 0.4475710988044739, 0.0011953115463256836, 0.5410506725311279, 0.365703284740448, 0.22097909450531006, 0.18256479501724243, 0.18246173858642578, 0.24652236700057983, 0.468669056892395, 0.2924607992172241, 0.9117817878723145, 0.35779237747192383, 0.07276511192321777, 0.35862547159194946, 0.5797829031944275, 0.0015131831169128418, 0.1680014729499817, 0.4581589698791504, 0.14889150857925415, 0.0943993330001831, 0.22005337476730347, 0.28561949729919434, 0.05962580442428589, 0.38333964347839355, 0.10493612289428711, 0.4945560097694397, 0.08989262580871582, 0.3203442096710205, 0.3127908706665039, 0.17626583576202393, 0.5551700592041016, 0.782254159450531, 0.3008662462234497, 0.5071941614151001, 0.49398136138916016, 0.2924402952194214, 0.1938815712928772, 0.3028528094291687, 0.09650063514709473, 0.6556767821311951, 0.21809375286102295, 0.642317533493042, 0.40842586755752563, 0.26843196153640747, 0.11508488655090332, 0.2967762351036072, 0.8373165726661682, 0.1781449317932129, 0.28189122676849365, 0.7254354953765869, 0.028726577758789062, 0.04408615827560425, 0.0376855731010437, 0.5223618149757385, 0.10691440105438232, 0.19221460819244385, 0.21518737077713013, 0.8415722250938416, 0.5755289196968079, 0.09120798110961914, 0.2176072597503662, 0.40355461835861206, 0.15814530849456787, 0.6356699466705322, 0.46158015727996826, 0.08123832941055298, 0.7789246439933777, 0.05584228038787842, 0.8565368056297302, 0.6424835324287415, 0.2241840958595276, 0.10693114995956421, 0.3518039584159851, 0.4712960124015808, 0.8191429376602173, 0.20376896858215332, 0.2549629211425781, 0.010226845741271973, 0.6123289465904236, 0.03468972444534302, 0.07374978065490723, 0.22051143646240234, 0.10451668500900269, 0.7549601197242737, 0.4276948571205139, 0.7942964434623718, 0.04604595899581909, 0.3214654326438904, 0.3447294235229492, 0.15550798177719116, 0.28153228759765625, 0.2834131717681885, 0.8720353841781616, 0.23199784755706787, 0.6316864490509033, 0.7914608716964722, 0.1469022035598755, 0.03846520185470581, 0.053808629512786865, 0.27156931161880493, 0.31141066551208496, 0.1955024003982544, 0.1002737283706665, 0.09505319595336914, 0.18604159355163574, 0.4814155101776123, 0.03199589252471924, 0.19942057132720947, 0.1028738021850586, 0.2931482791900635, 0.017354071140289307, 0.6215406060218811, 0.290305495262146, 0.6373586058616638, 0.13026970624923706, 0.31869322061538696, 0.14661043882369995, 0.7700682878494263, 0.08576267957687378, 0.6305412650108337, 0.1937730312347412, 0.3215787410736084, 0.6710940599441528, 0.26463353633880615, 0.09156006574630737, 0.5296944379806519, 0.3893579840660095, 0.13153642416000366, 0.6329454183578491, 0.30270153284072876, 0.7161217331886292, 0.30280447006225586, 0.0040934085845947266, 0.08058524131774902, 0.5918104648590088, 0.07058066129684448, 0.8414983749389648, 0.5183635354042053, 0.22315114736557007, 0.512227475643158, 0.4085155725479126, 0.16280853748321533, 0.26759886741638184, 0.14292758703231812, 0.0166548490524292, 0.7349134683609009, 0.26225656270980835, 0.0044203996658325195, 0.3654634356498718, 0.8081619739532471, 0.10229897499084473, 0.7330383658409119, 0.1928340196609497, 0.3145204782485962, 0.06555444002151489, 0.7637456655502319, 0.14762645959854126, 0.1578344702720642, 0.1876956820487976, 0.3987894654273987, 0.7241130471229553, 0.17080026865005493, 0.26217663288116455, 0.18361413478851318, 0.5358771085739136, 0.35429847240448, 0.253506064414978, 0.3428184986114502, 0.3108069896697998, 0.4428161382675171, 0.24628478288650513, 0.5690202116966248, 0.14761847257614136, 0.022307157516479492, 0.1046486496925354, 0.5127969980239868, 0.5528581738471985, 0.6743484139442444, 0.2758582830429077, 0.7297096848487854, 0.0025466084480285645, 0.5158884525299072, 0.41958099603652954, 0.45160186290740967, 0.4555079936981201, 0.370349645614624, 0.30466800928115845, 0.11714249849319458, 0.2722078561782837, 0.08203160762786865, 0.5792084336280823, 0.863732099533081, 0.5030187964439392, 0.3406268358230591, 0.3497735261917114, 0.029779672622680664, 0.41023099422454834, 0.27835357189178467, 0.10278737545013428, 0.2752060294151306, 0.019218623638153076, 0.0349690318107605, 0.06369566917419434, 0.43427449464797974, 0.8241938352584839, 0.7843455672264099, 0.3138315677642822, 0.6505194306373596, 0.017313361167907715, 0.3671573996543884, 0.6078795194625854, 0.6992712616920471, 0.4755609631538391, 0.6279252767562866, 0.07772397994995117, 0.208656907081604, 0.6231323480606079, 0.5438507199287415, 0.7834393978118896, 0.11290425062179565, 0.02400815486907959, 0.290407657623291, 0.038507699966430664, 0.06297034025192261, 0.1797773838043213, 0.19098645448684692, 0.1550329327583313, 0.6972036361694336, 0.016935288906097412, 0.5109255313873291, 0.3585333228111267, 0.5607963800430298, 0.2448464035987854, 0.3727805018424988, 0.04692697525024414, 0.4325844645500183, 0.4648275375366211, 0.5250178575515747, 0.797009289264679, 0.5488905906677246, 0.7394084334373474, 0.663002073764801, 0.02822268009185791, 0.050630033016204834, 0.8919333219528198, 0.8354225754737854, 0.054397404193878174, 0.12859797477722168, 0.06512492895126343, 0.3643966317176819, 0.07464832067489624, 0.03808021545410156, 0.23847979307174683, 0.3080137372016907, 0.5754142999649048, 0.24018871784210205, 0.019432902336120605, 0.4500381350517273, 0.22185516357421875, 0.374397873878479, 0.027028441429138184, 0.46189558506011963, 0.5490757822990417, 0.15334171056747437, 0.19738221168518066, 0.365331768989563, 0.01832568645477295, 0.6511465907096863, 0.25356441736221313, 0.5638230443000793, 0.10134947299957275, 0.6845502257347107, 0.24892908334732056, 0.6289352774620056, 0.6200994253158569, 0.2989159822463989, 0.05814081430435181, 0.5485873222351074, 0.08347409963607788, 0.07953375577926636, 0.2764895558357239, 0.4830162525177002, 0.41859978437423706, 0.08654546737670898, 0.058752477169036865, 0.16295987367630005, 0.5602502226829529, 0.7356219291687012, 0.21640121936798096, 0.21338003873825073, 0.34920817613601685, 0.49196362495422363, 0.48389309644699097, 0.21382629871368408, 0.3940278887748718, 0.5104437470436096, 0.09397298097610474, 0.20074009895324707, 0.021467208862304688, 0.2849028706550598, 0.21196675300598145, 0.6394050717353821, 0.4992713928222656, 0.10767114162445068, 0.7732418179512024, 0.04168933629989624, 0.37779897451400757, 0.001138925552368164, 0.39945828914642334, 0.4265255928039551, 0.19250673055648804, 0.17926281690597534, 0.18749397993087769, 0.9067512154579163, 0.4057931900024414, 0.5132089257240295, 0.2205829620361328, 0.20194679498672485, 0.28657907247543335, 0.19198524951934814, 0.4072108268737793, 0.07380926609039307, 0.4539884924888611, 0.21657806634902954, 0.23233187198638916, 0.5502214431762695, 0.055903613567352295, 0.1003769040107727, 0.0411912202835083, 0.35829097032546997, 0.035329580307006836, 0.030722618103027344, 0.46487462520599365, 0.527653157711029, 0.04275166988372803, 0.2921261191368103, 0.1518239974975586, 0.2767321467399597, 0.7140582203865051, 0.6213268637657166, 0.0958978533744812, 0.1899172067642212, 0.37689292430877686, 0.5315240025520325, 0.1367877721786499, 0.4800528883934021, 0.42286378145217896, 0.16338247060775757, 0.03872692584991455, 0.8440675139427185, 0.43657469749450684, 0.11594361066818237, 0.6743423342704773, 0.03787511587142944, 0.08569449186325073, 0.25823521614074707, 0.5654086470603943, 0.18191558122634888, 0.09639602899551392, 0.29728567600250244, 0.44324547052383423, 0.39569032192230225, 0.07451200485229492, 0.5544469952583313, 0.018573462963104248, 0.44822800159454346, 0.6867443323135376, 0.3012397885322571, 0.4849039316177368, 0.5892612338066101, 0.08819466829299927, 0.0002923011779785156, 0.28829991817474365, 0.34176987409591675, 0.2550986409187317, 0.7649021148681641, 0.08964699506759644, 0.28280162811279297, 0.37256771326065063, 0.7562962770462036, 0.2441709041595459, 0.4062206745147705, 0.11076360940933228, 0.5474088191986084, 0.028925538063049316, 0.6218035221099854, 0.0307961106300354, 0.1464707851409912, 0.22842204570770264, 0.6444474458694458, 0.267423152923584, 0.13695216178894043, 0.4385347366333008, 0.2428695559501648, 0.021092712879180908, 0.12941449880599976, 0.4712602496147156, 0.06342804431915283, 0.3463021516799927, 0.1831478476524353, 0.03634756803512573, 0.18317896127700806, 0.5284990668296814, 0.6781812310218811, 0.5446228981018066, 0.21823078393936157, 0.6065155863761902, 0.2338007092475891, 0.947064995765686, 0.6690632700920105, 0.4933236837387085, 0.5638110041618347, 0.3558257222175598, 0.7164060473442078, 0.10419571399688721, 0.5769186615943909, 0.027950823307037354, 0.7016804814338684, 0.803719162940979, 0.7044475078582764, 0.38239234685897827, 0.028981387615203857, 0.0030589699745178223, 0.09748023748397827, 0.037674009799957275, 0.1026991605758667, 0.6479988098144531, 0.16080844402313232, 0.3413265347480774, 0.3287089467048645, 0.6912689805030823, 0.153927743434906, 0.28103888034820557, 0.799753725528717, 0.14033275842666626, 0.10190880298614502, 0.2873881459236145, 0.45867919921875, 0.773452877998352, 0.5918351411819458, 0.20915043354034424, 0.015092074871063232, 0.13721001148223877, 0.6975832581520081, 0.6379479765892029, 0.14767789840698242, 0.004650413990020752, 0.19862300157546997, 0.003512084484100342, 0.5593728423118591, 0.1987842321395874, 0.373421311378479, 0.4653216004371643, 0.6521323323249817, 0.12914735078811646, 0.20756977796554565, 0.5374062657356262, 0.5379031300544739, 0.16656476259231567, 0.17952007055282593, 0.7681586742401123, 0.6575894951820374, 0.2839244604110718, 0.8436420559883118, 0.20782333612442017, 0.15712034702301025, 0.49544674158096313, 0.2016870379447937, 0.23866987228393555, 0.6732680797576904, 0.30749815702438354, 0.17303001880645752, 0.1339218020439148, 0.4078468680381775, 0.24697273969650269, 0.024512827396392822, 0.011669814586639404, 0.3843446969985962, 0.11468154191970825, 0.6986463069915771, 0.08853232860565186, 0.7049309015274048, 0.9886549115180969, 0.643403947353363, 0.3402175307273865, 0.10676676034927368, 0.8311334848403931, 0.5046454668045044, 0.4171862006187439, 0.16737794876098633, 0.05114680528640747, 0.7659618258476257, 0.2329654097557068, 0.08657097816467285, 0.13733363151550293, 0.07161158323287964, 0.4735170602798462, 0.14617210626602173, 0.011621654033660889, 0.39854562282562256, 0.3010218143463135, 0.35303807258605957, 0.9579369425773621, 0.3996986150741577, 0.8471337556838989, 0.5315296053886414, 0.2858671545982361, 0.7515742778778076, 0.4318317770957947, 0.15464264154434204, 0.7281470894813538, 0.4562219977378845, 0.22652530670166016, 0.7974570393562317, 0.32122719287872314, 0.20157283544540405, 0.10792684555053711, 0.1785905361175537, 0.11082625389099121, 0.2989349365234375, 0.6565253734588623, 0.5667581558227539, 0.5593482255935669, 0.3497559428215027, 0.36583536863327026, 0.2579857110977173, 0.12199223041534424, 0.4211299419403076, 0.049498677253723145, 0.22092700004577637, 0.7688868045806885, 0.04511451721191406, 0.3239735960960388, 0.1511726975440979, 0.48921751976013184, 0.31671738624572754, 0.7234959006309509, 0.2368706464767456, 0.18753200769424438, 0.2554265260696411, 0.8202657103538513, 0.09791702032089233, 0.5624351501464844, 0.8053689002990723, 0.3402720093727112, 0.14641517400741577, 0.3376607894897461, 0.4805832505226135, 0.7678685188293457, 0.31587594747543335, 0.6130484938621521, 0.3343844413757324, 0.309001088142395, 0.08638012409210205, 0.41468894481658936, 0.48839157819747925, 0.426566481590271, 0.06791001558303833, 0.3119003176689148, 0.11637240648269653, 0.36319971084594727, 0.47048550844192505, 0.3249225616455078, 0.6913303136825562, 0.01839423179626465]
on_border = [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]